In [ ]:
## This file implements neural networks before and after lasso selection for p0040presabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0040presabs_qual.csv')
df.shape

(253, 351)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
      ..
248    2
249    1
250    1
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,group_2236,group_8588,group_8800,group_10077,group_1598,group_8093,group_8182,group_8892,group_9858,pheno
0,107,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,109,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,115,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
3,120335,0,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,2
4,120337,0,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,2


In [6]:
df['pheno'].value_counts()

2    95
1    94
0    64
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 350)

In [9]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,TTTCAGCGACTT,...,group_2236,group_8588,group_8800,group_10077,group_1598,group_8093,group_8182,group_8892,group_9858,pheno
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,1,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,2
3,0,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,2
4,0,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,0,0,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 350) (253,)


In [11]:
############# Fully-Connected Neural Network ################

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [14]:
dat = pd.DataFrame(X_test.iloc[:,0])
dat['test'] = y_test

In [15]:
dat

,id,test
221,NRS266,2
177,NRS209,1
223,NRS272,2
49,CFBREBSa114,2
53,CFBREBSa119,1
...,...,...
162,NRS187,1
237,SR1287,0
84,CFBRSa50,0
168,NRS196,2


In [16]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [17]:
#### neural network on over-sampling data
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 651us/step - loss: 1.2077 - accuracy: 0.3051 - val_loss: 1.0494 - val_accuracy: 0.4211
Epoch 2/1000
177/177 [==============================] - 0s 97us/step - loss: 1.0270 - accuracy: 0.4068 - val_loss: 0.9747 - val_accuracy: 0.5000
Epoch 3/1000
177/177 [==============================] - 0s 125us/step - loss: 0.9792 - accuracy: 0.4576 - val_loss: 0.9440 - val_accuracy: 0.5789
Epoch 4/1000
177/177 [==============================] - 0s 124us/step - loss: 0.9537 - accuracy: 0.5198 - val_loss: 0.9238 - val_accuracy: 0.5658
Epoch 5/1000
177/177 [==============================] - 0s 128us/step - loss: 0.9206 - accuracy: 0.5650 - val_loss: 0.9235 - val_accuracy: 0.5526
Epoch 6/1000
177/177 [==============================] - 0s 126us/step - loss: 0.9010 - accuracy: 0.5650 - val_loss: 0.9064 - val_accuracy: 0.5789
Epoch 7/1000
177/177 [==============================] - 0s 114us/step - loss: 0.

Epoch 112/1000
177/177 [==============================] - 0s 100us/step - loss: 0.2055 - accuracy: 0.9492 - val_loss: 1.0396 - val_accuracy: 0.6579
Epoch 113/1000
177/177 [==============================] - 0s 134us/step - loss: 0.2017 - accuracy: 0.9492 - val_loss: 1.0416 - val_accuracy: 0.6053
Epoch 114/1000
177/177 [==============================] - 0s 147us/step - loss: 0.2014 - accuracy: 0.9492 - val_loss: 1.0743 - val_accuracy: 0.6711
Epoch 115/1000
177/177 [==============================] - 0s 130us/step - loss: 0.1967 - accuracy: 0.9435 - val_loss: 1.0397 - val_accuracy: 0.6053
Epoch 116/1000
177/177 [==============================] - 0s 143us/step - loss: 0.1967 - accuracy: 0.9605 - val_loss: 1.0287 - val_accuracy: 0.6184
Epoch 117/1000
177/177 [==============================] - 0s 127us/step - loss: 0.1912 - accuracy: 0.9435 - val_loss: 1.0751 - val_accuracy: 0.6579
Epoch 118/1000
177/177 [==============================] - 0s 130us/step - loss: 0.1900 - accuracy: 0.9661 - val_

177/177 [==============================] - 0s 96us/step - loss: 0.0796 - accuracy: 0.9831 - val_loss: 1.4243 - val_accuracy: 0.6316
Epoch 223/1000
177/177 [==============================] - 0s 101us/step - loss: 0.0835 - accuracy: 0.9774 - val_loss: 1.4329 - val_accuracy: 0.6053
Epoch 224/1000
177/177 [==============================] - 0s 88us/step - loss: 0.0761 - accuracy: 0.9887 - val_loss: 1.4505 - val_accuracy: 0.6447
Epoch 225/1000
177/177 [==============================] - 0s 89us/step - loss: 0.0797 - accuracy: 0.9887 - val_loss: 1.4421 - val_accuracy: 0.6316
Epoch 226/1000
177/177 [==============================] - 0s 92us/step - loss: 0.0817 - accuracy: 0.9831 - val_loss: 1.4536 - val_accuracy: 0.5789
Epoch 227/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0754 - accuracy: 0.9887 - val_loss: 1.4441 - val_accuracy: 0.6316
Epoch 228/1000
177/177 [==============================] - 0s 117us/step - loss: 0.0805 - accuracy: 0.9774 - val_loss: 1.4300 - val_a

Epoch 333/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0444 - accuracy: 0.9831 - val_loss: 1.7456 - val_accuracy: 0.6184
Epoch 334/1000
177/177 [==============================] - 0s 83us/step - loss: 0.0413 - accuracy: 0.9887 - val_loss: 1.7560 - val_accuracy: 0.6184
Epoch 335/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0416 - accuracy: 0.9887 - val_loss: 1.7628 - val_accuracy: 0.6184
Epoch 336/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0406 - accuracy: 0.9944 - val_loss: 1.7799 - val_accuracy: 0.6579
Epoch 337/1000
177/177 [==============================] - 0s 103us/step - loss: 0.0413 - accuracy: 0.9887 - val_loss: 1.7811 - val_accuracy: 0.6447
Epoch 338/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0407 - accuracy: 0.9887 - val_loss: 1.7739 - val_accuracy: 0.6316
Epoch 339/1000
177/177 [==============================] - 0s 90us/step - loss: 0.0432 - accuracy: 0.9887 - val_loss:

177/177 [==============================] - 0s 127us/step - loss: 0.0280 - accuracy: 0.9944 - val_loss: 2.0464 - val_accuracy: 0.6316
Epoch 444/1000
177/177 [==============================] - 0s 128us/step - loss: 0.0259 - accuracy: 0.9944 - val_loss: 2.0246 - val_accuracy: 0.6184
Epoch 445/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0330 - accuracy: 0.9887 - val_loss: 2.0008 - val_accuracy: 0.6447
Epoch 446/1000
177/177 [==============================] - 0s 103us/step - loss: 0.0308 - accuracy: 0.9944 - val_loss: 2.0421 - val_accuracy: 0.6053
Epoch 447/1000
177/177 [==============================] - 0s 92us/step - loss: 0.0284 - accuracy: 0.9944 - val_loss: 2.0749 - val_accuracy: 0.6053
Epoch 448/1000
177/177 [==============================] - 0s 101us/step - loss: 0.0274 - accuracy: 0.9944 - val_loss: 2.0377 - val_accuracy: 0.6316
Epoch 449/1000
177/177 [==============================] - 0s 93us/step - loss: 0.0257 - accuracy: 0.9887 - val_loss: 2.0198 - va

177/177 [==============================] - 0s 122us/step - loss: 0.0200 - accuracy: 0.9944 - val_loss: 2.2633 - val_accuracy: 0.6316
Epoch 554/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0245 - accuracy: 0.9887 - val_loss: 2.2633 - val_accuracy: 0.5921
Epoch 555/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0195 - accuracy: 0.9944 - val_loss: 2.2654 - val_accuracy: 0.6316
Epoch 556/1000
177/177 [==============================] - 0s 141us/step - loss: 0.0206 - accuracy: 0.9944 - val_loss: 2.2699 - val_accuracy: 0.6316
Epoch 557/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0193 - accuracy: 0.9944 - val_loss: 2.2685 - val_accuracy: 0.6316
Epoch 558/1000
177/177 [==============================] - 0s 110us/step - loss: 0.0197 - accuracy: 0.9944 - val_loss: 2.2617 - val_accuracy: 0.5921
Epoch 559/1000
177/177 [==============================] - 0s 139us/step - loss: 0.0215 - accuracy: 0.9944 - val_loss: 2.2803 - 

Epoch 664/1000
177/177 [==============================] - 0s 108us/step - loss: 0.0163 - accuracy: 0.9944 - val_loss: 2.4609 - val_accuracy: 0.6316
Epoch 665/1000
177/177 [==============================] - 0s 123us/step - loss: 0.0147 - accuracy: 0.9944 - val_loss: 2.4736 - val_accuracy: 0.6053
Epoch 666/1000
177/177 [==============================] - 0s 100us/step - loss: 0.0157 - accuracy: 0.9944 - val_loss: 2.4844 - val_accuracy: 0.6053
Epoch 667/1000
177/177 [==============================] - 0s 83us/step - loss: 0.0166 - accuracy: 0.9887 - val_loss: 2.4409 - val_accuracy: 0.6447
Epoch 668/1000
177/177 [==============================] - 0s 92us/step - loss: 0.0174 - accuracy: 0.9944 - val_loss: 2.4466 - val_accuracy: 0.6316
Epoch 669/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0165 - accuracy: 0.9944 - val_loss: 2.5114 - val_accuracy: 0.6053
Epoch 670/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0164 - accuracy: 0.9944 - val_los

Epoch 775/1000
177/177 [==============================] - 0s 196us/step - loss: 0.0108 - accuracy: 0.9944 - val_loss: 2.6831 - val_accuracy: 0.6316
Epoch 776/1000
177/177 [==============================] - 0s 104us/step - loss: 0.0173 - accuracy: 0.9944 - val_loss: 2.6587 - val_accuracy: 0.6447
Epoch 777/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0137 - accuracy: 0.9944 - val_loss: 2.6500 - val_accuracy: 0.6053
Epoch 778/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0144 - accuracy: 0.9944 - val_loss: 2.6672 - val_accuracy: 0.6053
Epoch 779/1000
177/177 [==============================] - 0s 106us/step - loss: 0.0134 - accuracy: 0.9944 - val_loss: 2.6758 - val_accuracy: 0.6184
Epoch 780/1000
177/177 [==============================] - 0s 119us/step - loss: 0.0128 - accuracy: 0.9887 - val_loss: 2.6482 - val_accuracy: 0.6447
Epoch 781/1000
177/177 [==============================] - 0s 116us/step - loss: 0.0181 - accuracy: 0.9944 - val_l

177/177 [==============================] - 0s 122us/step - loss: 0.0122 - accuracy: 0.9944 - val_loss: 2.8293 - val_accuracy: 0.6447
Epoch 886/1000
177/177 [==============================] - 0s 136us/step - loss: 0.0113 - accuracy: 0.9944 - val_loss: 2.8407 - val_accuracy: 0.6316
Epoch 887/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0112 - accuracy: 0.9887 - val_loss: 2.8220 - val_accuracy: 0.6316
Epoch 888/1000
177/177 [==============================] - 0s 106us/step - loss: 0.0116 - accuracy: 0.9887 - val_loss: 2.8068 - val_accuracy: 0.6316
Epoch 889/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0118 - accuracy: 0.9944 - val_loss: 2.8251 - val_accuracy: 0.6316
Epoch 890/1000
177/177 [==============================] - 0s 121us/step - loss: 0.0159 - accuracy: 0.9887 - val_loss: 2.7985 - val_accuracy: 0.6447
Epoch 891/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0157 - accuracy: 0.9887 - val_loss: 2.8072 - v

177/177 [==============================] - 0s 132us/step - loss: 0.0099 - accuracy: 0.9944 - val_loss: 2.9436 - val_accuracy: 0.6316
Epoch 996/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0103 - accuracy: 0.9887 - val_loss: 2.9575 - val_accuracy: 0.6316
Epoch 997/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0112 - accuracy: 0.9944 - val_loss: 2.9731 - val_accuracy: 0.6316
Epoch 998/1000
177/177 [==============================] - 0s 168us/step - loss: 0.0124 - accuracy: 0.9887 - val_loss: 2.9729 - val_accuracy: 0.6447
Epoch 999/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0134 - accuracy: 0.9887 - val_loss: 2.9751 - val_accuracy: 0.6316
Epoch 1000/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0106 - accuracy: 0.9944 - val_loss: 2.9750 - val_accuracy: 0.6316


In [36]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

76/76 [==============================] - 0s 120us/step
test accuracy: 59.21%


In [20]:
pred = model1.predict_classes(X_test)
pred

array([1, 0, 1, 2, 2, 0, 2, 0, 2, 2, 1, 2, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1,
       1, 2, 1, 0, 2, 2, 2, 1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 0,
       2, 1, 1, 1, 0, 2, 0, 1, 1, 2, 1, 0, 2, 2, 0, 2, 0, 1, 0, 1, 2, 2,
       2, 1, 1, 1, 2, 0, 1, 0, 2, 0])

In [21]:
dat['pred'] = pred
dat

,id,test,pred
221,NRS266,2,1
177,NRS209,1,0
223,NRS272,2,1
49,CFBREBSa114,2,2
53,CFBREBSa119,1,2
...,...,...,...
162,NRS187,1,0
237,SR1287,0,1
84,CFBRSa50,0,0
168,NRS196,2,2


In [22]:
proba1 = model1.predict_proba(X_test)
dat_proba1 = pd.DataFrame(proba1)

In [23]:
dat_proba1

,0,1,2
0,4.589014e-11,9.967214e-01,3.278552e-03
1,9.965531e-01,3.446955e-03,1.366631e-11
2,1.275390e-06,9.999987e-01,4.401492e-08
3,5.942705e-03,3.106124e-03,9.909512e-01
4,1.154088e-09,1.283615e-03,9.987164e-01
...,...,...,...
71,5.719361e-01,3.881656e-01,3.989832e-02
72,1.966408e-01,8.033592e-01,5.650785e-16
73,9.999391e-01,5.956309e-05,1.273303e-06
74,1.395339e-15,1.519872e-12,1.000000e+00


In [24]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [25]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p40p.csv", index = False,
         header=None)

In [40]:
hist1 = model1.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 136us/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 2.7611 - val_accuracy: 0.5789
Epoch 2/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0240 - accuracy: 0.9944 - val_loss: 2.7862 - val_accuracy: 0.5789
Epoch 3/1000
177/177 [==============================] - 0s 168us/step - loss: 0.0141 - accuracy: 0.9887 - val_loss: 2.7507 - val_accuracy: 0.6184
Epoch 4/1000
177/177 [==============================] - 0s 167us/step - loss: 0.0123 - accuracy: 0.9944 - val_loss: 2.6734 - val_accuracy: 0.6053
Epoch 5/1000
177/177 [==============================] - 0s 125us/step - loss: 0.0109 - accuracy: 0.9944 - val_loss: 2.6705 - val_accuracy: 0.6053
Epoch 6/1000
177/177 [==============================] - 0s 134us/step - loss: 0.0125 - accuracy: 0.9944 - val_loss: 2.7264 - val_accuracy: 0.5789
Epoch 7/1000
177/177 [==============================] - 0s 129us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 141us/step - loss: 0.0114 - accuracy: 0.9944 - val_loss: 2.8688 - val_accuracy: 0.5789
Epoch 113/1000
177/177 [==============================] - 0s 125us/step - loss: 0.0106 - accuracy: 0.9944 - val_loss: 2.8896 - val_accuracy: 0.6053
Epoch 114/1000
177/177 [==============================] - 0s 126us/step - loss: 0.0095 - accuracy: 0.9944 - val_loss: 2.8718 - val_accuracy: 0.6053
Epoch 115/1000
177/177 [==============================] - 0s 140us/step - loss: 0.0098 - accuracy: 0.9944 - val_loss: 2.8388 - val_accuracy: 0.5921
Epoch 116/1000
177/177 [==============================] - 0s 189us/step - loss: 0.0094 - accuracy: 0.9944 - val_loss: 2.8349 - val_accuracy: 0.5921
Epoch 117/1000
177/177 [==============================] - 0s 116us/step - loss: 0.0091 - accuracy: 0.9944 - val_loss: 2.8520 - val_accuracy: 0.5921
Epoch 118/1000
177/177 [==============================] - 0s 133us/step - loss: 0.0112 - accuracy: 0.9944 - val_

177/177 [==============================] - 0s 157us/step - loss: 0.0105 - accuracy: 0.9944 - val_loss: 2.9548 - val_accuracy: 0.5921
Epoch 223/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0085 - accuracy: 0.9944 - val_loss: 2.9736 - val_accuracy: 0.5789
Epoch 224/1000
177/177 [==============================] - 0s 142us/step - loss: 0.0078 - accuracy: 0.9944 - val_loss: 3.0202 - val_accuracy: 0.5658
Epoch 225/1000
177/177 [==============================] - 0s 194us/step - loss: 0.0144 - accuracy: 0.9944 - val_loss: 3.0443 - val_accuracy: 0.5789
Epoch 226/1000
177/177 [==============================] - 0s 209us/step - loss: 0.0133 - accuracy: 0.9944 - val_loss: 3.0299 - val_accuracy: 0.5789
Epoch 227/1000
177/177 [==============================] - 0s 139us/step - loss: 0.0088 - accuracy: 0.9944 - val_loss: 2.9945 - val_accuracy: 0.5789
Epoch 228/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0094 - accuracy: 0.9944 - val_loss: 2.9796 - 

177/177 [==============================] - 0s 132us/step - loss: 0.0112 - accuracy: 0.9887 - val_loss: 3.1378 - val_accuracy: 0.5789
Epoch 333/1000
177/177 [==============================] - 0s 160us/step - loss: 0.0081 - accuracy: 0.9944 - val_loss: 3.1496 - val_accuracy: 0.5526
Epoch 334/1000
177/177 [==============================] - 0s 191us/step - loss: 0.0117 - accuracy: 0.9944 - val_loss: 3.1658 - val_accuracy: 0.5526
Epoch 335/1000
177/177 [==============================] - 0s 170us/step - loss: 0.0096 - accuracy: 0.9944 - val_loss: 3.1607 - val_accuracy: 0.5789
Epoch 336/1000
177/177 [==============================] - 0s 165us/step - loss: 0.0085 - accuracy: 0.9944 - val_loss: 3.1479 - val_accuracy: 0.5789
Epoch 337/1000
177/177 [==============================] - 0s 128us/step - loss: 0.0093 - accuracy: 0.9944 - val_loss: 3.1342 - val_accuracy: 0.5658
Epoch 338/1000
177/177 [==============================] - 0s 127us/step - loss: 0.0107 - accuracy: 0.9887 - val_loss: 3.1423 - 

177/177 [==============================] - 0s 229us/step - loss: 0.0142 - accuracy: 0.9944 - val_loss: 3.3284 - val_accuracy: 0.5526
Epoch 443/1000
177/177 [==============================] - 0s 199us/step - loss: 0.0115 - accuracy: 0.9944 - val_loss: 3.3160 - val_accuracy: 0.5789
Epoch 444/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0084 - accuracy: 0.9944 - val_loss: 3.2784 - val_accuracy: 0.5921
Epoch 445/1000
177/177 [==============================] - 0s 150us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 3.2491 - val_accuracy: 0.5789
Epoch 446/1000
177/177 [==============================] - 0s 129us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 3.2562 - val_accuracy: 0.5526
Epoch 447/1000
177/177 [==============================] - 0s 149us/step - loss: 0.0104 - accuracy: 0.9944 - val_loss: 3.2803 - val_accuracy: 0.5526
Epoch 448/1000
177/177 [==============================] - 0s 171us/step - loss: 0.0088 - accuracy: 0.9944 - val_loss: 3.3107 - 

177/177 [==============================] - 0s 200us/step - loss: 0.0095 - accuracy: 0.9944 - val_loss: 3.4529 - val_accuracy: 0.5658
Epoch 553/1000
177/177 [==============================] - 0s 368us/step - loss: 0.0091 - accuracy: 0.9944 - val_loss: 3.4598 - val_accuracy: 0.5658
Epoch 554/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0088 - accuracy: 0.9887 - val_loss: 3.4707 - val_accuracy: 0.5658
Epoch 555/1000
177/177 [==============================] - 0s 100us/step - loss: 0.0087 - accuracy: 0.9887 - val_loss: 3.4870 - val_accuracy: 0.5658
Epoch 556/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0093 - accuracy: 0.9887 - val_loss: 3.4981 - val_accuracy: 0.5658
Epoch 557/1000
177/177 [==============================] - 0s 130us/step - loss: 0.0086 - accuracy: 0.9944 - val_loss: 3.4907 - val_accuracy: 0.5658
Epoch 558/1000
177/177 [==============================] - 0s 146us/step - loss: 0.0091 - accuracy: 0.9887 - val_loss: 3.4856 - v

177/177 [==============================] - 0s 615us/step - loss: 0.0105 - accuracy: 0.9944 - val_loss: 3.5892 - val_accuracy: 0.5658
Epoch 663/1000
177/177 [==============================] - 0s 248us/step - loss: 0.0113 - accuracy: 0.9944 - val_loss: 3.5824 - val_accuracy: 0.5526
Epoch 664/1000
177/177 [==============================] - 0s 215us/step - loss: 0.0087 - accuracy: 0.9944 - val_loss: 3.5887 - val_accuracy: 0.5526
Epoch 665/1000
177/177 [==============================] - 0s 224us/step - loss: 0.0093 - accuracy: 0.9887 - val_loss: 3.5960 - val_accuracy: 0.5526
Epoch 666/1000
177/177 [==============================] - 0s 181us/step - loss: 0.0086 - accuracy: 0.9887 - val_loss: 3.6107 - val_accuracy: 0.5526
Epoch 667/1000
177/177 [==============================] - 0s 214us/step - loss: 0.0087 - accuracy: 0.9887 - val_loss: 3.6076 - val_accuracy: 0.5526
Epoch 668/1000
177/177 [==============================] - 0s 147us/step - loss: 0.0091 - accuracy: 0.9944 - val_loss: 3.5875 - 

177/177 [==============================] - 0s 123us/step - loss: 0.0104 - accuracy: 0.9944 - val_loss: 3.7152 - val_accuracy: 0.5526
Epoch 773/1000
177/177 [==============================] - 0s 119us/step - loss: 0.0093 - accuracy: 0.9944 - val_loss: 3.7239 - val_accuracy: 0.5526
Epoch 774/1000
177/177 [==============================] - 0s 139us/step - loss: 0.0132 - accuracy: 0.9887 - val_loss: 3.7150 - val_accuracy: 0.5526
Epoch 775/1000
177/177 [==============================] - 0s 144us/step - loss: 0.0084 - accuracy: 0.9944 - val_loss: 3.7450 - val_accuracy: 0.5526
Epoch 776/1000
177/177 [==============================] - 0s 195us/step - loss: 0.0089 - accuracy: 0.9944 - val_loss: 3.7480 - val_accuracy: 0.5526
Epoch 777/1000
177/177 [==============================] - 0s 159us/step - loss: 0.0119 - accuracy: 0.9944 - val_loss: 3.7572 - val_accuracy: 0.5526
Epoch 778/1000
177/177 [==============================] - 0s 149us/step - loss: 0.0081 - accuracy: 0.9944 - val_loss: 3.7386 - 

177/177 [==============================] - 0s 186us/step - loss: 0.0102 - accuracy: 0.9887 - val_loss: 3.8598 - val_accuracy: 0.5526
Epoch 883/1000
177/177 [==============================] - 0s 117us/step - loss: 0.0086 - accuracy: 0.9944 - val_loss: 3.8753 - val_accuracy: 0.5526
Epoch 884/1000
177/177 [==============================] - 0s 136us/step - loss: 0.0080 - accuracy: 0.9944 - val_loss: 3.8965 - val_accuracy: 0.5526
Epoch 885/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0111 - accuracy: 0.9944 - val_loss: 3.9104 - val_accuracy: 0.5526
Epoch 886/1000
177/177 [==============================] - 0s 125us/step - loss: 0.0089 - accuracy: 0.9944 - val_loss: 3.8990 - val_accuracy: 0.5395
Epoch 887/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0102 - accuracy: 0.9887 - val_loss: 3.8726 - val_accuracy: 0.5395
Epoch 888/1000
177/177 [==============================] - 0s 177us/step - loss: 0.0112 - accuracy: 0.9944 - val_loss: 3.8583 - 

177/177 [==============================] - 0s 151us/step - loss: 0.0133 - accuracy: 0.9944 - val_loss: 3.9671 - val_accuracy: 0.5526
Epoch 993/1000
177/177 [==============================] - 0s 206us/step - loss: 0.0120 - accuracy: 0.9944 - val_loss: 4.0052 - val_accuracy: 0.5526
Epoch 994/1000
177/177 [==============================] - 0s 203us/step - loss: 0.0155 - accuracy: 0.9944 - val_loss: 4.0380 - val_accuracy: 0.5526
Epoch 995/1000
177/177 [==============================] - 0s 245us/step - loss: 0.0145 - accuracy: 0.9944 - val_loss: 4.0487 - val_accuracy: 0.5395
Epoch 996/1000
177/177 [==============================] - 0s 305us/step - loss: 0.0086 - accuracy: 0.9944 - val_loss: 4.0271 - val_accuracy: 0.5526
Epoch 997/1000
177/177 [==============================] - 0s 210us/step - loss: 0.0107 - accuracy: 0.9944 - val_loss: 3.9438 - val_accuracy: 0.5658
Epoch 998/1000
177/177 [==============================] - 0s 199us/step - loss: 0.0098 - accuracy: 0.9944 - val_loss: 3.9277 - 

In [41]:
print('train accuracy: %.2f%%' % (np.mean(hist1.history['accuracy'])*100))

train accuracy: 99.34%


In [19]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [20]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS027,1,0,0.759813,0.001304,0.238883
1,p0017kpresabs_qual,NRS246,1,0,0.947819,0.009591,0.042591
2,p0017kpresabs_qual,NRS218,2,1,0.008693,0.989390,0.001916
3,p0017kpresabs_qual,CFBRSa70,2,0,0.813774,0.001482,0.184744
4,p0017kpresabs_qual,NRS177,1,1,0.000916,0.998926,0.000157
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS187,1,1,0.216843,0.568123,0.215034
604,p0040presabsSTCC_qual,SR1287,0,1,0.418802,0.553160,0.028038
605,p0040presabsSTCC_qual,CFBRSa50,0,0,0.948100,0.035031,0.016869
606,p0040presabsSTCC_qual,NRS196,2,2,0.000964,0.039095,0.959940


In [21]:
y_prob = df_proba[df_proba['phage']=='p0040presabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[4.58901350e-11, 9.96721450e-01, 3.27855160e-03],
       [9.96553060e-01, 3.44695550e-03, 1.36663140e-11],
       [1.27539020e-06, 9.99998700e-01, 4.40149180e-08],
       [5.94270500e-03, 3.10612350e-03, 9.90951200e-01],
       [1.15408850e-09, 1.28361470e-03, 9.98716350e-01],
       [9.99996200e-01, 3.67548160e-06, 1.43063490e-07],
       [2.05519500e-03, 1.98593230e-04, 9.97746170e-01],
       [9.99757470e-01, 2.42573510e-04, 6.72707150e-17],
       [1.07968150e-09, 1.54001270e-01, 8.45998760e-01],
       [7.11352900e-14, 1.48383100e-10, 1.00000000e+00],
       [6.30780400e-03, 7.79820740e-01, 2.13871430e-01],
       [1.09218820e-12, 6.74509700e-05, 9.99932500e-01],
       [9.99115770e-01, 8.84197100e-04, 3.41930350e-12],
       [6.28653100e-04, 9.99371350e-01, 3.12035960e-14],
       [5.29283940e-03, 9.94702900e-01, 4.23854950e-06],
       [3.91259140e-03, 9.61462700e-01, 3.46246700e-02],
       [9.85880550e-01, 1.08943805e-02, 3.22504740e-03],
       [3.47266650e-05, 9.98973

In [22]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovo1 = rocauc_ovo(y_test, y_prob, average="macro", multi_class="ovo")
ovo1

0.7799349177634302

In [24]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [25]:
ovr1 = rocauc_ovr(y_test, y_prob, average="macro", multi_class="ovr")
ovr1

0.7799349177634302

In [26]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y)

In [27]:
dat2 = pd.DataFrame(X_test.iloc[:,0])
dat2['test'] = y_test

In [28]:
dat2

,id,test
165,NRS191,2
237,SR1287,0
243,SR3569,2
128,NRS100,0
107,NRS001,1
...,...,...
220,NRS265,1
233,NY439,2
68,CFBRSa05,0
175,NRS205,2


In [29]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [30]:
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [31]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model2.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 709us/step - loss: 1.0575 - accuracy: 0.4463 - val_loss: 0.9183 - val_accuracy: 0.5526
Epoch 2/1000
177/177 [==============================] - 0s 180us/step - loss: 0.9620 - accuracy: 0.5311 - val_loss: 0.9157 - val_accuracy: 0.5789
Epoch 3/1000
177/177 [==============================] - 0s 219us/step - loss: 0.9567 - accuracy: 0.4576 - val_loss: 0.9545 - val_accuracy: 0.4868
Epoch 4/1000
177/177 [==============================] - 0s 324us/step - loss: 0.9051 - accuracy: 0.5480 - val_loss: 0.8936 - val_accuracy: 0.5395
Epoch 5/1000
177/177 [==============================] - 0s 275us/step - loss: 0.8815 - accuracy: 0.5876 - val_loss: 0.8931 - val_accuracy: 0.5658
Epoch 6/1000
177/177 [==============================] - ETA: 0s - loss: 0.8291 - accuracy: 0.56 - 0s 221us/step - loss: 0.8752 - accuracy: 0.5593 - val_loss: 0.9091 - val_accuracy: 0.5000
Epoch 7/1000
177/177 [===============

Epoch 112/1000
177/177 [==============================] - 0s 232us/step - loss: 0.1532 - accuracy: 0.9661 - val_loss: 1.1898 - val_accuracy: 0.5921
Epoch 113/1000
177/177 [==============================] - 0s 308us/step - loss: 0.1498 - accuracy: 0.9718 - val_loss: 1.1786 - val_accuracy: 0.6053
Epoch 114/1000
177/177 [==============================] - 0s 338us/step - loss: 0.1470 - accuracy: 0.9661 - val_loss: 1.2135 - val_accuracy: 0.5658
Epoch 115/1000
177/177 [==============================] - 0s 326us/step - loss: 0.1465 - accuracy: 0.9661 - val_loss: 1.2221 - val_accuracy: 0.5921
Epoch 116/1000
177/177 [==============================] - 0s 345us/step - loss: 0.1427 - accuracy: 0.9718 - val_loss: 1.2052 - val_accuracy: 0.5789
Epoch 117/1000
177/177 [==============================] - 0s 303us/step - loss: 0.1417 - accuracy: 0.9661 - val_loss: 1.2138 - val_accuracy: 0.6053
Epoch 118/1000
177/177 [==============================] - 0s 270us/step - loss: 0.1387 - accuracy: 0.9718 - val_

177/177 [==============================] - 0s 234us/step - loss: 0.0599 - accuracy: 0.9831 - val_loss: 1.6944 - val_accuracy: 0.5789
Epoch 223/1000
177/177 [==============================] - 0s 194us/step - loss: 0.2066 - accuracy: 0.9435 - val_loss: 1.8831 - val_accuracy: 0.5000
Epoch 224/1000
177/177 [==============================] - 0s 188us/step - loss: 0.0968 - accuracy: 0.9661 - val_loss: 1.6916 - val_accuracy: 0.5921
Epoch 225/1000
177/177 [==============================] - 0s 241us/step - loss: 0.1462 - accuracy: 0.9492 - val_loss: 1.7310 - val_accuracy: 0.5395
Epoch 226/1000
177/177 [==============================] - 0s 251us/step - loss: 0.0802 - accuracy: 0.9718 - val_loss: 1.7814 - val_accuracy: 0.5921
Epoch 227/1000
177/177 [==============================] - 0s 210us/step - loss: 0.2733 - accuracy: 0.8983 - val_loss: 1.6945 - val_accuracy: 0.5789
Epoch 228/1000
177/177 [==============================] - 0s 202us/step - loss: 0.1505 - accuracy: 0.9435 - val_loss: 1.6996 - 

177/177 [==============================] - 0s 194us/step - loss: 0.0377 - accuracy: 0.9887 - val_loss: 2.0134 - val_accuracy: 0.5921
Epoch 333/1000
177/177 [==============================] - 0s 188us/step - loss: 0.0416 - accuracy: 0.9831 - val_loss: 2.0215 - val_accuracy: 0.5789
Epoch 334/1000
177/177 [==============================] - 0s 223us/step - loss: 0.0412 - accuracy: 0.9831 - val_loss: 2.0341 - val_accuracy: 0.5921
Epoch 335/1000
177/177 [==============================] - 0s 219us/step - loss: 0.0358 - accuracy: 0.9887 - val_loss: 2.0640 - val_accuracy: 0.5789
Epoch 336/1000
177/177 [==============================] - 0s 272us/step - loss: 0.0392 - accuracy: 0.9887 - val_loss: 2.0467 - val_accuracy: 0.5789
Epoch 337/1000
177/177 [==============================] - 0s 246us/step - loss: 0.0373 - accuracy: 0.9887 - val_loss: 2.0781 - val_accuracy: 0.5789
Epoch 338/1000
177/177 [==============================] - ETA: 0s - loss: 0.0221 - accuracy: 1.00 - 0s 346us/step - loss: 0.036

177/177 [==============================] - 0s 146us/step - loss: 0.0246 - accuracy: 0.9944 - val_loss: 2.4327 - val_accuracy: 0.5789
Epoch 443/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0235 - accuracy: 0.9944 - val_loss: 2.4070 - val_accuracy: 0.5921
Epoch 444/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0242 - accuracy: 0.9887 - val_loss: 2.4337 - val_accuracy: 0.5789
Epoch 445/1000
177/177 [==============================] - 0s 121us/step - loss: 0.0255 - accuracy: 0.9887 - val_loss: 2.4030 - val_accuracy: 0.5789
Epoch 446/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0261 - accuracy: 0.9887 - val_loss: 2.4581 - val_accuracy: 0.5658
Epoch 447/1000
177/177 [==============================] - 0s 205us/step - loss: 0.0228 - accuracy: 0.9944 - val_loss: 2.5376 - val_accuracy: 0.5658
Epoch 448/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0237 - accuracy: 1.0000 - val_loss: 2.4939 - 

177/177 [==============================] - 0s 214us/step - loss: 0.0169 - accuracy: 0.9944 - val_loss: 2.7693 - val_accuracy: 0.5789
Epoch 553/1000
177/177 [==============================] - 0s 176us/step - loss: 0.0173 - accuracy: 0.9944 - val_loss: 2.7607 - val_accuracy: 0.5789
Epoch 554/1000
177/177 [==============================] - 0s 177us/step - loss: 0.0193 - accuracy: 0.9944 - val_loss: 2.7653 - val_accuracy: 0.5789
Epoch 555/1000
177/177 [==============================] - 0s 214us/step - loss: 0.0943 - accuracy: 0.9661 - val_loss: 2.8509 - val_accuracy: 0.5658
Epoch 556/1000
177/177 [==============================] - 0s 171us/step - loss: 0.0351 - accuracy: 0.9831 - val_loss: 2.8143 - val_accuracy: 0.5658
Epoch 557/1000
177/177 [==============================] - 0s 193us/step - loss: 0.0281 - accuracy: 0.9887 - val_loss: 2.8177 - val_accuracy: 0.5789
Epoch 558/1000
177/177 [==============================] - 0s 166us/step - loss: 0.0165 - accuracy: 0.9944 - val_loss: 2.8281 - 

177/177 [==============================] - 0s 194us/step - loss: 0.0140 - accuracy: 0.9944 - val_loss: 3.1422 - val_accuracy: 0.5921
Epoch 663/1000
177/177 [==============================] - 0s 164us/step - loss: 0.0162 - accuracy: 0.9887 - val_loss: 3.1562 - val_accuracy: 0.5921
Epoch 664/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0141 - accuracy: 0.9944 - val_loss: 3.1489 - val_accuracy: 0.5921
Epoch 665/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0184 - accuracy: 0.9944 - val_loss: 3.1578 - val_accuracy: 0.5921
Epoch 666/1000
177/177 [==============================] - 0s 164us/step - loss: 0.0185 - accuracy: 0.9944 - val_loss: 3.1841 - val_accuracy: 0.6053
Epoch 667/1000
177/177 [==============================] - 0s 225us/step - loss: 0.0243 - accuracy: 0.9887 - val_loss: 3.1383 - val_accuracy: 0.6053
Epoch 668/1000
177/177 [==============================] - 0s 164us/step - loss: 0.0131 - accuracy: 0.9944 - val_loss: 3.1533 - 

177/177 [==============================] - 0s 150us/step - loss: 0.0150 - accuracy: 0.9944 - val_loss: 3.5145 - val_accuracy: 0.5526
Epoch 773/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0138 - accuracy: 0.9944 - val_loss: 3.4936 - val_accuracy: 0.5658
Epoch 774/1000
177/177 [==============================] - 0s 185us/step - loss: 0.0159 - accuracy: 0.9944 - val_loss: 3.4956 - val_accuracy: 0.5658
Epoch 775/1000
177/177 [==============================] - 0s 168us/step - loss: 0.0140 - accuracy: 0.9944 - val_loss: 3.5084 - val_accuracy: 0.5395
Epoch 776/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0148 - accuracy: 0.9887 - val_loss: 3.4996 - val_accuracy: 0.5658
Epoch 777/1000
177/177 [==============================] - 0s 164us/step - loss: 0.0135 - accuracy: 0.9887 - val_loss: 3.4884 - val_accuracy: 0.5658
Epoch 778/1000
177/177 [==============================] - 0s 165us/step - loss: 0.0167 - accuracy: 0.9887 - val_loss: 3.4902 - 

177/177 [==============================] - 0s 183us/step - loss: 0.0170 - accuracy: 0.9944 - val_loss: 3.5325 - val_accuracy: 0.5658
Epoch 883/1000
177/177 [==============================] - 0s 184us/step - loss: 0.0140 - accuracy: 0.9944 - val_loss: 3.6007 - val_accuracy: 0.5658
Epoch 884/1000
177/177 [==============================] - 0s 168us/step - loss: 0.0156 - accuracy: 0.9944 - val_loss: 3.5940 - val_accuracy: 0.5658
Epoch 885/1000
177/177 [==============================] - 0s 177us/step - loss: 0.0163 - accuracy: 0.9944 - val_loss: 3.5930 - val_accuracy: 0.5658
Epoch 886/1000
177/177 [==============================] - 0s 167us/step - loss: 0.0125 - accuracy: 0.9944 - val_loss: 3.6137 - val_accuracy: 0.5658
Epoch 887/1000
177/177 [==============================] - 0s 173us/step - loss: 0.0189 - accuracy: 0.9944 - val_loss: 3.6263 - val_accuracy: 0.5526
Epoch 888/1000
177/177 [==============================] - 0s 167us/step - loss: 0.0113 - accuracy: 0.9944 - val_loss: 3.6030 - 

177/177 [==============================] - 0s 160us/step - loss: 0.0135 - accuracy: 0.9887 - val_loss: 3.5497 - val_accuracy: 0.5789
Epoch 993/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0128 - accuracy: 0.9887 - val_loss: 3.5399 - val_accuracy: 0.5789
Epoch 994/1000
177/177 [==============================] - 0s 168us/step - loss: 0.0122 - accuracy: 0.9944 - val_loss: 3.5368 - val_accuracy: 0.5658
Epoch 995/1000
177/177 [==============================] - 0s 167us/step - loss: 0.0119 - accuracy: 0.9887 - val_loss: 3.5426 - val_accuracy: 0.5789
Epoch 996/1000
177/177 [==============================] - 0s 182us/step - loss: 0.0156 - accuracy: 0.9944 - val_loss: 3.5375 - val_accuracy: 0.5789
Epoch 997/1000
177/177 [==============================] - 0s 184us/step - loss: 0.0114 - accuracy: 0.9887 - val_loss: 3.5224 - val_accuracy: 0.5789
Epoch 998/1000
177/177 [==============================] - 0s 178us/step - loss: 0.0125 - accuracy: 0.9944 - val_loss: 3.5289 - 

In [57]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

76/76 [==============================] - 0s 84us/step
test accuracy: 56.58%


In [33]:
pred2 = model2.predict_classes(X_test)
pred2

array([2, 1, 0, 1, 1, 1, 1, 2, 2, 1, 0, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2,
       0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 1, 0, 1, 1, 2, 0, 0, 2, 2, 2, 0, 1,
       0, 0, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 0, 1, 1, 1, 2, 1, 2, 1, 2,
       1, 2, 1, 2, 0, 1, 2, 1, 2, 1])

In [34]:
dat2['pred'] = pred2
dat2

,id,test,pred
165,NRS191,2,2
237,SR1287,0,1
243,SR3569,2,0
128,NRS100,0,1
107,NRS001,1,1
...,...,...,...
220,NRS265,1,1
233,NY439,2,2
68,CFBRSa05,0,1
175,NRS205,2,2


In [35]:
proba2 = model2.predict_proba(X_test)
dat_proba2 = pd.DataFrame(proba2)

In [36]:
dat_proba2

,0,1,2
0,1.434029e-09,3.946666e-08,1.000000e+00
1,5.438534e-02,9.456146e-01,1.931615e-13
2,7.172636e-01,1.157568e-03,2.815787e-01
3,1.409306e-03,9.985906e-01,1.012169e-11
4,2.871214e-01,7.124556e-01,4.229277e-04
...,...,...,...
71,5.094616e-04,8.642955e-01,1.351951e-01
72,2.574035e-04,2.221528e-01,7.775899e-01
73,1.606326e-05,9.999839e-01,2.586999e-10
74,8.320108e-15,4.489797e-09,1.000000e+00


In [37]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [38]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p40p.csv", index = False,
         header=None)

In [61]:
hist2 = model2.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 302us/step - loss: 0.0125 - accuracy: 0.9944 - val_loss: 3.7822 - val_accuracy: 0.6053
Epoch 2/1000
177/177 [==============================] - 0s 277us/step - loss: 0.0260 - accuracy: 0.9887 - val_loss: 3.8013 - val_accuracy: 0.5789
Epoch 3/1000
177/177 [==============================] - 0s 193us/step - loss: 0.0158 - accuracy: 0.9887 - val_loss: 3.7993 - val_accuracy: 0.5789
Epoch 4/1000
177/177 [==============================] - 0s 202us/step - loss: 0.0166 - accuracy: 0.9887 - val_loss: 3.7721 - val_accuracy: 0.5921
Epoch 5/1000
177/177 [==============================] - 0s 194us/step - loss: 0.0148 - accuracy: 0.9887 - val_loss: 3.7736 - val_accuracy: 0.5789
Epoch 6/1000
177/177 [==============================] - 0s 166us/step - loss: 0.0136 - accuracy: 0.9944 - val_loss: 3.7614 - val_accuracy: 0.5789
Epoch 7/1000
177/177 [==============================] - 0s 210us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0155 - accuracy: 0.9887 - val_loss: 4.0631 - val_accuracy: 0.5921
Epoch 113/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0104 - accuracy: 0.9944 - val_loss: 4.0543 - val_accuracy: 0.5789
Epoch 114/1000
177/177 [==============================] - 0s 165us/step - loss: 0.0121 - accuracy: 0.9944 - val_loss: 4.0522 - val_accuracy: 0.5658
Epoch 115/1000
177/177 [==============================] - 0s 158us/step - loss: 0.0139 - accuracy: 0.9944 - val_loss: 4.0564 - val_accuracy: 0.5658
Epoch 116/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0151 - accuracy: 0.9944 - val_loss: 4.0651 - val_accuracy: 0.5789
Epoch 117/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0104 - accuracy: 0.9944 - val_loss: 4.0543 - val_accuracy: 0.5789
Epoch 118/1000
177/177 [==============================] - 0s 149us/step - loss: 0.0142 - accuracy: 0.9944 - val_

177/177 [==============================] - 0s 216us/step - loss: 0.0105 - accuracy: 0.9944 - val_loss: 3.9908 - val_accuracy: 0.5921
Epoch 223/1000
177/177 [==============================] - 0s 193us/step - loss: 0.0108 - accuracy: 0.9944 - val_loss: 3.9869 - val_accuracy: 0.5921
Epoch 224/1000
177/177 [==============================] - 0s 209us/step - loss: 0.0130 - accuracy: 0.9887 - val_loss: 3.9869 - val_accuracy: 0.5921
Epoch 225/1000
177/177 [==============================] - 0s 244us/step - loss: 0.0110 - accuracy: 0.9944 - val_loss: 3.9767 - val_accuracy: 0.5789
Epoch 226/1000
177/177 [==============================] - 0s 275us/step - loss: 0.0122 - accuracy: 0.9944 - val_loss: 3.9784 - val_accuracy: 0.5921
Epoch 227/1000
177/177 [==============================] - 0s 236us/step - loss: 0.0115 - accuracy: 0.9944 - val_loss: 3.9687 - val_accuracy: 0.5789
Epoch 228/1000
177/177 [==============================] - 0s 263us/step - loss: 0.0436 - accuracy: 0.9887 - val_loss: 3.9935 - 

177/177 [==============================] - 0s 443us/step - loss: 0.0112 - accuracy: 0.9887 - val_loss: 3.9319 - val_accuracy: 0.5658
Epoch 333/1000
177/177 [==============================] - 0s 402us/step - loss: 0.0112 - accuracy: 0.9944 - val_loss: 3.9378 - val_accuracy: 0.5658
Epoch 334/1000
177/177 [==============================] - 0s 225us/step - loss: 0.0128 - accuracy: 0.9887 - val_loss: 3.9447 - val_accuracy: 0.5395
Epoch 335/1000
177/177 [==============================] - 0s 187us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 3.9371 - val_accuracy: 0.5658
Epoch 336/1000
177/177 [==============================] - 0s 184us/step - loss: 0.0109 - accuracy: 0.9944 - val_loss: 3.9429 - val_accuracy: 0.5526
Epoch 337/1000
177/177 [==============================] - 0s 171us/step - loss: 0.0146 - accuracy: 0.9887 - val_loss: 3.9463 - val_accuracy: 0.5395
Epoch 338/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0098 - accuracy: 0.9887 - val_loss: 3.9382 - 

177/177 [==============================] - 0s 395us/step - loss: 0.0105 - accuracy: 0.9887 - val_loss: 4.0442 - val_accuracy: 0.5526
Epoch 443/1000
177/177 [==============================] - 0s 192us/step - loss: 0.0112 - accuracy: 0.9944 - val_loss: 4.0489 - val_accuracy: 0.5526
Epoch 444/1000
177/177 [==============================] - 0s 188us/step - loss: 0.0126 - accuracy: 0.9887 - val_loss: 4.0446 - val_accuracy: 0.5789
Epoch 445/1000
177/177 [==============================] - 0s 256us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 4.0519 - val_accuracy: 0.5526
Epoch 446/1000
177/177 [==============================] - 0s 299us/step - loss: 0.0117 - accuracy: 0.9944 - val_loss: 4.0572 - val_accuracy: 0.5526
Epoch 447/1000
177/177 [==============================] - 0s 267us/step - loss: 0.0098 - accuracy: 0.9944 - val_loss: 4.0474 - val_accuracy: 0.5789
Epoch 448/1000
177/177 [==============================] - 0s 194us/step - loss: 0.0104 - accuracy: 0.9944 - val_loss: 4.0542 - 

177/177 [==============================] - 0s 391us/step - loss: 0.0099 - accuracy: 0.9944 - val_loss: 4.2289 - val_accuracy: 0.5789
Epoch 553/1000
177/177 [==============================] - 0s 424us/step - loss: 0.0100 - accuracy: 0.9944 - val_loss: 4.2205 - val_accuracy: 0.5789
Epoch 554/1000
177/177 [==============================] - 0s 227us/step - loss: 0.0096 - accuracy: 0.9944 - val_loss: 4.2311 - val_accuracy: 0.5789
Epoch 555/1000
177/177 [==============================] - 0s 261us/step - loss: 0.0095 - accuracy: 0.9944 - val_loss: 4.2254 - val_accuracy: 0.5789
Epoch 556/1000
177/177 [==============================] - 0s 296us/step - loss: 0.0104 - accuracy: 0.9944 - val_loss: 4.2282 - val_accuracy: 0.5789
Epoch 557/1000
177/177 [==============================] - 0s 300us/step - loss: 0.0101 - accuracy: 0.9887 - val_loss: 4.2298 - val_accuracy: 0.5789
Epoch 558/1000
177/177 [==============================] - 0s 284us/step - loss: 0.0090 - accuracy: 0.9944 - val_loss: 4.2353 - 

177/177 [==============================] - 0s 175us/step - loss: 0.0106 - accuracy: 0.9887 - val_loss: 4.3984 - val_accuracy: 0.5921
Epoch 663/1000
177/177 [==============================] - 0s 247us/step - loss: 0.0120 - accuracy: 0.9944 - val_loss: 4.3893 - val_accuracy: 0.5789
Epoch 664/1000
177/177 [==============================] - 0s 189us/step - loss: 0.0096 - accuracy: 0.9944 - val_loss: 4.3880 - val_accuracy: 0.5921
Epoch 665/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0178 - accuracy: 0.9944 - val_loss: 4.3908 - val_accuracy: 0.5921
Epoch 666/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0110 - accuracy: 0.9944 - val_loss: 4.4074 - val_accuracy: 0.5789
Epoch 667/1000
177/177 [==============================] - 0s 157us/step - loss: 0.0095 - accuracy: 0.9944 - val_loss: 4.3943 - val_accuracy: 0.5921
Epoch 668/1000
177/177 [==============================] - 0s 144us/step - loss: 0.0116 - accuracy: 0.9887 - val_loss: 4.3996 - 

177/177 [==============================] - 0s 160us/step - loss: 0.0113 - accuracy: 0.9944 - val_loss: 4.5896 - val_accuracy: 0.5789
Epoch 773/1000
177/177 [==============================] - 0s 184us/step - loss: 0.0107 - accuracy: 0.9887 - val_loss: 4.5877 - val_accuracy: 0.5789
Epoch 774/1000
177/177 [==============================] - 0s 330us/step - loss: 0.0100 - accuracy: 0.9887 - val_loss: 4.5780 - val_accuracy: 0.5921
Epoch 775/1000
177/177 [==============================] - 0s 220us/step - loss: 0.0126 - accuracy: 0.9944 - val_loss: 4.5789 - val_accuracy: 0.5921
Epoch 776/1000
177/177 [==============================] - 0s 233us/step - loss: 0.0090 - accuracy: 0.9944 - val_loss: 4.5942 - val_accuracy: 0.5789
Epoch 777/1000
177/177 [==============================] - 0s 240us/step - loss: 0.0111 - accuracy: 0.9944 - val_loss: 4.5942 - val_accuracy: 0.5789
Epoch 778/1000
177/177 [==============================] - 0s 262us/step - loss: 0.0088 - accuracy: 0.9887 - val_loss: 4.5991 - 

177/177 [==============================] - 0s 281us/step - loss: 0.0087 - accuracy: 0.9944 - val_loss: 4.5012 - val_accuracy: 0.5658
Epoch 883/1000
177/177 [==============================] - 0s 444us/step - loss: 0.0087 - accuracy: 0.9944 - val_loss: 4.5086 - val_accuracy: 0.5658
Epoch 884/1000
177/177 [==============================] - 0s 239us/step - loss: 0.0114 - accuracy: 0.9887 - val_loss: 4.5086 - val_accuracy: 0.5658
Epoch 885/1000
177/177 [==============================] - 0s 285us/step - loss: 0.0099 - accuracy: 0.9944 - val_loss: 4.5135 - val_accuracy: 0.5658
Epoch 886/1000
177/177 [==============================] - 0s 365us/step - loss: 0.0111 - accuracy: 0.9944 - val_loss: 4.5222 - val_accuracy: 0.5658
Epoch 887/1000
177/177 [==============================] - 0s 294us/step - loss: 0.0108 - accuracy: 0.9887 - val_loss: 4.5248 - val_accuracy: 0.5789
Epoch 888/1000
177/177 [==============================] - 0s 220us/step - loss: 0.0129 - accuracy: 0.9887 - val_loss: 4.5296 - 

177/177 [==============================] - 0s 173us/step - loss: 0.0091 - accuracy: 0.9944 - val_loss: 4.7068 - val_accuracy: 0.5789
Epoch 993/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0116 - accuracy: 0.9944 - val_loss: 4.6906 - val_accuracy: 0.5658
Epoch 994/1000
177/177 [==============================] - 0s 163us/step - loss: 0.0135 - accuracy: 0.9944 - val_loss: 4.6959 - val_accuracy: 0.5658
Epoch 995/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0091 - accuracy: 0.9944 - val_loss: 4.7004 - val_accuracy: 0.5658
Epoch 996/1000
177/177 [==============================] - 0s 164us/step - loss: 0.0106 - accuracy: 0.9944 - val_loss: 4.7020 - val_accuracy: 0.5658
Epoch 997/1000
177/177 [==============================] - 0s 188us/step - loss: 0.0111 - accuracy: 0.9944 - val_loss: 4.7040 - val_accuracy: 0.5658
Epoch 998/1000
177/177 [==============================] - 0s 175us/step - loss: 0.0152 - accuracy: 0.9887 - val_loss: 4.7039 - 

In [62]:
print('train accuracy: %.2f%%' % (np.mean(hist2.history['accuracy'])*100))

train accuracy: 99.09%


In [31]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [32]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS210,2,0,0.999887,0.000112,8.851192e-07
1,p0017kpresabs_qual,Grady1,0,0,0.625329,0.369782,4.889404e-03
2,p0017kpresabs_qual,CFBRSa29,2,0,0.999098,0.000269,6.335156e-04
3,p0017kpresabs_qual,CFBRSa03,0,0,0.647338,0.331796,2.086646e-02
4,p0017kpresabs_qual,217,1,0,0.613342,0.381903,4.754707e-03
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS265,1,1,0.025601,0.687962,2.864372e-01
604,p0040presabsSTCC_qual,NY439,2,2,0.161947,0.266501,5.715521e-01
605,p0040presabsSTCC_qual,CFBRSa05,0,0,0.652983,0.254172,9.284494e-02
606,p0040presabsSTCC_qual,NRS205,2,2,0.000927,0.033760,9.653131e-01


In [33]:
y_prob2 = df_proba2[df_proba2['phage']=='p0040presabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[1.43402920e-09, 3.94666600e-08, 1.00000000e+00],
       [5.43853380e-02, 9.45614640e-01, 1.93161460e-13],
       [7.17263640e-01, 1.15756830e-03, 2.81578720e-01],
       [1.40930580e-03, 9.98590650e-01, 1.01216920e-11],
       [2.87121420e-01, 7.12455630e-01, 4.22927700e-04],
       [5.12890360e-05, 5.65083000e-01, 4.34865700e-01],
       [2.81761050e-01, 5.82132300e-01, 1.36106640e-01],
       [1.47275800e-18, 6.94407100e-13, 1.00000000e+00],
       [3.14586030e-07, 1.32543660e-07, 9.99999500e-01],
       [5.40624840e-04, 9.95986160e-01, 3.47321480e-03],
       [9.96009950e-01, 7.87025400e-09, 3.99001100e-03],
       [1.52915170e-06, 4.68382750e-05, 9.99951600e-01],
       [1.60538880e-01, 4.08706070e-05, 8.39420260e-01],
       [2.28182820e-02, 9.75833700e-01, 1.34800210e-03],
       [1.11225310e-02, 3.34961150e-01, 6.53916360e-01],
       [3.59981640e-07, 2.58497110e-02, 9.74149940e-01],
       [1.72430660e-18, 7.12059160e-13, 1.00000000e+00],
       [3.86707430e-04, 3.81988

In [34]:
ovo2 = rocauc_ovo(y_test, y_prob2, average="macro", multi_class="ovo")
ovo2

0.7390860656487339

In [35]:
ovr2 = rocauc_ovr(y_test, y_prob2, average="macro", multi_class="ovr")
ovr2

0.7390860656487339

In [36]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y)

In [37]:
dat3 = pd.DataFrame(X_test.iloc[:,0])
dat3['test'] = y_test

In [38]:
dat3

,id,test
165,NRS191,2
64,CFBREBSa135,0
121,NRS064,1
228,NY224,1
114,NRS035,1
...,...,...
16,BCH-SA-01,0
13,504,0
96,GA27,2
177,NRS209,1


In [39]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [40]:
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [44]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model3.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 711us/step - loss: 1.1650 - accuracy: 0.3672 - val_loss: 1.0982 - val_accuracy: 0.3947
Epoch 2/1000
177/177 [==============================] - 0s 95us/step - loss: 1.0362 - accuracy: 0.4859 - val_loss: 1.0461 - val_accuracy: 0.4474
Epoch 3/1000
177/177 [==============================] - 0s 84us/step - loss: 1.0020 - accuracy: 0.5367 - val_loss: 1.0313 - val_accuracy: 0.4474
Epoch 4/1000
177/177 [==============================] - 0s 95us/step - loss: 0.9655 - accuracy: 0.5593 - val_loss: 1.0252 - val_accuracy: 0.4342
Epoch 5/1000
177/177 [==============================] - 0s 96us/step - loss: 0.9255 - accuracy: 0.5819 - val_loss: 1.0351 - val_accuracy: 0.4342
Epoch 6/1000
177/177 [==============================] - 0s 125us/step - loss: 0.9059 - accuracy: 0.5819 - val_loss: 1.0516 - val_accuracy: 0.4737
Epoch 7/1000
177/177 [==============================] - 0s 87us/step - loss: 0.8883

Epoch 57/1000
177/177 [==============================] - 0s 129us/step - loss: 0.4436 - accuracy: 0.8814 - val_loss: 1.1624 - val_accuracy: 0.5132
Epoch 58/1000
177/177 [==============================] - 0s 99us/step - loss: 0.4357 - accuracy: 0.8814 - val_loss: 1.1712 - val_accuracy: 0.5263
Epoch 59/1000
177/177 [==============================] - 0s 128us/step - loss: 0.4322 - accuracy: 0.8701 - val_loss: 1.1799 - val_accuracy: 0.5132
Epoch 60/1000
177/177 [==============================] - 0s 140us/step - loss: 0.4261 - accuracy: 0.8927 - val_loss: 1.1895 - val_accuracy: 0.4868
Epoch 61/1000
177/177 [==============================] - 0s 92us/step - loss: 0.4235 - accuracy: 0.8814 - val_loss: 1.2101 - val_accuracy: 0.4868
Epoch 62/1000
177/177 [==============================] - 0s 120us/step - loss: 0.4178 - accuracy: 0.8757 - val_loss: 1.1899 - val_accuracy: 0.5132
Epoch 63/1000
177/177 [==============================] - 0s 108us/step - loss: 0.4111 - accuracy: 0.8757 - val_loss: 1.1

Epoch 113/1000
177/177 [==============================] - 0s 82us/step - loss: 0.2377 - accuracy: 0.9605 - val_loss: 1.4902 - val_accuracy: 0.4605
Epoch 114/1000
177/177 [==============================] - 0s 133us/step - loss: 0.2382 - accuracy: 0.9605 - val_loss: 1.5025 - val_accuracy: 0.4605
Epoch 115/1000
177/177 [==============================] - 0s 112us/step - loss: 0.2340 - accuracy: 0.9605 - val_loss: 1.5089 - val_accuracy: 0.4474
Epoch 116/1000
177/177 [==============================] - 0s 80us/step - loss: 0.2321 - accuracy: 0.9548 - val_loss: 1.5205 - val_accuracy: 0.4474
Epoch 117/1000
177/177 [==============================] - 0s 76us/step - loss: 0.2322 - accuracy: 0.9548 - val_loss: 1.5369 - val_accuracy: 0.4474
Epoch 118/1000
177/177 [==============================] - 0s 71us/step - loss: 0.2269 - accuracy: 0.9605 - val_loss: 1.5436 - val_accuracy: 0.4605
Epoch 119/1000
177/177 [==============================] - 0s 113us/step - loss: 0.2257 - accuracy: 0.9605 - val_loss

177/177 [==============================] - 0s 69us/step - loss: 0.1055 - accuracy: 0.9774 - val_loss: 2.1240 - val_accuracy: 0.4474
Epoch 224/1000
177/177 [==============================] - 0s 103us/step - loss: 0.1056 - accuracy: 0.9774 - val_loss: 2.1325 - val_accuracy: 0.4605
Epoch 225/1000
177/177 [==============================] - 0s 96us/step - loss: 0.1045 - accuracy: 0.9774 - val_loss: 2.1375 - val_accuracy: 0.4605
Epoch 226/1000
177/177 [==============================] - 0s 117us/step - loss: 0.1039 - accuracy: 0.9774 - val_loss: 2.1447 - val_accuracy: 0.4605
Epoch 227/1000
177/177 [==============================] - 0s 76us/step - loss: 0.1038 - accuracy: 0.9774 - val_loss: 2.1440 - val_accuracy: 0.4605
Epoch 228/1000
177/177 [==============================] - 0s 121us/step - loss: 0.1053 - accuracy: 0.9774 - val_loss: 2.1479 - val_accuracy: 0.4605
Epoch 229/1000
177/177 [==============================] - 0s 119us/step - loss: 0.1034 - accuracy: 0.9774 - val_loss: 2.1432 - val

Epoch 279/1000
177/177 [==============================] - 0s 147us/step - loss: 0.0834 - accuracy: 0.9774 - val_loss: 2.3741 - val_accuracy: 0.4605
Epoch 280/1000
177/177 [==============================] - 0s 163us/step - loss: 0.0807 - accuracy: 0.9774 - val_loss: 2.3782 - val_accuracy: 0.4737
Epoch 281/1000
177/177 [==============================] - 0s 150us/step - loss: 0.0807 - accuracy: 0.9774 - val_loss: 2.3779 - val_accuracy: 0.4737
Epoch 282/1000
177/177 [==============================] - 0s 225us/step - loss: 0.0798 - accuracy: 0.9774 - val_loss: 2.3866 - val_accuracy: 0.4737
Epoch 283/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0802 - accuracy: 0.9774 - val_loss: 2.3951 - val_accuracy: 0.4737
Epoch 284/1000
177/177 [==============================] - 0s 176us/step - loss: 0.0806 - accuracy: 0.9774 - val_loss: 2.3937 - val_accuracy: 0.4605
Epoch 285/1000
177/177 [==============================] - 0s 93us/step - loss: 0.0797 - accuracy: 0.9718 - val_l

...

Epoch 611/1000
177/177 [==============================] - 0s 97us/step - loss: 0.0363 - accuracy: 0.9887 - val_loss: 3.3898 - val_accuracy: 0.4474
Epoch 612/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0367 - accuracy: 0.9831 - val_loss: 3.4193 - val_accuracy: 0.4474
Epoch 613/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0361 - accuracy: 0.9887 - val_loss: 3.4041 - val_accuracy: 0.4474
Epoch 614/1000
177/177 [==============================] - 0s 83us/step - loss: 0.0363 - accuracy: 0.9887 - val_loss: 3.4050 - val_accuracy: 0.4605
Epoch 615/1000
177/177 [==============================] - 0s 173us/step - loss: 0.0357 - accuracy: 0.9887 - val_loss: 3.4002 - val_accuracy: 0.4605
Epoch 616/1000
177/177 [==============================] - 0s 207us/step - loss: 0.0347 - accuracy: 0.9887 - val_loss: 3.3914 - val_accuracy: 0.4605
Epoch 617/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0363 - accuracy: 0.9887 - val_loss

Epoch 667/1000
177/177 [==============================] - 0s 68us/step - loss: 0.0325 - accuracy: 0.9887 - val_loss: 3.4964 - val_accuracy: 0.4605
Epoch 668/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0324 - accuracy: 0.9887 - val_loss: 3.5017 - val_accuracy: 0.4605
Epoch 669/1000
177/177 [==============================] - ETA: 0s - loss: 0.0183 - accuracy: 1.00 - 0s 103us/step - loss: 0.0331 - accuracy: 0.9887 - val_loss: 3.5096 - val_accuracy: 0.4605
Epoch 670/1000
177/177 [==============================] - 0s 71us/step - loss: 0.0326 - accuracy: 0.9831 - val_loss: 3.5165 - val_accuracy: 0.4605
Epoch 671/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0332 - accuracy: 0.9831 - val_loss: 3.5080 - val_accuracy: 0.4474
Epoch 672/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0346 - accuracy: 0.9831 - val_loss: 3.5408 - val_accuracy: 0.4605
Epoch 673/1000
177/177 [==============================] - 0s 75us/step - lo

Epoch 778/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0298 - accuracy: 0.9831 - val_loss: 3.7326 - val_accuracy: 0.4605
Epoch 779/1000
177/177 [==============================] - 0s 71us/step - loss: 0.0290 - accuracy: 0.9887 - val_loss: 3.7236 - val_accuracy: 0.4605
Epoch 780/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0287 - accuracy: 0.9887 - val_loss: 3.7132 - val_accuracy: 0.4605
Epoch 781/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0291 - accuracy: 0.9887 - val_loss: 3.7106 - val_accuracy: 0.4474
Epoch 782/1000
177/177 [==============================] - 0s 82us/step - loss: 0.0287 - accuracy: 0.9887 - val_loss: 3.7276 - val_accuracy: 0.4474
Epoch 783/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0288 - accuracy: 0.9887 - val_loss: 3.7482 - val_accuracy: 0.4474
Epoch 784/1000
177/177 [==============================] - 0s 91us/step - loss: 0.0292 - accuracy: 0.9831 - val_loss: 3

Epoch 834/1000
177/177 [==============================] - 0s 58us/step - loss: 0.0277 - accuracy: 0.9887 - val_loss: 3.8309 - val_accuracy: 0.4474
Epoch 835/1000
177/177 [==============================] - 0s 66us/step - loss: 0.0278 - accuracy: 0.9887 - val_loss: 3.8465 - val_accuracy: 0.4342
Epoch 836/1000
177/177 [==============================] - 0s 92us/step - loss: 0.0280 - accuracy: 0.9887 - val_loss: 3.8488 - val_accuracy: 0.4342
Epoch 837/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0285 - accuracy: 0.9831 - val_loss: 3.8201 - val_accuracy: 0.4605
Epoch 838/1000
177/177 [==============================] - 0s 93us/step - loss: 0.0278 - accuracy: 0.9887 - val_loss: 3.8241 - val_accuracy: 0.4605
Epoch 839/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0275 - accuracy: 0.9887 - val_loss: 3.8355 - val_accuracy: 0.4474
Epoch 840/1000
177/177 [==============================] - 0s 282us/step - loss: 0.0275 - accuracy: 0.9887 - val_loss: 

Epoch 945/1000
177/177 [==============================] - 0s 97us/step - loss: 0.0273 - accuracy: 0.9887 - val_loss: 3.9870 - val_accuracy: 0.4474
Epoch 946/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0269 - accuracy: 0.9887 - val_loss: 4.0262 - val_accuracy: 0.4342
Epoch 947/1000
177/177 [==============================] - 0s 103us/step - loss: 0.0267 - accuracy: 0.9887 - val_loss: 4.0436 - val_accuracy: 0.4342
Epoch 948/1000
177/177 [==============================] - 0s 124us/step - loss: 0.0263 - accuracy: 0.9831 - val_loss: 4.0140 - val_accuracy: 0.4342
Epoch 949/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0257 - accuracy: 0.9887 - val_loss: 4.0045 - val_accuracy: 0.4342
Epoch 950/1000
177/177 [==============================] - 0s 93us/step - loss: 0.0252 - accuracy: 0.9887 - val_loss: 4.0289 - val_accuracy: 0.4474
Epoch 951/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0263 - accuracy: 0.9887 - val_loss

In [82]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

76/76 [==============================] - 0s 138us/step
test accuracy: 47.37%


In [46]:
pred3 = model3.predict_classes(X_test)
pred3

array([2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 0, 0, 1, 1, 0, 1,
       0, 2, 2, 2, 2, 1, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 0, 2, 1, 2, 1, 1,
       1, 0, 2, 0, 0, 1, 2, 1, 0, 0, 2, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       2, 2, 2, 0, 1, 0, 1, 1, 0, 0])

In [47]:
dat3['pred'] = pred3
dat3

,id,test,pred
165,NRS191,2,2
64,CFBREBSa135,0,0
121,NRS064,1,0
228,NY224,1,2
114,NRS035,1,2
...,...,...,...
16,BCH-SA-01,0,0
13,504,0,1
96,GA27,2,1
177,NRS209,1,0


In [48]:
proba3 = model3.predict_proba(X_test)
dat_proba3 = pd.DataFrame(proba3)

In [49]:
dat_proba3

,0,1,2
0,0.009549,2.489312e-02,9.655582e-01
1,0.999999,1.427992e-06,6.495252e-11
2,0.558418,4.316774e-01,9.904408e-03
3,0.000041,6.440350e-02,9.355550e-01
4,0.001248,2.634320e-03,9.961181e-01
...,...,...,...
71,0.997672,2.325171e-03,2.614610e-06
72,0.000182,9.998055e-01,1.258049e-05
73,0.108618,6.415803e-01,2.498018e-01
74,0.999804,1.964406e-04,2.147680e-09


In [50]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [51]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p40p.csv", index = False,
         header=None)

In [86]:
hist3 = model3.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 106us/step - loss: 0.0250 - accuracy: 0.9887 - val_loss: 3.6336 - val_accuracy: 0.4737
Epoch 2/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0260 - accuracy: 0.9887 - val_loss: 3.6730 - val_accuracy: 0.4737
Epoch 3/1000
177/177 [==============================] - 0s 129us/step - loss: 0.0264 - accuracy: 0.9887 - val_loss: 3.6660 - val_accuracy: 0.4737
Epoch 4/1000
177/177 [==============================] - 0s 104us/step - loss: 0.0254 - accuracy: 0.9887 - val_loss: 3.6703 - val_accuracy: 0.4737
Epoch 5/1000
177/177 [==============================] - 0s 121us/step - loss: 0.0255 - accuracy: 0.9887 - val_loss: 3.6811 - val_accuracy: 0.4737
Epoch 6/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0250 - accuracy: 0.9887 - val_loss: 3.6495 - val_accuracy: 0.4737
Epoch 7/1000
177/177 [==============================] - 0s 115us/step - loss: 0.

177/177 [==============================] - 0s 914us/step - loss: 0.0255 - accuracy: 0.9887 - val_loss: 3.8030 - val_accuracy: 0.4737
Epoch 112/1000
177/177 [==============================] - 0s 97us/step - loss: 0.0238 - accuracy: 0.9887 - val_loss: 3.7666 - val_accuracy: 0.4737
Epoch 113/1000
177/177 [==============================] - 0s 249us/step - loss: 0.0247 - accuracy: 0.9887 - val_loss: 3.7743 - val_accuracy: 0.4737
Epoch 114/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0239 - accuracy: 0.9887 - val_loss: 3.8138 - val_accuracy: 0.4737
Epoch 115/1000
177/177 [==============================] - 0s 89us/step - loss: 0.0237 - accuracy: 0.9887 - val_loss: 3.8159 - val_accuracy: 0.4737
Epoch 116/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0244 - accuracy: 0.9887 - val_loss: 3.7827 - val_accuracy: 0.4737
Epoch 117/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0233 - accuracy: 0.9887 - val_loss: 3.7669 - val_a

Epoch 222/1000
177/177 [==============================] - 0s 104us/step - loss: 0.0229 - accuracy: 0.9887 - val_loss: 3.9060 - val_accuracy: 0.4737
Epoch 223/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0216 - accuracy: 0.9944 - val_loss: 3.9468 - val_accuracy: 0.4737
Epoch 224/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0289 - accuracy: 0.9887 - val_loss: 3.9713 - val_accuracy: 0.4737
Epoch 225/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0281 - accuracy: 0.9887 - val_loss: 3.8654 - val_accuracy: 0.4737
Epoch 226/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0230 - accuracy: 0.9887 - val_loss: 3.8502 - val_accuracy: 0.4737
Epoch 227/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0233 - accuracy: 0.9887 - val_loss: 3.8885 - val_accuracy: 0.4737
Epoch 228/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0234 - accuracy: 0.9831 - val_loss: 

Epoch 333/1000
177/177 [==============================] - 0s 68us/step - loss: 0.0225 - accuracy: 0.9887 - val_loss: 4.0764 - val_accuracy: 0.4737
Epoch 334/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0217 - accuracy: 0.9887 - val_loss: 4.0631 - val_accuracy: 0.4737
Epoch 335/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0223 - accuracy: 0.9831 - val_loss: 4.0728 - val_accuracy: 0.4737
Epoch 336/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0232 - accuracy: 0.9887 - val_loss: 4.0925 - val_accuracy: 0.4737
Epoch 337/1000
177/177 [==============================] - 0s 66us/step - loss: 0.0216 - accuracy: 0.9887 - val_loss: 4.0534 - val_accuracy: 0.4737
Epoch 338/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0229 - accuracy: 0.9887 - val_loss: 4.0532 - val_accuracy: 0.4737
Epoch 339/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0212 - accuracy: 0.9887 - val_loss: 4

Epoch 389/1000
177/177 [==============================] - 0s 123us/step - loss: 0.0218 - accuracy: 0.9887 - val_loss: 4.1604 - val_accuracy: 0.4605
Epoch 390/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0219 - accuracy: 0.9887 - val_loss: 4.1429 - val_accuracy: 0.4737
Epoch 391/1000
177/177 [==============================] - 0s 125us/step - loss: 0.0216 - accuracy: 0.9887 - val_loss: 4.1201 - val_accuracy: 0.4737
Epoch 392/1000
177/177 [==============================] - 0s 133us/step - loss: 0.0218 - accuracy: 0.9887 - val_loss: 4.0966 - val_accuracy: 0.4737
Epoch 393/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0222 - accuracy: 0.9887 - val_loss: 4.1163 - val_accuracy: 0.4737
Epoch 394/1000
177/177 [==============================] - 0s 111us/step - loss: 0.0227 - accuracy: 0.9887 - val_loss: 4.1243 - val_accuracy: 0.4737
Epoch 395/1000
177/177 [==============================] - 0s 127us/step - loss: 0.0212 - accuracy: 0.9887 - val_

...

Epoch 555/1000
177/177 [==============================] - 0s 132us/step - loss: 0.0231 - accuracy: 0.9887 - val_loss: 4.2221 - val_accuracy: 0.4868
Epoch 556/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0216 - accuracy: 0.9887 - val_loss: 4.2636 - val_accuracy: 0.4868
Epoch 557/1000
177/177 [==============================] - 0s 91us/step - loss: 0.0207 - accuracy: 0.9887 - val_loss: 4.3028 - val_accuracy: 0.4737
Epoch 558/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0220 - accuracy: 0.9887 - val_loss: 4.3134 - val_accuracy: 0.4737
Epoch 559/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0208 - accuracy: 0.9887 - val_loss: 4.2977 - val_accuracy: 0.4737
Epoch 560/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0209 - accuracy: 0.9887 - val_loss: 4.2786 - val_accuracy: 0.4868
Epoch 561/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0218 - accuracy: 0.9831 - val_loss:

Epoch 666/1000
177/177 [==============================] - 0s 257us/step - loss: 0.0231 - accuracy: 0.9887 - val_loss: 4.3481 - val_accuracy: 0.4868
Epoch 667/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0213 - accuracy: 0.9887 - val_loss: 4.3982 - val_accuracy: 0.4868
Epoch 668/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0240 - accuracy: 0.9831 - val_loss: 4.4500 - val_accuracy: 0.4737
Epoch 669/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0214 - accuracy: 0.9887 - val_loss: 4.4028 - val_accuracy: 0.4868
Epoch 670/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0233 - accuracy: 0.9831 - val_loss: 4.3624 - val_accuracy: 0.4868
Epoch 671/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0215 - accuracy: 0.9887 - val_loss: 4.4052 - val_accuracy: 0.4868
Epoch 672/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0210 - accuracy: 0.9887 - val_loss: 

177/177 [==============================] - 0s 103us/step - loss: 0.0234 - accuracy: 0.9887 - val_loss: 4.4406 - val_accuracy: 0.4868
Epoch 777/1000
177/177 [==============================] - 0s 109us/step - loss: 0.0233 - accuracy: 0.9831 - val_loss: 4.4879 - val_accuracy: 0.4737
Epoch 778/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0202 - accuracy: 0.9887 - val_loss: 4.4779 - val_accuracy: 0.4737
Epoch 779/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0208 - accuracy: 0.9831 - val_loss: 4.4581 - val_accuracy: 0.4737
Epoch 780/1000
177/177 [==============================] - 0s 95us/step - loss: 0.0212 - accuracy: 0.9831 - val_loss: 4.4685 - val_accuracy: 0.4737
Epoch 781/1000
177/177 [==============================] - 0s 169us/step - loss: 0.0205 - accuracy: 0.9887 - val_loss: 4.4635 - val_accuracy: 0.4737
Epoch 782/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0202 - accuracy: 0.9887 - val_loss: 4.4514 - val

177/177 [==============================] - 0s 71us/step - loss: 0.0218 - accuracy: 0.9887 - val_loss: 4.6350 - val_accuracy: 0.4737
Epoch 887/1000
177/177 [==============================] - 0s 88us/step - loss: 0.0206 - accuracy: 0.9887 - val_loss: 4.6055 - val_accuracy: 0.4737
Epoch 888/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0206 - accuracy: 0.9887 - val_loss: 4.6045 - val_accuracy: 0.4737
Epoch 889/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0213 - accuracy: 0.9887 - val_loss: 4.5917 - val_accuracy: 0.4737
Epoch 890/1000
177/177 [==============================] - 0s 77us/step - loss: 0.0201 - accuracy: 0.9887 - val_loss: 4.6103 - val_accuracy: 0.4737
Epoch 891/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0212 - accuracy: 0.9887 - val_loss: 4.6306 - val_accuracy: 0.4737
Epoch 892/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0201 - accuracy: 0.9887 - val_loss: 4.6064 - val_acc

177/177 [==============================] - 0s 84us/step - loss: 0.0222 - accuracy: 0.9887 - val_loss: 4.6453 - val_accuracy: 0.4868
Epoch 997/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0216 - accuracy: 0.9887 - val_loss: 4.6647 - val_accuracy: 0.4737
Epoch 998/1000
177/177 [==============================] - 0s 122us/step - loss: 0.0196 - accuracy: 0.9887 - val_loss: 4.7035 - val_accuracy: 0.4737
Epoch 999/1000
177/177 [==============================] - 0s 91us/step - loss: 0.0218 - accuracy: 0.9887 - val_loss: 4.7215 - val_accuracy: 0.4737
Epoch 1000/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0219 - accuracy: 0.9887 - val_loss: 4.6943 - val_accuracy: 0.4868


In [87]:
print('train accuracy: %.2f%%' % (np.mean(hist3.history['accuracy'])*100))

train accuracy: 98.78%


In [41]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [42]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,2,2.165526e-02,4.848140e-01,0.493531
1,p0017kpresabs_qual,EUH25,2,0,9.986388e-01,1.245148e-03,0.000116
2,p0017kpresabs_qual,EUH15,2,2,9.227520e-04,1.424882e-02,0.984828
3,p0017kpresabs_qual,NRS241,0,0,8.374333e-01,1.614128e-01,0.001154
4,p0017kpresabs_qual,SR2852,2,2,3.976981e-09,5.145955e-10,1.000000
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,BCH-SA-01,0,0,9.305407e-01,6.356251e-02,0.005897
604,p0040presabsSTCC_qual,504,0,1,4.476389e-02,8.577548e-01,0.097481
605,p0040presabsSTCC_qual,GA27,2,1,2.779456e-01,5.384378e-01,0.183617
606,p0040presabsSTCC_qual,NRS209,1,0,4.210180e-01,3.559393e-01,0.223043


In [43]:
y_prob3 = df_proba3[df_proba3['phage']=='p0040presabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.5487210e-03, 2.4893120e-02, 9.6555820e-01],
       [9.9999857e-01, 1.4279918e-06, 6.4952520e-11],
       [5.5841820e-01, 4.3167743e-01, 9.9044080e-03],
       [4.1489110e-05, 6.4403500e-02, 9.3555500e-01],
       [1.2476562e-03, 2.6343197e-03, 9.9611807e-01],
       [1.6435917e-05, 9.9994874e-01, 3.4805395e-05],
       [3.4076170e-02, 9.5397604e-01, 1.1947808e-02],
       [4.6969426e-01, 5.3028953e-01, 1.6184980e-05],
       [1.9877884e-06, 9.6901950e-01, 3.0978562e-02],
       [1.0820310e-06, 9.9999870e-01, 2.1898592e-07],
       [2.2454446e-06, 5.8652848e-05, 9.9993910e-01],
       [7.6595485e-02, 9.2301940e-01, 3.8516693e-04],
       [4.9977007e-05, 9.9209520e-01, 7.8547720e-03],
       [3.7799658e-10, 9.9997570e-01, 2.4270434e-05],
       [5.8269060e-04, 5.9828823e-03, 9.9343440e-01],
       [2.7889004e-05, 9.9997130e-01, 7.8890787e-07],
       [5.7154155e-01, 4.6925232e-04, 4.2798920e-01],
       [9.9998450e-01, 1.5460250e-05, 3.8612015e-08],
       [8.2496520e-05, 9.917

In [44]:
ovo3 = rocauc_ovo(y_test, y_prob3, average="macro", multi_class="ovo")
ovo3

0.6375221485030232

In [45]:
ovr3 = rocauc_ovr(y_test, y_prob3, average="macro", multi_class="ovr")
ovr3

0.6375221485030232

In [46]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y)

In [47]:
dat4 = pd.DataFrame(X_test.iloc[:,0])
dat4['test'] = y_test

In [48]:
dat4

,id,test
54,CFBREBSa121,1
28,BCH-SA-13,1
143,NRS148,2
73,CFBRSa23,2
57,CFBREBSa125,0
...,...,...
208,NRS247,1
183,NRS215,1
248,SR4152,2
114,NRS035,1


In [49]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [50]:
model4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [57]:
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
model4.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 671us/step - loss: 1.1281 - accuracy: 0.4576 - val_loss: 1.0738 - val_accuracy: 0.4079
Epoch 2/1000
177/177 [==============================] - 0s 188us/step - loss: 0.9746 - accuracy: 0.4915 - val_loss: 1.0757 - val_accuracy: 0.3553
Epoch 3/1000
177/177 [==============================] - 0s 213us/step - loss: 0.9506 - accuracy: 0.5311 - val_loss: 1.0607 - val_accuracy: 0.3947
Epoch 4/1000
177/177 [==============================] - 0s 175us/step - loss: 0.9012 - accuracy: 0.5480 - val_loss: 1.0293 - val_accuracy: 0.3947
Epoch 5/1000
177/177 [==============================] - 0s 205us/step - loss: 0.8766 - accuracy: 0.5706 - val_loss: 1.0408 - val_accuracy: 0.3947
Epoch 6/1000
177/177 [==============================] - 0s 207us/step - loss: 0.8563 - accuracy: 0.6441 - val_loss: 1.0671 - val_accuracy: 0.4737
Epoch 7/1000
177/177 [==============================] - 0s 260us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 181us/step - loss: 0.1891 - accuracy: 0.9774 - val_loss: 1.5378 - val_accuracy: 0.5000
Epoch 113/1000
177/177 [==============================] - 0s 153us/step - loss: 0.1957 - accuracy: 0.9492 - val_loss: 1.5107 - val_accuracy: 0.5263
Epoch 114/1000
177/177 [==============================] - 0s 194us/step - loss: 0.1888 - accuracy: 0.9661 - val_loss: 1.5451 - val_accuracy: 0.5395
Epoch 115/1000
177/177 [==============================] - 0s 182us/step - loss: 0.2058 - accuracy: 0.9435 - val_loss: 1.5470 - val_accuracy: 0.5395
Epoch 116/1000
177/177 [==============================] - 0s 265us/step - loss: 0.2470 - accuracy: 0.8757 - val_loss: 1.4816 - val_accuracy: 0.5789
Epoch 117/1000
177/177 [==============================] - 0s 206us/step - loss: 0.2096 - accuracy: 0.9379 - val_loss: 1.5396 - val_accuracy: 0.5263
Epoch 118/1000
177/177 [==============================] - 0s 279us/step - loss: 0.1898 - accuracy: 0.9435 - val_

177/177 [==============================] - 0s 264us/step - loss: 0.1509 - accuracy: 0.9492 - val_loss: 1.8395 - val_accuracy: 0.5000
Epoch 168/1000
177/177 [==============================] - 0s 286us/step - loss: 0.4549 - accuracy: 0.8475 - val_loss: 2.2395 - val_accuracy: 0.5395
Epoch 169/1000
177/177 [==============================] - 0s 342us/step - loss: 0.3546 - accuracy: 0.8814 - val_loss: 2.0534 - val_accuracy: 0.4474
Epoch 170/1000
177/177 [==============================] - 0s 339us/step - loss: 0.1886 - accuracy: 0.9209 - val_loss: 1.9987 - val_accuracy: 0.4605
Epoch 171/1000
177/177 [==============================] - 0s 187us/step - loss: 0.1475 - accuracy: 0.9548 - val_loss: 1.9626 - val_accuracy: 0.5132
Epoch 172/1000
177/177 [==============================] - 0s 157us/step - loss: 0.1252 - accuracy: 0.9774 - val_loss: 1.9424 - val_accuracy: 0.5132
Epoch 173/1000
177/177 [==============================] - 0s 137us/step - loss: 0.1211 - accuracy: 0.9831 - val_loss: 1.9240 - 

177/177 [==============================] - 0s 152us/step - loss: 0.0550 - accuracy: 0.9944 - val_loss: 2.4449 - val_accuracy: 0.5000
Epoch 278/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0629 - accuracy: 0.9831 - val_loss: 2.4422 - val_accuracy: 0.4605
Epoch 279/1000
177/177 [==============================] - 0s 240us/step - loss: 0.2007 - accuracy: 0.9096 - val_loss: 2.4134 - val_accuracy: 0.5526
Epoch 280/1000
177/177 [==============================] - 0s 226us/step - loss: 0.1001 - accuracy: 0.9661 - val_loss: 2.5684 - val_accuracy: 0.5263
Epoch 281/1000
177/177 [==============================] - 0s 302us/step - loss: 0.0801 - accuracy: 0.9718 - val_loss: 2.4823 - val_accuracy: 0.5526
Epoch 282/1000
177/177 [==============================] - 0s 233us/step - loss: 0.0597 - accuracy: 0.9887 - val_loss: 2.4606 - val_accuracy: 0.5132
Epoch 283/1000
177/177 [==============================] - 0s 219us/step - loss: 0.0521 - accuracy: 1.0000 - val_loss: 2.5108 - 

177/177 [==============================] - 0s 155us/step - loss: 0.0385 - accuracy: 0.9887 - val_loss: 2.9348 - val_accuracy: 0.5263
Epoch 388/1000
177/177 [==============================] - 0s 186us/step - loss: 0.0342 - accuracy: 0.9944 - val_loss: 2.9547 - val_accuracy: 0.5132
Epoch 389/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0313 - accuracy: 1.0000 - val_loss: 2.9880 - val_accuracy: 0.5395
Epoch 390/1000
177/177 [==============================] - 0s 175us/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 2.9683 - val_accuracy: 0.5263
Epoch 391/1000
177/177 [==============================] - 0s 177us/step - loss: 0.0368 - accuracy: 0.9944 - val_loss: 2.9761 - val_accuracy: 0.5395
Epoch 392/1000
177/177 [==============================] - 0s 169us/step - loss: 0.0327 - accuracy: 0.9887 - val_loss: 2.9995 - val_accuracy: 0.5395
Epoch 393/1000
177/177 [==============================] - 0s 215us/step - loss: 0.0295 - accuracy: 1.0000 - val_loss: 2.9881 - 

177/177 [==============================] - 0s 120us/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 3.5561 - val_accuracy: 0.5000
Epoch 498/1000
177/177 [==============================] - 0s 117us/step - loss: 0.0189 - accuracy: 0.9944 - val_loss: 3.5142 - val_accuracy: 0.5263
Epoch 499/1000
177/177 [==============================] - 0s 154us/step - loss: 0.0168 - accuracy: 1.0000 - val_loss: 3.5421 - val_accuracy: 0.5132
Epoch 500/1000
177/177 [==============================] - 0s 145us/step - loss: 0.0146 - accuracy: 1.0000 - val_loss: 3.5427 - val_accuracy: 0.5132
Epoch 501/1000
177/177 [==============================] - 0s 125us/step - loss: 0.0145 - accuracy: 1.0000 - val_loss: 3.5412 - val_accuracy: 0.5132
Epoch 502/1000
177/177 [==============================] - 0s 153us/step - loss: 0.0148 - accuracy: 1.0000 - val_loss: 3.5620 - val_accuracy: 0.5132
Epoch 503/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0156 - accuracy: 1.0000 - val_loss: 3.5772 - 

177/177 [==============================] - 0s 211us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 4.0222 - val_accuracy: 0.5263
Epoch 608/1000
177/177 [==============================] - 0s 159us/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 4.0382 - val_accuracy: 0.5263
Epoch 609/1000
177/177 [==============================] - 0s 204us/step - loss: 0.0086 - accuracy: 1.0000 - val_loss: 4.0507 - val_accuracy: 0.5263
Epoch 610/1000
177/177 [==============================] - 0s 170us/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 4.0406 - val_accuracy: 0.5263
Epoch 611/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 4.0563 - val_accuracy: 0.5263
Epoch 612/1000
177/177 [==============================] - 0s 198us/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 4.0855 - val_accuracy: 0.5000
Epoch 613/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 4.0637 - 

177/177 [==============================] - 0s 161us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 4.6268 - val_accuracy: 0.5132
Epoch 718/1000
177/177 [==============================] - 0s 208us/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 4.6209 - val_accuracy: 0.5000
Epoch 719/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 4.6360 - val_accuracy: 0.5132
Epoch 720/1000
177/177 [==============================] - 0s 233us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 4.6641 - val_accuracy: 0.5132
Epoch 721/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 4.6590 - val_accuracy: 0.5132
Epoch 722/1000
177/177 [==============================] - 0s 142us/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 4.6602 - val_accuracy: 0.5132
Epoch 723/1000
177/177 [==============================] - 0s 185us/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 4.6514 - 

177/177 [==============================] - 0s 216us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 5.0862 - val_accuracy: 0.5000
Epoch 828/1000
177/177 [==============================] - 0s 223us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 5.0956 - val_accuracy: 0.5132
Epoch 829/1000
177/177 [==============================] - 0s 235us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 5.0999 - val_accuracy: 0.5132
Epoch 830/1000
177/177 [==============================] - 0s 234us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 5.1083 - val_accuracy: 0.5132
Epoch 831/1000
177/177 [==============================] - 0s 197us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 5.0977 - val_accuracy: 0.5000
Epoch 832/1000
177/177 [==============================] - 0s 229us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 5.1154 - val_accuracy: 0.5132
Epoch 833/1000
177/177 [==============================] - 0s 299us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 5.1191 - 

177/177 [==============================] - 0s 174us/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 5.2223 - val_accuracy: 0.5132
Epoch 938/1000
177/177 [==============================] - 0s 225us/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 5.2344 - val_accuracy: 0.5000
Epoch 939/1000
177/177 [==============================] - 0s 197us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 5.2512 - val_accuracy: 0.5132
Epoch 940/1000
177/177 [==============================] - 0s 191us/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 5.2632 - val_accuracy: 0.5132
Epoch 941/1000
177/177 [==============================] - 0s 166us/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 5.2742 - val_accuracy: 0.5132
Epoch 942/1000
177/177 [==============================] - 0s 173us/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 5.2911 - val_accuracy: 0.5132
Epoch 943/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 5.3153 - 

In [103]:
acc_test4 = model4.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test4*100))

76/76 [==============================] - 0s 116us/step
test accuracy: 53.95%


In [59]:
pred4 = model4.predict_classes(X_test)
pred4

array([0, 0, 2, 0, 0, 0, 2, 0, 1, 2, 2, 1, 0, 0, 0, 1, 2, 0, 0, 2, 2, 1,
       0, 2, 1, 2, 2, 1, 2, 2, 0, 2, 2, 0, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1,
       0, 2, 0, 0, 1, 0, 0, 2, 1, 1, 1, 0, 1, 0, 2, 1, 0, 0, 0, 2, 1, 2,
       1, 1, 1, 1, 1, 2, 1, 2, 1, 2])

In [60]:
dat4['pred'] = pred4
dat4

,id,test,pred
54,CFBREBSa121,1,0
28,BCH-SA-13,1,0
143,NRS148,2,2
73,CFBRSa23,2,0
57,CFBREBSa125,0,0
...,...,...,...
208,NRS247,1,2
183,NRS215,1,1
248,SR4152,2,2
114,NRS035,1,1


In [61]:
proba4 = model4.predict_proba(X_test)
dat_proba4 = pd.DataFrame(proba4)

In [62]:
dat_proba4

,0,1,2
0,1.000000e+00,8.003374e-10,1.396647e-11
1,1.000000e+00,2.686744e-09,8.378914e-09
2,1.110505e-18,1.400358e-13,1.000000e+00
3,9.959140e-01,1.751107e-04,3.910834e-03
4,9.999990e-01,8.950273e-07,1.016467e-09
...,...,...,...
71,2.198302e-04,6.985904e-02,9.299211e-01
72,1.508720e-08,9.793884e-01,2.061171e-02
73,4.018819e-07,1.306746e-07,9.999995e-01
74,3.497802e-04,7.919186e-01,2.077317e-01


In [63]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [64]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p40p.csv", index = False,
         header=None)

In [107]:
hist4 = model4.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 210us/step - loss: 8.3860e-04 - accuracy: 1.0000 - val_loss: 5.2569 - val_accuracy: 0.5526
Epoch 2/1000
177/177 [==============================] - 0s 287us/step - loss: 8.3567e-04 - accuracy: 1.0000 - val_loss: 5.2608 - val_accuracy: 0.5395
Epoch 3/1000
177/177 [==============================] - 0s 195us/step - loss: 8.0461e-04 - accuracy: 1.0000 - val_loss: 5.2711 - val_accuracy: 0.5526
Epoch 4/1000
177/177 [==============================] - 0s 192us/step - loss: 8.0475e-04 - accuracy: 1.0000 - val_loss: 5.2744 - val_accuracy: 0.5526
Epoch 5/1000
177/177 [==============================] - 0s 180us/step - loss: 7.9112e-04 - accuracy: 1.0000 - val_loss: 5.2805 - val_accuracy: 0.5658
Epoch 6/1000
177/177 [==============================] - 0s 181us/step - loss: 8.2674e-04 - accuracy: 1.0000 - val_loss: 5.2840 - val_accuracy: 0.5526
Epoch 7/1000
177/177 [==============================] -

177/177 [==============================] - 0s 206us/step - loss: 6.7379e-04 - accuracy: 1.0000 - val_loss: 5.4479 - val_accuracy: 0.5395
Epoch 56/1000
177/177 [==============================] - 0s 214us/step - loss: 6.5583e-04 - accuracy: 1.0000 - val_loss: 5.4610 - val_accuracy: 0.5395
Epoch 57/1000
177/177 [==============================] - 0s 225us/step - loss: 7.4943e-04 - accuracy: 1.0000 - val_loss: 5.4653 - val_accuracy: 0.5395
Epoch 58/1000
177/177 [==============================] - 0s 233us/step - loss: 6.7711e-04 - accuracy: 1.0000 - val_loss: 5.4755 - val_accuracy: 0.5395
Epoch 59/1000
177/177 [==============================] - 0s 179us/step - loss: 6.4971e-04 - accuracy: 1.0000 - val_loss: 5.4621 - val_accuracy: 0.5395
Epoch 60/1000
177/177 [==============================] - 0s 325us/step - loss: 6.4162e-04 - accuracy: 1.0000 - val_loss: 5.4579 - val_accuracy: 0.5395
Epoch 61/1000
177/177 [==============================] - 0s 176us/step - loss: 6.4218e-04 - accuracy: 1.0000

177/177 [==============================] - 0s 336us/step - loss: 3.6220e-04 - accuracy: 1.0000 - val_loss: 5.8005 - val_accuracy: 0.5263
Epoch 164/1000
177/177 [==============================] - 0s 245us/step - loss: 4.2925e-04 - accuracy: 1.0000 - val_loss: 5.8165 - val_accuracy: 0.5395
Epoch 165/1000
177/177 [==============================] - 0s 251us/step - loss: 4.2164e-04 - accuracy: 1.0000 - val_loss: 5.8304 - val_accuracy: 0.5395
Epoch 166/1000
177/177 [==============================] - 0s 235us/step - loss: 4.7625e-04 - accuracy: 1.0000 - val_loss: 5.8311 - val_accuracy: 0.5395
Epoch 167/1000
177/177 [==============================] - 0s 244us/step - loss: 3.7949e-04 - accuracy: 1.0000 - val_loss: 5.8165 - val_accuracy: 0.5395
Epoch 168/1000
177/177 [==============================] - 0s 197us/step - loss: 3.7482e-04 - accuracy: 1.0000 - val_loss: 5.8149 - val_accuracy: 0.5263
Epoch 169/1000
177/177 [==============================] - 0s 188us/step - loss: 3.9053e-04 - accuracy: 

Epoch 271/1000
177/177 [==============================] - 0s 621us/step - loss: 4.4256e-04 - accuracy: 1.0000 - val_loss: 6.0143 - val_accuracy: 0.5395
Epoch 272/1000
177/177 [==============================] - 0s 262us/step - loss: 4.4522e-04 - accuracy: 1.0000 - val_loss: 6.0128 - val_accuracy: 0.5395
Epoch 273/1000
177/177 [==============================] - 0s 265us/step - loss: 4.4538e-04 - accuracy: 1.0000 - val_loss: 6.0167 - val_accuracy: 0.5395
Epoch 274/1000
177/177 [==============================] - 0s 325us/step - loss: 4.4643e-04 - accuracy: 1.0000 - val_loss: 6.0207 - val_accuracy: 0.5395
Epoch 275/1000
177/177 [==============================] - 0s 239us/step - loss: 4.7318e-04 - accuracy: 1.0000 - val_loss: 6.0211 - val_accuracy: 0.5263
Epoch 276/1000
177/177 [==============================] - 0s 329us/step - loss: 4.5879e-04 - accuracy: 1.0000 - val_loss: 6.0205 - val_accuracy: 0.5395
Epoch 277/1000
177/177 [==============================] - 0s 697us/step - loss: 4.4453e-

Epoch 325/1000
177/177 [==============================] - 0s 315us/step - loss: 3.7647e-04 - accuracy: 1.0000 - val_loss: 6.1427 - val_accuracy: 0.5395
Epoch 326/1000
177/177 [==============================] - 0s 323us/step - loss: 3.6751e-04 - accuracy: 1.0000 - val_loss: 6.1389 - val_accuracy: 0.5395
Epoch 327/1000
177/177 [==============================] - 0s 517us/step - loss: 3.7266e-04 - accuracy: 1.0000 - val_loss: 6.1389 - val_accuracy: 0.5395
Epoch 328/1000
177/177 [==============================] - 0s 303us/step - loss: 3.6099e-04 - accuracy: 1.0000 - val_loss: 6.1383 - val_accuracy: 0.5395
Epoch 329/1000
177/177 [==============================] - 0s 307us/step - loss: 3.6521e-04 - accuracy: 1.0000 - val_loss: 6.1362 - val_accuracy: 0.5395
Epoch 330/1000
177/177 [==============================] - 0s 249us/step - loss: 3.6321e-04 - accuracy: 1.0000 - val_loss: 6.1435 - val_accuracy: 0.5395
Epoch 331/1000
177/177 [==============================] - 0s 238us/step - loss: 3.6273e-

...

Epoch 649/1000
177/177 [==============================] - 0s 242us/step - loss: 1.0246e-04 - accuracy: 1.0000 - val_loss: 6.8329 - val_accuracy: 0.5132
Epoch 650/1000
177/177 [==============================] - 0s 290us/step - loss: 1.0377e-04 - accuracy: 1.0000 - val_loss: 6.8361 - val_accuracy: 0.5263
Epoch 651/1000
177/177 [==============================] - 0s 268us/step - loss: 1.0138e-04 - accuracy: 1.0000 - val_loss: 6.8376 - val_accuracy: 0.5263
Epoch 652/1000
177/177 [==============================] - 0s 280us/step - loss: 1.1144e-04 - accuracy: 1.0000 - val_loss: 6.8500 - val_accuracy: 0.5263
Epoch 653/1000
177/177 [==============================] - 0s 477us/step - loss: 1.0936e-04 - accuracy: 1.0000 - val_loss: 6.8471 - val_accuracy: 0.5263
Epoch 654/1000
177/177 [==============================] - 0s 293us/step - loss: 9.8724e-05 - accuracy: 1.0000 - val_loss: 6.8429 - val_accuracy: 0.5132
Epoch 655/1000
177/177 [==============================] - 0s 229us/step - loss: 1.0291e-

Epoch 756/1000
177/177 [==============================] - 0s 184us/step - loss: 8.4107e-05 - accuracy: 1.0000 - val_loss: 7.0537 - val_accuracy: 0.5132
Epoch 757/1000
177/177 [==============================] - 0s 254us/step - loss: 8.4373e-05 - accuracy: 1.0000 - val_loss: 7.0559 - val_accuracy: 0.5132
Epoch 758/1000
177/177 [==============================] - 0s 144us/step - loss: 8.5498e-05 - accuracy: 1.0000 - val_loss: 7.0540 - val_accuracy: 0.5132
Epoch 759/1000
177/177 [==============================] - 0s 129us/step - loss: 8.4843e-05 - accuracy: 1.0000 - val_loss: 7.0617 - val_accuracy: 0.5263
Epoch 760/1000
177/177 [==============================] - 0s 138us/step - loss: 8.3436e-05 - accuracy: 1.0000 - val_loss: 7.0543 - val_accuracy: 0.5132
Epoch 761/1000
177/177 [==============================] - ETA: 0s - loss: 3.9432e-05 - accuracy: 1.00 - 0s 183us/step - loss: 8.2296e-05 - accuracy: 1.0000 - val_loss: 7.0474 - val_accuracy: 0.5132
Epoch 762/1000
177/177 [==================

Epoch 863/1000
177/177 [==============================] - 0s 129us/step - loss: 5.0603e-05 - accuracy: 1.0000 - val_loss: 7.3247 - val_accuracy: 0.5132
Epoch 864/1000
177/177 [==============================] - 0s 175us/step - loss: 5.3927e-05 - accuracy: 1.0000 - val_loss: 7.3336 - val_accuracy: 0.5132
Epoch 865/1000
177/177 [==============================] - 0s 150us/step - loss: 5.1198e-05 - accuracy: 1.0000 - val_loss: 7.3392 - val_accuracy: 0.5132
Epoch 866/1000
177/177 [==============================] - 0s 155us/step - loss: 4.9407e-05 - accuracy: 1.0000 - val_loss: 7.3398 - val_accuracy: 0.5132
Epoch 867/1000
177/177 [==============================] - 0s 161us/step - loss: 4.9680e-05 - accuracy: 1.0000 - val_loss: 7.3398 - val_accuracy: 0.5132
Epoch 868/1000
177/177 [==============================] - 0s 140us/step - loss: 4.9627e-05 - accuracy: 1.0000 - val_loss: 7.3383 - val_accuracy: 0.5132
Epoch 869/1000
177/177 [==============================] - 0s 170us/step - loss: 5.0748e-

Epoch 917/1000
177/177 [==============================] - 0s 182us/step - loss: 4.3046e-05 - accuracy: 1.0000 - val_loss: 7.4702 - val_accuracy: 0.5132
Epoch 918/1000
177/177 [==============================] - 0s 166us/step - loss: 4.0603e-05 - accuracy: 1.0000 - val_loss: 7.4747 - val_accuracy: 0.5132
Epoch 919/1000
177/177 [==============================] - 0s 171us/step - loss: 4.0479e-05 - accuracy: 1.0000 - val_loss: 7.4771 - val_accuracy: 0.5132
Epoch 920/1000
177/177 [==============================] - 0s 161us/step - loss: 4.0261e-05 - accuracy: 1.0000 - val_loss: 7.4761 - val_accuracy: 0.5132
Epoch 921/1000
177/177 [==============================] - 0s 152us/step - loss: 3.9714e-05 - accuracy: 1.0000 - val_loss: 7.4816 - val_accuracy: 0.5132
Epoch 922/1000
177/177 [==============================] - 0s 171us/step - loss: 3.9982e-05 - accuracy: 1.0000 - val_loss: 7.4804 - val_accuracy: 0.5132
Epoch 923/1000
177/177 [==============================] - 0s 173us/step - loss: 3.9172e-

Epoch 971/1000
177/177 [==============================] - 0s 249us/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 7.5276 - val_accuracy: 0.5263
Epoch 972/1000
177/177 [==============================] - 0s 240us/step - loss: 0.1383 - accuracy: 0.9661 - val_loss: 8.0714 - val_accuracy: 0.5132
Epoch 973/1000
177/177 [==============================] - 0s 225us/step - loss: 0.1243 - accuracy: 0.9548 - val_loss: 8.1949 - val_accuracy: 0.5000
Epoch 974/1000
177/177 [==============================] - 0s 275us/step - loss: 0.3410 - accuracy: 0.9209 - val_loss: 7.5707 - val_accuracy: 0.5000
Epoch 975/1000
177/177 [==============================] - 0s 303us/step - loss: 0.1121 - accuracy: 0.9718 - val_loss: 7.6734 - val_accuracy: 0.5000
Epoch 976/1000
177/177 [==============================] - 0s 310us/step - loss: 0.0374 - accuracy: 0.9944 - val_loss: 7.1840 - val_accuracy: 0.5000
Epoch 977/1000
177/177 [==============================] - 0s 316us/step - loss: 0.0267 - accuracy: 0.9831 - val_

In [108]:
print('train accuracy: %.2f%%' % (np.mean(hist4.history['accuracy'])*100))

train accuracy: 99.95%


In [51]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [52]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,SR1129,2,2,4.821690e-07,0.000001,9.999983e-01
1,p0017kpresabs_qual,CA105,2,2,2.652370e-03,0.000049,9.972990e-01
2,p0017kpresabs_qual,NRS175,1,1,1.198157e-02,0.988018,5.232074e-09
3,p0017kpresabs_qual,EUH25,2,1,1.388957e-01,0.735954,1.251503e-01
4,p0017kpresabs_qual,NRS070,2,1,7.231966e-03,0.810317,1.824512e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS247,1,2,2.957360e-02,0.385730,5.846961e-01
604,p0040presabsSTCC_qual,NRS215,1,2,3.196635e-03,0.445057,5.517461e-01
605,p0040presabsSTCC_qual,SR4152,2,2,4.319404e-02,0.028984,9.278219e-01
606,p0040presabsSTCC_qual,NRS035,1,1,4.902312e-03,0.680427,3.146706e-01


In [53]:
y_prob4 = df_proba4[df_proba4['phage']=='p0040presabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[1.00000000e+00, 8.00337400e-10, 1.39664670e-11],
       [1.00000000e+00, 2.68674420e-09, 8.37891400e-09],
       [1.11050510e-18, 1.40035840e-13, 1.00000000e+00],
       [9.95914040e-01, 1.75110680e-04, 3.91083350e-03],
       [9.99999050e-01, 8.95027300e-07, 1.01646690e-09],
       [9.99375050e-01, 6.21805900e-04, 3.24929300e-06],
       [2.41182940e-06, 1.18025110e-07, 9.99997500e-01],
       [9.99995350e-01, 3.04297330e-13, 4.60682600e-06],
       [9.06085200e-04, 9.88204960e-01, 1.08889100e-02],
       [2.09726840e-06, 2.73453000e-01, 7.26544860e-01],
       [4.71095330e-19, 8.31724950e-08, 9.99999900e-01],
       [1.95520180e-09, 9.99950500e-01, 4.94701740e-05],
       [9.99999760e-01, 1.96140160e-07, 1.16029020e-11],
       [9.99653100e-01, 3.46847600e-04, 2.10411010e-08],
       [1.00000000e+00, 3.37264720e-08, 2.88732790e-11],
       [1.78780280e-07, 9.99990600e-01, 9.25574500e-06],
       [3.70803570e-09, 6.61460130e-09, 1.00000000e+00],
       [9.99999760e-01, 2.49455

In [54]:
ovo4 = rocauc_ovo(y_test, y_prob4, average="macro", multi_class="ovo")
ovo4

0.6644390945643898

In [55]:
ovr4 = rocauc_ovr(y_test, y_prob4, average="macro", multi_class="ovr")
ovr4

0.6644390945643898

In [56]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.7052455566198943

In [57]:
np.std(ovos)

0.05695460488804243

In [58]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.7052455566198943

In [59]:
np.std(ovrs)

0.05695460488804243

In [109]:
accs = [acc_test1, acc_test2, acc_test3, acc_test4]

In [110]:
mean = np.mean(accs)
print('test accuracy mean: %.2f%%' % (mean*100))

test accuracy mean: 54.28%


In [111]:
std = np.std(accs)
print('test accuracy standard deviation:', std)

test accuracy standard deviation: 0.044010153703062105


In [112]:
accs_train = [np.mean(hist1.history['accuracy']), np.mean(hist2.history['accuracy']), np.mean(hist3.history['accuracy']),
             np.mean(hist4.history['accuracy'])]

In [113]:
mean_train = np.mean(accs_train)
print('train accuracy mean: %.2f%%' % (mean_train*100))

train accuracy mean: 99.29%


In [114]:
std_train = np.std(accs_train)
print('train accuracy standard deviation:', std_train)

train accuracy standard deviation: 0.004310013


In [108]:
############ Feature selection using lasso ##########

In [60]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [61]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X.loc[:, X.columns != 'id'], y)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [62]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [63]:
X_train_features = np.vstack((names, X.loc[:, X.columns != 'id']))
X_train_features = pd.DataFrame(X_train_features)

In [64]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 349
selected features: 120


In [65]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  0,  12,  41,  44,  56,  62,  65,  68,  70,  71,  75,  76,  81,
         82,  83,  87,  88,  93,  94, 113, 116, 125, 126, 132, 134, 135,
        146, 148, 153, 164, 174, 180, 183, 184, 185, 186, 188, 189, 191,
        192, 193, 195, 196, 197, 198, 200, 201, 202, 203, 207, 210, 211,
        213, 215, 216, 217, 219, 220, 221, 223, 225, 227, 228, 230, 231,
        232, 236, 237, 239, 241, 244, 246, 247, 248, 251, 252, 253, 254,
        255, 263, 266, 269, 271, 273, 274, 278, 281, 284, 288, 291, 294,
        295, 298, 299, 301, 303, 304, 306, 308, 309, 311, 312, 313, 315,
        316, 318, 319, 321, 323, 324, 325, 326, 329, 330, 332, 333, 334,
        340, 342, 345]])

In [66]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT',
       'TTTCACTGCCTGT', 'TGTCTGATTTTTT',
       'TGGCTTAGGCTATAAATTGGAAAGACCTAAAGAACAATGATGAAGTTTCACCACCGCTTAATGTTATTGATAAGCAGTATATTAATTATCAGTTTTGTTA',
       'TGAACGTTATCT', 'TCGTGTCTGT', 'TCGCTGAAATATT',
       'TCGCTGAAATATTTGCGACAT', 'TCATCAAACTTT', 'TCAGTAGAGAT',
       'TCACACCGCCT', 'TCACACCGCCTAT', 'TAGTCATACAAT', 'TAGCTAAATCC',
       'TAGATTCAAATAT', 'TAATGGTAGTAGATAATTTTTC', 'TAATCTTGTTGTT',
       'TAAATTCCATAC',
       'TAAATCGTCTTTATATTTAATTATTAAATTAACAAATTTTAATTGGCGGATGAGGTATCCAGTTACCTCGTTCGCCAATTATTTTTCGCAATATAAAAAG',
       'GTGGCTTAGGCTATAAATTGGAAAGACCTAAAGAACAATGATGAAGTTTCACCACCGCTTAATGTTATTGATAAGCAGTATATTAATTATCAGTTTTGTT',
       'GTGAATTCATG', 'GGAGGATGAG', 'GGAGGATGAGG', 'GCTAATTATGTTTCTGGATT',
       'GAGGAGGATGAGG', 'GAGGAAGCAGAT', 'CTGTTTAATGATT', 'CTAATCCTTCAAT',
       'CGCTGAAATATTTGCG', 'ATGTCGCAAATATTTCAGCGACTTGT',
       'ACTTTTTATATTGCGAAAAATAAT

In [67]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [68]:
df_sel['strain'] = X.iloc[:,0]

In [69]:
df_sel

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTCACTGCCTGT,TGTCTGATTTTTT,TGGCTTAGGCTATAAATTGGAAAGACCTAAAGAACAATGATGAAGTTTCACCACCGCTTAATGTTATTGATAAGCAGTATATTAATTATCAGTTTTGTTA,TGAACGTTATCT,TCGTGTCTGT,TCGCTGAAATATT,TCGCTGAAATATTTGCGACAT,TCATCAAACTTT,TCAGTAGAGAT,...,dus_1,group_4168,group_861,group_862,group_210,group_2236,group_8800,group_8093,pheno,strain
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2,107
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,109
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,2,115
3,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,2,120335
4,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,2,SR4152
249,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,SR4153
250,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,SR4155
251,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,2,SR4156


In [70]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 121) (253,) (253, 122)


In [71]:
df_sel['pheno'].value_counts()

2    95
1    94
0    64
Name: pheno, dtype: int64

In [72]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel)

In [73]:
dat5 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat5['test'] = y_sel_test

In [74]:
dat5

,strain,test
251,SR4156,2
17,BCH-SA-02,0
158,NRS180,2
232,NY417,1
47,CFBREBSa110,2
...,...,...
62,CFBREBSa131,2
138,NRS112,2
21,BCH-SA-06,0
15,834N,2


In [75]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [76]:
#### neural network on over-sampling data
model_sel = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [82]:
model_sel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
model_sel.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 638us/step - loss: 1.1217 - accuracy: 0.3898 - val_loss: 1.0287 - val_accuracy: 0.4737
Epoch 2/1000
177/177 [==============================] - 0s 168us/step - loss: 0.9833 - accuracy: 0.4972 - val_loss: 1.0013 - val_accuracy: 0.3947
Epoch 3/1000
177/177 [==============================] - 0s 172us/step - loss: 0.9347 - accuracy: 0.5198 - val_loss: 1.0024 - val_accuracy: 0.3947
Epoch 4/1000
177/177 [==============================] - 0s 174us/step - loss: 0.9035 - accuracy: 0.5367 - val_loss: 1.0071 - val_accuracy: 0.4474
Epoch 5/1000
177/177 [==============================] - 0s 208us/step - loss: 0.8817 - accuracy: 0.5424 - val_loss: 1.0137 - val_accuracy: 0.4342
Epoch 6/1000
177/177 [==============================] - 0s 251us/step - loss: 0.8541 - accuracy: 0.5593 - val_loss: 1.0172 - val_accuracy: 0.4474
Epoch 7/1000
177/177 [==============================] - 0s 234us/step - loss: 0

177/177 [==============================] - 0s 252us/step - loss: 0.1860 - accuracy: 0.9379 - val_loss: 1.4687 - val_accuracy: 0.4211
Epoch 112/1000
177/177 [==============================] - 0s 272us/step - loss: 0.1845 - accuracy: 0.9322 - val_loss: 1.4956 - val_accuracy: 0.4342
Epoch 113/1000
177/177 [==============================] - 0s 243us/step - loss: 0.1784 - accuracy: 0.9435 - val_loss: 1.4824 - val_accuracy: 0.4342
Epoch 114/1000
177/177 [==============================] - 0s 191us/step - loss: 0.1944 - accuracy: 0.9379 - val_loss: 1.5128 - val_accuracy: 0.4079
Epoch 115/1000
177/177 [==============================] - 0s 268us/step - loss: 0.2249 - accuracy: 0.9153 - val_loss: 1.6240 - val_accuracy: 0.4474
Epoch 116/1000
177/177 [==============================] - 0s 316us/step - loss: 0.1947 - accuracy: 0.9266 - val_loss: 1.5303 - val_accuracy: 0.4211
Epoch 117/1000
177/177 [==============================] - 0s 234us/step - loss: 0.1808 - accuracy: 0.9435 - val_loss: 1.5177 - 

177/177 [==============================] - 0s 116us/step - loss: 0.0959 - accuracy: 0.9774 - val_loss: 2.0159 - val_accuracy: 0.4474
Epoch 222/1000
177/177 [==============================] - 0s 110us/step - loss: 0.0916 - accuracy: 0.9831 - val_loss: 1.9880 - val_accuracy: 0.4342
Epoch 223/1000
177/177 [==============================] - 0s 113us/step - loss: 0.0877 - accuracy: 0.9831 - val_loss: 1.9742 - val_accuracy: 0.4211
Epoch 224/1000
177/177 [==============================] - 0s 135us/step - loss: 0.0866 - accuracy: 0.9831 - val_loss: 1.9781 - val_accuracy: 0.4211
Epoch 225/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0858 - accuracy: 0.9831 - val_loss: 1.9818 - val_accuracy: 0.4211
Epoch 226/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0910 - accuracy: 0.9831 - val_loss: 1.9793 - val_accuracy: 0.4342
Epoch 227/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0905 - accuracy: 0.9774 - val_loss: 2.0082 - 

177/177 [==============================] - 0s 124us/step - loss: 0.0593 - accuracy: 0.9831 - val_loss: 2.3375 - val_accuracy: 0.4474
Epoch 332/1000
177/177 [==============================] - 0s 119us/step - loss: 0.0575 - accuracy: 0.9774 - val_loss: 2.3522 - val_accuracy: 0.4474
Epoch 333/1000
177/177 [==============================] - 0s 116us/step - loss: 0.0571 - accuracy: 0.9831 - val_loss: 2.3445 - val_accuracy: 0.4474
Epoch 334/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0579 - accuracy: 0.9831 - val_loss: 2.3490 - val_accuracy: 0.4605
Epoch 335/1000
177/177 [==============================] - 0s 132us/step - loss: 0.0563 - accuracy: 0.9774 - val_loss: 2.3728 - val_accuracy: 0.4474
Epoch 336/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0550 - accuracy: 0.9831 - val_loss: 2.3677 - val_accuracy: 0.4342
Epoch 337/1000
177/177 [==============================] - 0s 121us/step - loss: 0.0533 - accuracy: 0.9831 - val_loss: 2.3748 - 

177/177 [==============================] - 0s 137us/step - loss: 0.0444 - accuracy: 0.9831 - val_loss: 2.7609 - val_accuracy: 0.4737
Epoch 442/1000
177/177 [==============================] - 0s 171us/step - loss: 0.0407 - accuracy: 0.9831 - val_loss: 2.7682 - val_accuracy: 0.4737
Epoch 443/1000
177/177 [==============================] - 0s 314us/step - loss: 0.0381 - accuracy: 0.9887 - val_loss: 2.7866 - val_accuracy: 0.4737
Epoch 444/1000
177/177 [==============================] - 0s 300us/step - loss: 0.0375 - accuracy: 0.9831 - val_loss: 2.7973 - val_accuracy: 0.4605
Epoch 445/1000
177/177 [==============================] - 0s 321us/step - loss: 0.0359 - accuracy: 0.9887 - val_loss: 2.7846 - val_accuracy: 0.4605
Epoch 446/1000
177/177 [==============================] - 0s 286us/step - loss: 0.0372 - accuracy: 0.9887 - val_loss: 2.7837 - val_accuracy: 0.4474
Epoch 447/1000
177/177 [==============================] - 0s 282us/step - loss: 0.0367 - accuracy: 0.9887 - val_loss: 2.8233 - 

177/177 [==============================] - 0s 255us/step - loss: 0.0298 - accuracy: 0.9774 - val_loss: 3.1747 - val_accuracy: 0.4211
Epoch 552/1000
177/177 [==============================] - 0s 132us/step - loss: 0.0282 - accuracy: 0.9887 - val_loss: 3.1617 - val_accuracy: 0.4211
Epoch 553/1000
177/177 [==============================] - 0s 144us/step - loss: 0.0264 - accuracy: 0.9887 - val_loss: 3.1438 - val_accuracy: 0.4211
Epoch 554/1000
177/177 [==============================] - 0s 198us/step - loss: 0.0280 - accuracy: 0.9774 - val_loss: 3.1389 - val_accuracy: 0.4474
Epoch 555/1000
177/177 [==============================] - 0s 320us/step - loss: 0.0268 - accuracy: 0.9887 - val_loss: 3.1438 - val_accuracy: 0.4342
Epoch 556/1000
177/177 [==============================] - 0s 217us/step - loss: 0.0265 - accuracy: 0.9887 - val_loss: 3.1418 - val_accuracy: 0.4342
Epoch 557/1000
177/177 [==============================] - 0s 157us/step - loss: 0.0257 - accuracy: 0.9831 - val_loss: 3.1419 - 

177/177 [==============================] - 0s 173us/step - loss: 0.0230 - accuracy: 0.9887 - val_loss: 3.4810 - val_accuracy: 0.4342
Epoch 662/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0226 - accuracy: 0.9887 - val_loss: 3.4804 - val_accuracy: 0.4342
Epoch 663/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0231 - accuracy: 0.9887 - val_loss: 3.4776 - val_accuracy: 0.4211
Epoch 664/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0256 - accuracy: 0.9887 - val_loss: 3.4857 - val_accuracy: 0.4211
Epoch 665/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0254 - accuracy: 0.9831 - val_loss: 3.4818 - val_accuracy: 0.4342
Epoch 666/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0228 - accuracy: 0.9887 - val_loss: 3.4830 - val_accuracy: 0.4342
Epoch 667/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0268 - accuracy: 0.9831 - val_loss: 3.4915 - 

177/177 [==============================] - 0s 112us/step - loss: 0.0207 - accuracy: 0.9887 - val_loss: 3.7350 - val_accuracy: 0.4605
Epoch 772/1000
177/177 [==============================] - 0s 119us/step - loss: 0.0210 - accuracy: 0.9831 - val_loss: 3.7564 - val_accuracy: 0.4474
Epoch 773/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0225 - accuracy: 0.9831 - val_loss: 3.7398 - val_accuracy: 0.4474
Epoch 774/1000
177/177 [==============================] - 0s 123us/step - loss: 0.0200 - accuracy: 0.9887 - val_loss: 3.7296 - val_accuracy: 0.4605
Epoch 775/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0209 - accuracy: 0.9831 - val_loss: 3.7425 - val_accuracy: 0.4605
Epoch 776/1000
177/177 [==============================] - 0s 122us/step - loss: 0.0222 - accuracy: 0.9831 - val_loss: 3.7498 - val_accuracy: 0.4605
Epoch 777/1000
177/177 [==============================] - 0s 135us/step - loss: 0.0200 - accuracy: 0.9887 - val_loss: 3.7729 - 

177/177 [==============================] - 0s 159us/step - loss: 0.0212 - accuracy: 0.9831 - val_loss: 3.9502 - val_accuracy: 0.4605
Epoch 882/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0210 - accuracy: 0.9887 - val_loss: 3.9444 - val_accuracy: 0.4605
Epoch 883/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0195 - accuracy: 0.9887 - val_loss: 3.9473 - val_accuracy: 0.4605
Epoch 884/1000
177/177 [==============================] - 0s 166us/step - loss: 0.0188 - accuracy: 0.9831 - val_loss: 3.9502 - val_accuracy: 0.4474
Epoch 885/1000
177/177 [==============================] - 0s 176us/step - loss: 0.0218 - accuracy: 0.9774 - val_loss: 3.9432 - val_accuracy: 0.4605
Epoch 886/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0229 - accuracy: 0.9887 - val_loss: 3.9602 - val_accuracy: 0.4474
Epoch 887/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0195 - accuracy: 0.9831 - val_loss: 3.9867 - 

177/177 [==============================] - 0s 158us/step - loss: 0.0195 - accuracy: 0.9887 - val_loss: 4.1856 - val_accuracy: 0.4342
Epoch 992/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0245 - accuracy: 0.9831 - val_loss: 4.1803 - val_accuracy: 0.4342
Epoch 993/1000
177/177 [==============================] - 0s 162us/step - loss: 0.0184 - accuracy: 0.9887 - val_loss: 4.1955 - val_accuracy: 0.4342
Epoch 994/1000
177/177 [==============================] - 0s 153us/step - loss: 0.0182 - accuracy: 0.9831 - val_loss: 4.2014 - val_accuracy: 0.4474
Epoch 995/1000
177/177 [==============================] - 0s 160us/step - loss: 0.0222 - accuracy: 0.9887 - val_loss: 4.2207 - val_accuracy: 0.4474
Epoch 996/1000
177/177 [==============================] - 0s 153us/step - loss: 0.0209 - accuracy: 0.9831 - val_loss: 4.1837 - val_accuracy: 0.4474
Epoch 997/1000
177/177 [==============================] - 0s 158us/step - loss: 0.0193 - accuracy: 0.9887 - val_loss: 4.1836 - 

In [146]:
acc_test_sel = model_sel.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel*100))

76/76 [==============================] - 0s 95us/step
over-sampling test accuracy: 47.37%


In [84]:
pred5 = model_sel.predict_classes(X_sel_test)
pred5

array([2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 0, 0, 2, 2, 2, 1, 1,
       2, 0, 2, 1, 0, 1, 2, 0, 0, 2, 2, 1, 2, 0, 2, 2, 2, 1, 0, 1, 1, 0,
       1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0,
       2, 2, 1, 1, 1, 1, 1, 1, 2, 0])

In [85]:
dat5['pred'] = pred5
dat5

,strain,test,pred
251,SR4156,2,2
17,BCH-SA-02,0,1
158,NRS180,2,2
232,NY417,1,1
47,CFBREBSa110,2,2
...,...,...,...
62,CFBREBSa131,2,1
138,NRS112,2,1
21,BCH-SA-06,0,1
15,834N,2,2


In [86]:
proba5 = model_sel.predict_proba(X_sel_test)
dat_proba5 = pd.DataFrame(proba5)

In [87]:
dat_proba5

,0,1,2
0,1.091314e-06,4.212722e-01,0.578727
1,4.947578e-01,4.958048e-01,0.009438
2,8.248782e-12,2.430831e-05,0.999976
3,1.014371e-03,9.198223e-01,0.079163
4,6.187982e-03,4.270526e-03,0.989541
...,...,...,...
71,2.007713e-08,9.794039e-01,0.020596
72,3.642940e-11,9.406140e-01,0.059386
73,3.146670e-01,6.621915e-01,0.023142
74,8.501248e-15,1.828658e-11,1.000000


In [88]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [89]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p40p.csv", index = False,
         header=None)

In [150]:
hist_sel = model_sel.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 217us/step - loss: 0.0184 - accuracy: 0.9831 - val_loss: 4.4264 - val_accuracy: 0.4737
Epoch 2/1000
177/177 [==============================] - 0s 244us/step - loss: 0.0194 - accuracy: 0.9831 - val_loss: 4.4030 - val_accuracy: 0.4868
Epoch 3/1000
177/177 [==============================] - 0s 209us/step - loss: 0.0192 - accuracy: 0.9831 - val_loss: 4.4425 - val_accuracy: 0.4737
Epoch 4/1000
177/177 [==============================] - 0s 189us/step - loss: 0.0178 - accuracy: 0.9887 - val_loss: 4.4209 - val_accuracy: 0.4868
Epoch 5/1000
177/177 [==============================] - 0s 184us/step - loss: 0.0175 - accuracy: 0.9887 - val_loss: 4.4053 - val_accuracy: 0.4868
Epoch 6/1000
177/177 [==============================] - 0s 248us/step - loss: 0.0186 - accuracy: 0.9887 - val_loss: 4.4007 - val_accuracy: 0.4737
Epoch 7/1000
177/177 [==============================] - 0s 197us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 570us/step - loss: 0.0188 - accuracy: 0.9887 - val_loss: 4.6401 - val_accuracy: 0.4737
Epoch 113/1000
177/177 [==============================] - 0s 476us/step - loss: 0.0198 - accuracy: 0.9774 - val_loss: 4.6027 - val_accuracy: 0.4868
Epoch 114/1000
177/177 [==============================] - 0s 288us/step - loss: 0.0188 - accuracy: 0.9887 - val_loss: 4.6070 - val_accuracy: 0.4868
Epoch 115/1000
177/177 [==============================] - 0s 243us/step - loss: 0.0195 - accuracy: 0.9831 - val_loss: 4.6193 - val_accuracy: 0.4868
Epoch 116/1000
177/177 [==============================] - 0s 186us/step - loss: 0.0172 - accuracy: 0.9887 - val_loss: 4.6461 - val_accuracy: 0.4737
Epoch 117/1000
177/177 [==============================] - 0s 228us/step - loss: 0.0193 - accuracy: 0.9887 - val_loss: 4.6555 - val_accuracy: 0.4737
Epoch 118/1000
177/177 [==============================] - 0s 231us/step - loss: 0.0235 - accuracy: 0.9774 - val_

177/177 [==============================] - 0s 183us/step - loss: 0.0216 - accuracy: 0.9887 - val_loss: 4.8687 - val_accuracy: 0.4605
Epoch 168/1000
177/177 [==============================] - 0s 164us/step - loss: 0.0224 - accuracy: 0.9887 - val_loss: 4.8675 - val_accuracy: 0.4868
Epoch 169/1000
177/177 [==============================] - 0s 173us/step - loss: 0.0240 - accuracy: 0.9774 - val_loss: 4.8843 - val_accuracy: 0.4605
Epoch 170/1000
177/177 [==============================] - 0s 217us/step - loss: 0.0191 - accuracy: 0.9887 - val_loss: 4.8404 - val_accuracy: 0.4868
Epoch 171/1000
177/177 [==============================] - 0s 346us/step - loss: 0.0206 - accuracy: 0.9887 - val_loss: 4.8302 - val_accuracy: 0.4868
Epoch 172/1000
177/177 [==============================] - 0s 269us/step - loss: 0.0201 - accuracy: 0.9831 - val_loss: 4.8454 - val_accuracy: 0.4605
Epoch 173/1000
177/177 [==============================] - 0s 235us/step - loss: 0.0217 - accuracy: 0.9831 - val_loss: 4.8304 - 

177/177 [==============================] - 0s 530us/step - loss: 0.0189 - accuracy: 0.9887 - val_loss: 4.9348 - val_accuracy: 0.4605
Epoch 278/1000
177/177 [==============================] - 0s 284us/step - loss: 0.0187 - accuracy: 0.9887 - val_loss: 4.9090 - val_accuracy: 0.4737
Epoch 279/1000
177/177 [==============================] - 0s 234us/step - loss: 0.0185 - accuracy: 0.9887 - val_loss: 4.9106 - val_accuracy: 0.4737
Epoch 280/1000
177/177 [==============================] - 0s 201us/step - loss: 0.0176 - accuracy: 0.9887 - val_loss: 4.9420 - val_accuracy: 0.4737
Epoch 281/1000
177/177 [==============================] - 0s 226us/step - loss: 0.0169 - accuracy: 0.9887 - val_loss: 4.9574 - val_accuracy: 0.4605
Epoch 282/1000
177/177 [==============================] - 0s 398us/step - loss: 0.0180 - accuracy: 0.9887 - val_loss: 4.9463 - val_accuracy: 0.4605
Epoch 283/1000
177/177 [==============================] - 0s 401us/step - loss: 0.0193 - accuracy: 0.9831 - val_loss: 4.9464 - 

177/177 [==============================] - 0s 174us/step - loss: 0.0175 - accuracy: 0.9887 - val_loss: 4.9591 - val_accuracy: 0.5000
Epoch 388/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0195 - accuracy: 0.9887 - val_loss: 4.9276 - val_accuracy: 0.4868
Epoch 389/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0193 - accuracy: 0.9831 - val_loss: 4.9220 - val_accuracy: 0.4868
Epoch 390/1000
177/177 [==============================] - 0s 178us/step - loss: 0.0170 - accuracy: 0.9887 - val_loss: 4.9320 - val_accuracy: 0.4737
Epoch 391/1000
177/177 [==============================] - 0s 530us/step - loss: 0.0182 - accuracy: 0.9831 - val_loss: 4.9305 - val_accuracy: 0.4868
Epoch 392/1000
177/177 [==============================] - 0s 336us/step - loss: 0.0181 - accuracy: 0.9831 - val_loss: 4.9446 - val_accuracy: 0.4737
Epoch 393/1000
177/177 [==============================] - 0s 477us/step - loss: 0.0183 - accuracy: 0.9774 - val_loss: 4.9385 - 

177/177 [==============================] - 0s 158us/step - loss: 0.0178 - accuracy: 0.9887 - val_loss: 4.9573 - val_accuracy: 0.4605
Epoch 498/1000
177/177 [==============================] - 0s 162us/step - loss: 0.0185 - accuracy: 0.9774 - val_loss: 4.9482 - val_accuracy: 0.4737
Epoch 499/1000
177/177 [==============================] - 0s 217us/step - loss: 0.0181 - accuracy: 0.9774 - val_loss: 4.9452 - val_accuracy: 0.4737
Epoch 500/1000
177/177 [==============================] - 0s 162us/step - loss: 0.0165 - accuracy: 0.9887 - val_loss: 4.9336 - val_accuracy: 0.4737
Epoch 501/1000
177/177 [==============================] - 0s 190us/step - loss: 0.0174 - accuracy: 0.9831 - val_loss: 4.9443 - val_accuracy: 0.4737
Epoch 502/1000
177/177 [==============================] - 0s 368us/step - loss: 0.0175 - accuracy: 0.9887 - val_loss: 4.9346 - val_accuracy: 0.4737
Epoch 503/1000
177/177 [==============================] - 0s 170us/step - loss: 0.0172 - accuracy: 0.9887 - val_loss: 4.9398 - 

177/177 [==============================] - 0s 640us/step - loss: 0.0215 - accuracy: 0.9887 - val_loss: 4.9105 - val_accuracy: 0.4605
Epoch 608/1000
177/177 [==============================] - 0s 317us/step - loss: 0.0198 - accuracy: 0.9831 - val_loss: 4.8848 - val_accuracy: 0.4605
Epoch 609/1000
177/177 [==============================] - 0s 221us/step - loss: 0.0213 - accuracy: 0.9887 - val_loss: 4.8540 - val_accuracy: 0.4737
Epoch 610/1000
177/177 [==============================] - 0s 185us/step - loss: 0.0171 - accuracy: 0.9887 - val_loss: 4.8788 - val_accuracy: 0.4474
Epoch 611/1000
177/177 [==============================] - 0s 162us/step - loss: 0.0200 - accuracy: 0.9887 - val_loss: 4.8966 - val_accuracy: 0.4474
Epoch 612/1000
177/177 [==============================] - 0s 160us/step - loss: 0.0192 - accuracy: 0.9831 - val_loss: 4.8949 - val_accuracy: 0.4737
Epoch 613/1000
177/177 [==============================] - 0s 146us/step - loss: 0.0180 - accuracy: 0.9831 - val_loss: 4.9075 - 

177/177 [==============================] - 0s 160us/step - loss: 0.0252 - accuracy: 0.9887 - val_loss: 4.9692 - val_accuracy: 0.4868
Epoch 718/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0224 - accuracy: 0.9887 - val_loss: 4.9384 - val_accuracy: 0.4868
Epoch 719/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0209 - accuracy: 0.9831 - val_loss: 4.9148 - val_accuracy: 0.4737
Epoch 720/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0203 - accuracy: 0.9887 - val_loss: 5.0883 - val_accuracy: 0.4737
Epoch 721/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0316 - accuracy: 0.9887 - val_loss: 5.1220 - val_accuracy: 0.4868
Epoch 722/1000
177/177 [==============================] - 0s 252us/step - loss: 0.0301 - accuracy: 0.9887 - val_loss: 5.0659 - val_accuracy: 0.4868
Epoch 723/1000
177/177 [==============================] - 0s 193us/step - loss: 0.0255 - accuracy: 0.9887 - val_loss: 5.0477 - 

177/177 [==============================] - 0s 162us/step - loss: 0.0174 - accuracy: 0.9887 - val_loss: 5.2977 - val_accuracy: 0.5000
Epoch 828/1000
177/177 [==============================] - 0s 168us/step - loss: 0.0183 - accuracy: 0.9774 - val_loss: 5.3394 - val_accuracy: 0.4868
Epoch 829/1000
177/177 [==============================] - 0s 194us/step - loss: 0.0168 - accuracy: 0.9774 - val_loss: 5.3283 - val_accuracy: 0.5000
Epoch 830/1000
177/177 [==============================] - 0s 218us/step - loss: 0.0167 - accuracy: 0.9887 - val_loss: 5.3324 - val_accuracy: 0.4868
Epoch 831/1000
177/177 [==============================] - 0s 163us/step - loss: 0.0176 - accuracy: 0.9831 - val_loss: 5.3520 - val_accuracy: 0.4868
Epoch 832/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0177 - accuracy: 0.9887 - val_loss: 5.3367 - val_accuracy: 0.4868
Epoch 833/1000
177/177 [==============================] - 0s 184us/step - loss: 0.0164 - accuracy: 0.9887 - val_loss: 5.3178 - 

177/177 [==============================] - 0s 182us/step - loss: 0.0184 - accuracy: 0.9887 - val_loss: 5.3868 - val_accuracy: 0.5000
Epoch 938/1000
177/177 [==============================] - 0s 225us/step - loss: 0.0179 - accuracy: 0.9831 - val_loss: 5.4013 - val_accuracy: 0.4868
Epoch 939/1000
177/177 [==============================] - 0s 178us/step - loss: 0.0176 - accuracy: 0.9831 - val_loss: 5.4049 - val_accuracy: 0.4868
Epoch 940/1000
177/177 [==============================] - 0s 146us/step - loss: 0.0177 - accuracy: 0.9887 - val_loss: 5.3802 - val_accuracy: 0.4868
Epoch 941/1000
177/177 [==============================] - 0s 201us/step - loss: 0.0167 - accuracy: 0.9887 - val_loss: 5.3983 - val_accuracy: 0.5000
Epoch 942/1000
177/177 [==============================] - 0s 177us/step - loss: 0.0215 - accuracy: 0.9831 - val_loss: 5.4393 - val_accuracy: 0.4868
Epoch 943/1000
177/177 [==============================] - 0s 175us/step - loss: 0.0207 - accuracy: 0.9774 - val_loss: 5.3694 - 

In [151]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel.history['accuracy'])*100))

train accuracy: 98.45%


In [77]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [78]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,1,3.848032e-03,0.996096,0.000056
1,p0017kpresabs_qual,NRS113,0,0,8.441349e-01,0.000015,0.155850
2,p0017kpresabs_qual,120337,2,2,1.466081e-02,0.005713,0.979626
3,p0017kpresabs_qual,CFBREBSa127,1,0,9.985392e-01,0.000350,0.001111
4,p0017kpresabs_qual,GA27,2,2,1.665667e-08,0.000005,0.999995
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa131,2,1,6.617341e-03,0.565843,0.427540
604,p0040presabsSTCC_qual,NRS112,2,2,1.055758e-03,0.045611,0.953333
605,p0040presabsSTCC_qual,BCH-SA-06,0,0,8.747076e-01,0.071162,0.054130
606,p0040presabsSTCC_qual,834N,2,2,2.593112e-06,0.000013,0.999985


In [79]:
y_prob5 = df_proba5[df_proba5['phage']=='p0040presabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.0913143e-06, 4.2127216e-01, 5.7872670e-01],
       [4.9475777e-01, 4.9580476e-01, 9.4375040e-03],
       [8.2487820e-12, 2.4308307e-05, 9.9997570e-01],
       [1.0143710e-03, 9.1982230e-01, 7.9163310e-02],
       [6.1879820e-03, 4.2705256e-03, 9.8954150e-01],
       [6.1879820e-03, 4.2705256e-03, 9.8954150e-01],
       [5.9689080e-14, 5.3355325e-07, 9.9999950e-01],
       [1.3654150e-04, 2.0491991e-04, 9.9965860e-01],
       [2.5053605e-18, 4.2520380e-09, 1.0000000e+00],
       [7.7749375e-04, 9.9921906e-01, 3.4275950e-06],
       [1.1288823e-06, 9.9899730e-01, 1.0016364e-03],
       [3.4138182e-05, 9.7878313e-01, 2.1182684e-02],
       [1.7271646e-04, 1.9572505e-03, 9.9786997e-01],
       [6.6944380e-02, 9.3300170e-01, 5.3894684e-05],
       [5.8121606e-07, 9.8714720e-01, 1.2852192e-02],
       [7.3444920e-01, 3.7005090e-04, 2.6518068e-01],
       [9.9998510e-01, 6.7893770e-07, 1.4228350e-05],
       [1.1731885e-01, 8.0351780e-02, 8.0232936e-01],
       [9.5585570e-04, 4.871

In [80]:
ovo5 = rocauc_ovo(y_sel_test, y_prob5, average="macro", multi_class="ovo")
ovo5

0.6301147288807253

In [81]:
ovr5 = rocauc_ovr(y_sel_test, y_prob5, average="macro", multi_class="ovr")
ovr5

0.6301147288807253

In [82]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel)

In [83]:
dat6 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat6['test'] = y_sel_test

In [84]:
dat6

,strain,test
152,NRS169,1
210,NRS249,2
218,NRS262,2
238,SR1746,1
113,NRS029,1
...,...,...
96,GA27,2
95,GA231,2
237,SR1287,0
14,506,2


In [85]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [86]:
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [95]:
model_sel2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 638us/step - loss: 1.5654 - accuracy: 0.3559 - val_loss: 1.3612 - val_accuracy: 0.2763
Epoch 2/1000
177/177 [==============================] - 0s 128us/step - loss: 1.2556 - accuracy: 0.3503 - val_loss: 1.1560 - val_accuracy: 0.3421
Epoch 3/1000
177/177 [==============================] - 0s 133us/step - loss: 1.0829 - accuracy: 0.3729 - val_loss: 1.0589 - val_accuracy: 0.4868
Epoch 4/1000
177/177 [==============================] - 0s 133us/step - loss: 1.0192 - accuracy: 0.4689 - val_loss: 1.0159 - val_accuracy: 0.4605
Epoch 5/1000
177/177 [==============================] - 0s 145us/step - loss: 0.9808 - accuracy: 0.4859 - val_loss: 0.9903 - val_accuracy: 0.5000
Epoch 6/1000
177/177 [==============================] - 0s 124us/step - loss: 0.9447 - accuracy: 0.5198 - val_loss: 0.9771 - val_accuracy: 0.4737
Epoch 7/1000
177/177 [==============================] - 0s 134us/step - loss: 0

Epoch 112/1000
177/177 [==============================] - 0s 142us/step - loss: 0.2096 - accuracy: 0.9492 - val_loss: 1.1736 - val_accuracy: 0.5263
Epoch 113/1000
177/177 [==============================] - 0s 116us/step - loss: 0.2142 - accuracy: 0.9379 - val_loss: 1.2211 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 134us/step - loss: 0.2058 - accuracy: 0.9548 - val_loss: 1.2266 - val_accuracy: 0.5132
Epoch 115/1000
177/177 [==============================] - 0s 174us/step - loss: 0.2054 - accuracy: 0.9492 - val_loss: 1.2158 - val_accuracy: 0.5132
Epoch 116/1000
177/177 [==============================] - 0s 153us/step - loss: 0.2016 - accuracy: 0.9492 - val_loss: 1.2215 - val_accuracy: 0.5132
Epoch 117/1000
177/177 [==============================] - 0s 148us/step - loss: 0.2038 - accuracy: 0.9548 - val_loss: 1.2238 - val_accuracy: 0.5132
Epoch 118/1000
177/177 [==============================] - 0s 180us/step - loss: 0.2049 - accuracy: 0.9492 - val_

177/177 [==============================] - 0s 124us/step - loss: 0.0970 - accuracy: 0.9661 - val_loss: 1.6788 - val_accuracy: 0.5000
Epoch 223/1000
177/177 [==============================] - 0s 125us/step - loss: 0.0954 - accuracy: 0.9661 - val_loss: 1.6720 - val_accuracy: 0.5132
Epoch 224/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0953 - accuracy: 0.9718 - val_loss: 1.6684 - val_accuracy: 0.5000
Epoch 225/1000
177/177 [==============================] - 0s 153us/step - loss: 0.0957 - accuracy: 0.9718 - val_loss: 1.6812 - val_accuracy: 0.5000
Epoch 226/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0999 - accuracy: 0.9661 - val_loss: 1.6971 - val_accuracy: 0.5000
Epoch 227/1000
177/177 [==============================] - 0s 107us/step - loss: 0.0962 - accuracy: 0.9661 - val_loss: 1.6609 - val_accuracy: 0.5132
Epoch 228/1000
177/177 [==============================] - 0s 89us/step - loss: 0.0973 - accuracy: 0.9718 - val_loss: 1.6600 - v

Epoch 278/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0754 - accuracy: 0.9661 - val_loss: 1.8713 - val_accuracy: 0.4868
Epoch 279/1000
177/177 [==============================] - 0s 89us/step - loss: 0.0732 - accuracy: 0.9718 - val_loss: 1.8582 - val_accuracy: 0.4868
Epoch 280/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0750 - accuracy: 0.9774 - val_loss: 1.8557 - val_accuracy: 0.5132
Epoch 281/1000
177/177 [==============================] - 0s 99us/step - loss: 0.0738 - accuracy: 0.9718 - val_loss: 1.8664 - val_accuracy: 0.4868
Epoch 282/1000
177/177 [==============================] - 0s 100us/step - loss: 0.0745 - accuracy: 0.9718 - val_loss: 1.8906 - val_accuracy: 0.4737
Epoch 283/1000
177/177 [==============================] - 0s 177us/step - loss: 0.0748 - accuracy: 0.9718 - val_loss: 1.9075 - val_accuracy: 0.4868
Epoch 284/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0751 - accuracy: 0.9661 - val_los

177/177 [==============================] - 0s 142us/step - loss: 0.0553 - accuracy: 0.9718 - val_loss: 2.1775 - val_accuracy: 0.5000
Epoch 389/1000
177/177 [==============================] - 0s 194us/step - loss: 0.0535 - accuracy: 0.9774 - val_loss: 2.2077 - val_accuracy: 0.5000
Epoch 390/1000
177/177 [==============================] - 0s 133us/step - loss: 0.0511 - accuracy: 0.9831 - val_loss: 2.2201 - val_accuracy: 0.5000
Epoch 391/1000
177/177 [==============================] - 0s 145us/step - loss: 0.0529 - accuracy: 0.9831 - val_loss: 2.2180 - val_accuracy: 0.5000
Epoch 392/1000
177/177 [==============================] - 0s 130us/step - loss: 0.0523 - accuracy: 0.9831 - val_loss: 2.2320 - val_accuracy: 0.5000
Epoch 393/1000
177/177 [==============================] - 0s 109us/step - loss: 0.0516 - accuracy: 0.9831 - val_loss: 2.2046 - val_accuracy: 0.5000
Epoch 394/1000
177/177 [==============================] - 0s 129us/step - loss: 0.0509 - accuracy: 0.9831 - val_loss: 2.2188 - 

...

Epoch 609/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0348 - accuracy: 0.9831 - val_loss: 2.7005 - val_accuracy: 0.5132
Epoch 610/1000
177/177 [==============================] - 0s 100us/step - loss: 0.0352 - accuracy: 0.9774 - val_loss: 2.7054 - val_accuracy: 0.5000
Epoch 611/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0357 - accuracy: 0.9774 - val_loss: 2.7211 - val_accuracy: 0.5000
Epoch 612/1000
177/177 [==============================] - 0s 144us/step - loss: 0.0361 - accuracy: 0.9831 - val_loss: 2.7583 - val_accuracy: 0.5132
Epoch 613/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0354 - accuracy: 0.9831 - val_loss: 2.7518 - val_accuracy: 0.5132
Epoch 614/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0369 - accuracy: 0.9774 - val_loss: 2.7511 - val_accuracy: 0.5263
Epoch 615/1000
177/177 [==============================] - 0s 106us/step - loss: 0.0358 - accuracy: 0.9831 - val_l

Epoch 720/1000
177/177 [==============================] - 0s 148us/step - loss: 0.0328 - accuracy: 0.9718 - val_loss: 3.0155 - val_accuracy: 0.5000
Epoch 721/1000
177/177 [==============================] - 0s 160us/step - loss: 0.0320 - accuracy: 0.9831 - val_loss: 2.9853 - val_accuracy: 0.5000
Epoch 722/1000
177/177 [==============================] - 0s 135us/step - loss: 0.0326 - accuracy: 0.9774 - val_loss: 2.9475 - val_accuracy: 0.5000
Epoch 723/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0314 - accuracy: 0.9774 - val_loss: 2.9372 - val_accuracy: 0.5263
Epoch 724/1000
177/177 [==============================] - 0s 90us/step - loss: 0.0308 - accuracy: 0.9831 - val_loss: 2.9476 - val_accuracy: 0.5263
Epoch 725/1000
177/177 [==============================] - 0s 133us/step - loss: 0.0318 - accuracy: 0.9774 - val_loss: 2.9646 - val_accuracy: 0.5000
Epoch 726/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0314 - accuracy: 0.9774 - val_lo

177/177 [==============================] - 0s 79us/step - loss: 0.0315 - accuracy: 0.9831 - val_loss: 3.1670 - val_accuracy: 0.5000
Epoch 831/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0299 - accuracy: 0.9718 - val_loss: 3.1525 - val_accuracy: 0.5132
Epoch 832/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0288 - accuracy: 0.9831 - val_loss: 3.1312 - val_accuracy: 0.5000
Epoch 833/1000
177/177 [==============================] - 0s 140us/step - loss: 0.0298 - accuracy: 0.9831 - val_loss: 3.1249 - val_accuracy: 0.5000
Epoch 834/1000
177/177 [==============================] - 0s 109us/step - loss: 0.0281 - accuracy: 0.9887 - val_loss: 3.1298 - val_accuracy: 0.5132
Epoch 835/1000
177/177 [==============================] - 0s 97us/step - loss: 0.0323 - accuracy: 0.9774 - val_loss: 3.1648 - val_accuracy: 0.4868
Epoch 836/1000
177/177 [==============================] - 0s 99us/step - loss: 0.0305 - accuracy: 0.9831 - val_loss: 3.1502 - val_

177/177 [==============================] - 0s 73us/step - loss: 0.0284 - accuracy: 0.9831 - val_loss: 3.3679 - val_accuracy: 0.4737
Epoch 941/1000
177/177 [==============================] - 0s 89us/step - loss: 0.0298 - accuracy: 0.9774 - val_loss: 3.3062 - val_accuracy: 0.4737
Epoch 942/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0296 - accuracy: 0.9774 - val_loss: 3.2714 - val_accuracy: 0.4868
Epoch 943/1000
177/177 [==============================] - 0s 99us/step - loss: 0.0280 - accuracy: 0.9831 - val_loss: 3.2774 - val_accuracy: 0.5000
Epoch 944/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0291 - accuracy: 0.9831 - val_loss: 3.3138 - val_accuracy: 0.4868
Epoch 945/1000
177/177 [==============================] - 0s 92us/step - loss: 0.0304 - accuracy: 0.9774 - val_loss: 3.2987 - val_accuracy: 0.5000
Epoch 946/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0284 - accuracy: 0.9774 - val_loss: 3.3513 - val_acc

177/177 [==============================] - 0s 145us/step - loss: 0.0264 - accuracy: 0.9831 - val_loss: 3.4290 - val_accuracy: 0.4868
Epoch 996/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0266 - accuracy: 0.9774 - val_loss: 3.4467 - val_accuracy: 0.4737
Epoch 997/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0284 - accuracy: 0.9831 - val_loss: 3.4422 - val_accuracy: 0.4737
Epoch 998/1000
177/177 [==============================] - 0s 162us/step - loss: 0.0274 - accuracy: 0.9831 - val_loss: 3.4289 - val_accuracy: 0.4605
Epoch 999/1000
177/177 [==============================] - 0s 100us/step - loss: 0.0282 - accuracy: 0.9831 - val_loss: 3.4161 - val_accuracy: 0.4868
Epoch 1000/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0273 - accuracy: 0.9831 - val_loss: 3.4079 - val_accuracy: 0.5000


In [171]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

76/76 [==============================] - 0s 122us/step
test accuracy: 55.26%


In [97]:
pred6 = model_sel2.predict_classes(X_sel_test)
pred6

array([1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 0, 1, 1, 2, 2, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 1, 2, 2, 0, 2, 0, 0, 2, 1, 1, 2,
       1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 0, 0, 2, 0, 0,
       1, 2, 0, 2, 1, 1, 1, 1, 0, 1])

In [98]:
dat6['pred'] = pred6
dat6

,strain,test,pred
152,NRS169,1,1
210,NRS249,2,1
218,NRS262,2,1
238,SR1746,1,1
113,NRS029,1,1
...,...,...,...
96,GA27,2,1
95,GA231,2,1
237,SR1287,0,1
14,506,2,0


In [99]:
proba6 = model_sel2.predict_proba(X_sel_test)
dat_proba6 = pd.DataFrame(proba6)

In [100]:
dat_proba6

,0,1,2
0,1.079168e-08,0.993211,6.788571e-03
1,7.090530e-09,0.989655,1.034534e-02
2,1.517064e-04,0.999719,1.294190e-04
3,4.189974e-06,0.999926,6.996702e-05
4,3.448038e-05,0.999966,2.380547e-08
...,...,...,...
71,1.550768e-09,1.000000,2.249901e-08
72,3.175609e-05,0.999797,1.708810e-04
73,1.640366e-04,0.999836,6.729850e-12
74,9.127981e-01,0.018590,6.861152e-02


In [101]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [102]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p40p.csv", index = False,
         header=None)

In [175]:
hist_sel2 = model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 119us/step - loss: 0.0267 - accuracy: 0.9774 - val_loss: 3.3635 - val_accuracy: 0.5526
Epoch 2/1000
177/177 [==============================] - 0s 104us/step - loss: 0.0267 - accuracy: 0.9718 - val_loss: 3.3600 - val_accuracy: 0.5395
Epoch 3/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0268 - accuracy: 0.9831 - val_loss: 3.3514 - val_accuracy: 0.5395
Epoch 4/1000
177/177 [==============================] - 0s 171us/step - loss: 0.0271 - accuracy: 0.9774 - val_loss: 3.3374 - val_accuracy: 0.5395
Epoch 5/1000
177/177 [==============================] - 0s 159us/step - loss: 0.0276 - accuracy: 0.9774 - val_loss: 3.3414 - val_accuracy: 0.5526
Epoch 6/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0264 - accuracy: 0.9831 - val_loss: 3.3385 - val_accuracy: 0.5526
Epoch 7/1000
177/177 [==============================] - 0s 148us/step - loss: 0

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.109404). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 140us/step - loss: 0.0273 - accuracy: 0.9774 - val_loss: 3.4348 - val_accuracy: 0.5526
Epoch 105/1000
177/177 [==============================] - 0s 699us/step - loss: 0.0275 - accuracy: 0.9831 - val_loss: 3.4062 - val_accuracy: 0.5526
Epoch 106/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0269 - accuracy: 0.9774 - val_loss: 3.4392 - val_accuracy: 0.5526
Epoch 107/1000
177/177 [==============================] - 0s 200us/step - loss: 0.0267 - accuracy: 0.9831 - val_loss: 3.4423 - val_accuracy: 0.5526
Epoch 108/1000
177/177 [==============================] - 0s 144us/step - loss: 0.0277 - accuracy: 0.9831 - val_loss: 3.4469 - val_accuracy: 0.5526
Epoch 109/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0261 - accuracy: 0.9774 - val_loss: 3.4660 - val_accuracy: 0.5526
Epoch 110/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0266 - accuracy: 0.9774 - val_loss: 3.4880 - 

177/177 [==============================] - 0s 207us/step - loss: 0.0259 - accuracy: 0.9831 - val_loss: 3.5827 - val_accuracy: 0.5658
Epoch 215/1000
177/177 [==============================] - 0s 145us/step - loss: 0.0264 - accuracy: 0.9774 - val_loss: 3.5910 - val_accuracy: 0.5526
Epoch 216/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0261 - accuracy: 0.9774 - val_loss: 3.5729 - val_accuracy: 0.5526
Epoch 217/1000
177/177 [==============================] - 0s 106us/step - loss: 0.0265 - accuracy: 0.9831 - val_loss: 3.6085 - val_accuracy: 0.5263
Epoch 218/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0290 - accuracy: 0.9774 - val_loss: 3.6131 - val_accuracy: 0.5526
Epoch 219/1000
177/177 [==============================] - 0s 115us/step - loss: 0.0257 - accuracy: 0.9774 - val_loss: 3.5705 - val_accuracy: 0.5658
Epoch 220/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0258 - accuracy: 0.9831 - val_loss: 3.5758 - v

177/177 [==============================] - 0s 333us/step - loss: 0.0275 - accuracy: 0.9774 - val_loss: 3.6962 - val_accuracy: 0.5263
Epoch 325/1000
177/177 [==============================] - 0s 269us/step - loss: 0.0273 - accuracy: 0.9774 - val_loss: 3.7209 - val_accuracy: 0.5263
Epoch 326/1000
177/177 [==============================] - 0s 164us/step - loss: 0.0259 - accuracy: 0.9831 - val_loss: 3.7364 - val_accuracy: 0.5263
Epoch 327/1000
177/177 [==============================] - 0s 294us/step - loss: 0.0254 - accuracy: 0.9831 - val_loss: 3.7176 - val_accuracy: 0.5395
Epoch 328/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0260 - accuracy: 0.9831 - val_loss: 3.7062 - val_accuracy: 0.5395
Epoch 329/1000
177/177 [==============================] - 0s 210us/step - loss: 0.0257 - accuracy: 0.9831 - val_loss: 3.6362 - val_accuracy: 0.5658
Epoch 330/1000
177/177 [==============================] - 0s 133us/step - loss: 0.0253 - accuracy: 0.9831 - val_loss: 3.6276 - 

...

177/177 [==============================] - 0s 84us/step - loss: 0.0252 - accuracy: 0.9831 - val_loss: 3.8960 - val_accuracy: 0.5658
Epoch 545/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0262 - accuracy: 0.9831 - val_loss: 3.9033 - val_accuracy: 0.5658
Epoch 546/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0270 - accuracy: 0.9831 - val_loss: 3.8773 - val_accuracy: 0.5658
Epoch 547/1000
177/177 [==============================] - 0s 90us/step - loss: 0.0251 - accuracy: 0.9831 - val_loss: 3.8933 - val_accuracy: 0.5658
Epoch 548/1000
177/177 [==============================] - 0s 91us/step - loss: 0.0265 - accuracy: 0.9831 - val_loss: 3.9200 - val_accuracy: 0.5526
Epoch 549/1000
177/177 [==============================] - 0s 89us/step - loss: 0.0255 - accuracy: 0.9831 - val_loss: 3.9158 - val_accuracy: 0.5526
Epoch 550/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0245 - accuracy: 0.9831 - val_loss: 3.9368 - val_ac

177/177 [==============================] - 0s 122us/step - loss: 0.0263 - accuracy: 0.9831 - val_loss: 4.0338 - val_accuracy: 0.5132
Epoch 655/1000
177/177 [==============================] - 0s 181us/step - loss: 0.0252 - accuracy: 0.9774 - val_loss: 4.0340 - val_accuracy: 0.5395
Epoch 656/1000
177/177 [==============================] - 0s 286us/step - loss: 0.0263 - accuracy: 0.9831 - val_loss: 4.0159 - val_accuracy: 0.5395
Epoch 657/1000
177/177 [==============================] - 0s 357us/step - loss: 0.0257 - accuracy: 0.9831 - val_loss: 3.9795 - val_accuracy: 0.5526
Epoch 658/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0252 - accuracy: 0.9774 - val_loss: 3.9856 - val_accuracy: 0.5395
Epoch 659/1000
177/177 [==============================] - 0s 163us/step - loss: 0.0251 - accuracy: 0.9831 - val_loss: 3.9789 - val_accuracy: 0.5526
Epoch 660/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0252 - accuracy: 0.9831 - val_loss: 4.0068 - 

177/177 [==============================] - 0s 219us/step - loss: 0.0264 - accuracy: 0.9774 - val_loss: 4.1121 - val_accuracy: 0.5395
Epoch 765/1000
177/177 [==============================] - 0s 247us/step - loss: 0.0255 - accuracy: 0.9831 - val_loss: 4.1022 - val_accuracy: 0.5395
Epoch 766/1000
177/177 [==============================] - 0s 170us/step - loss: 0.0267 - accuracy: 0.9831 - val_loss: 4.0900 - val_accuracy: 0.5263
Epoch 767/1000
177/177 [==============================] - 0s 133us/step - loss: 0.0269 - accuracy: 0.9831 - val_loss: 4.1274 - val_accuracy: 0.5263
Epoch 768/1000
177/177 [==============================] - 0s 121us/step - loss: 0.0257 - accuracy: 0.9831 - val_loss: 4.1498 - val_accuracy: 0.5263
Epoch 769/1000
177/177 [==============================] - 0s 273us/step - loss: 0.0244 - accuracy: 0.9774 - val_loss: 4.1472 - val_accuracy: 0.5263
Epoch 770/1000
177/177 [==============================] - 0s 260us/step - loss: 0.0262 - accuracy: 0.9831 - val_loss: 4.1311 - 

177/177 [==============================] - 0s 100us/step - loss: 0.0252 - accuracy: 0.9831 - val_loss: 4.3037 - val_accuracy: 0.5263
Epoch 875/1000
177/177 [==============================] - 0s 126us/step - loss: 0.0252 - accuracy: 0.9831 - val_loss: 4.2877 - val_accuracy: 0.5263
Epoch 876/1000
177/177 [==============================] - 0s 220us/step - loss: 0.0266 - accuracy: 0.9831 - val_loss: 4.2717 - val_accuracy: 0.5263
Epoch 877/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0256 - accuracy: 0.9774 - val_loss: 4.2376 - val_accuracy: 0.5132
Epoch 878/1000
177/177 [==============================] - 0s 207us/step - loss: 0.0251 - accuracy: 0.9718 - val_loss: 4.2345 - val_accuracy: 0.5132
Epoch 879/1000
177/177 [==============================] - 0s 172us/step - loss: 0.0250 - accuracy: 0.9774 - val_loss: 4.2326 - val_accuracy: 0.5132
Epoch 880/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0263 - accuracy: 0.9718 - val_loss: 4.2647 - 

177/177 [==============================] - 0s 140us/step - loss: 0.0258 - accuracy: 0.9831 - val_loss: 4.3456 - val_accuracy: 0.5263
Epoch 985/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0263 - accuracy: 0.9831 - val_loss: 4.3382 - val_accuracy: 0.5132
Epoch 986/1000
177/177 [==============================] - 0s 136us/step - loss: 0.0254 - accuracy: 0.9774 - val_loss: 4.3507 - val_accuracy: 0.5000
Epoch 987/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0260 - accuracy: 0.9831 - val_loss: 4.3658 - val_accuracy: 0.5000
Epoch 988/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0258 - accuracy: 0.9831 - val_loss: 4.3498 - val_accuracy: 0.5000
Epoch 989/1000
177/177 [==============================] - 0s 113us/step - loss: 0.0251 - accuracy: 0.9831 - val_loss: 4.3852 - val_accuracy: 0.5132
Epoch 990/1000
177/177 [==============================] - 0s 93us/step - loss: 0.0252 - accuracy: 0.9718 - val_loss: 4.3850 - v

In [176]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel2.history['accuracy'])*100))

train accuracy: 98.01%


In [87]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [88]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,CFBREBSa104,1,2,1.290039e-09,1.567630e-07,9.999999e-01
1,p0017kpresabs_qual,NRS199,2,2,2.856965e-05,2.843749e-03,9.971277e-01
2,p0017kpresabs_qual,NRS233,1,0,9.999446e-01,4.541289e-06,5.090974e-05
3,p0017kpresabs_qual,SR1746,0,2,6.282367e-02,7.075194e-04,9.364688e-01
4,p0017kpresabs_qual,NRS202,2,2,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,GA27,2,1,3.964591e-03,9.959286e-01,1.068284e-04
604,p0040presabsSTCC_qual,GA231,2,1,3.309226e-04,9.996691e-01,6.232397e-10
605,p0040presabsSTCC_qual,SR1287,0,1,8.445997e-06,9.999915e-01,1.182947e-13
606,p0040presabsSTCC_qual,506,2,0,6.516150e-01,1.480882e-02,3.335762e-01


In [89]:
y_prob6 = df_proba6[df_proba6['phage']=='p0040presabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.07916770e-08, 9.93211400e-01, 6.78857140e-03],
       [7.09053000e-09, 9.89654660e-01, 1.03453430e-02],
       [1.51706360e-04, 9.99718840e-01, 1.29419000e-04],
       [4.18997400e-06, 9.99925850e-01, 6.99670200e-05],
       [3.44803850e-05, 9.99965550e-01, 2.38054660e-08],
       [2.21292540e-07, 9.99999760e-01, 8.94908700e-14],
       [3.21465860e-04, 4.17447400e-01, 5.82231100e-01],
       [3.42867850e-07, 8.03490200e-01, 1.96509380e-01],
       [6.42057800e-06, 9.99499200e-01, 4.94367970e-04],
       [9.80405500e-06, 9.99859200e-01, 1.30998030e-04],
       [7.31793800e-03, 3.37777020e-01, 6.54905000e-01],
       [3.61108430e-05, 1.46578940e-05, 9.99949200e-01],
       [6.39631750e-02, 9.36033000e-01, 3.84440100e-06],
       [5.97735050e-01, 3.83508440e-01, 1.87565600e-02],
       [1.93460510e-04, 9.35488100e-01, 6.43184100e-02],
       [2.95254160e-02, 9.70443070e-01, 3.14224600e-05],
       [1.02900015e-10, 9.91677700e-06, 9.99990100e-01],
       [1.71773890e-06, 3.98229

In [90]:
ovo6 = rocauc_ovo(y_sel_test, y_prob6, average="macro", multi_class="ovo")
ovo6

0.6716457848091352

In [91]:
ovr6 = rocauc_ovr(y_sel_test, y_prob6, average="macro", multi_class="ovr")
ovr6

0.6716457848091352

In [92]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel)

In [93]:
dat7 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat7['test'] = y_sel_test

In [94]:
dat7

,strain,test
120,NRS063,1
187,NRS219,2
96,GA27,2
53,CFBREBSa119,1
197,NRS233,2
...,...,...
47,CFBREBSa110,2
68,CFBRSa05,0
56,CFBREBSa123,0
231,NY360,1


In [95]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [96]:
model_sel3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [97]:
model_sel3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [109]:
model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 642us/step - loss: 1.3559 - accuracy: 0.2712 - val_loss: 1.1723 - val_accuracy: 0.3421
Epoch 2/1000
177/177 [==============================] - 0s 96us/step - loss: 1.1111 - accuracy: 0.3729 - val_loss: 1.0794 - val_accuracy: 0.4474
Epoch 3/1000
177/177 [==============================] - 0s 138us/step - loss: 1.0293 - accuracy: 0.4350 - val_loss: 1.0600 - val_accuracy: 0.4342
Epoch 4/1000
177/177 [==============================] - 0s 300us/step - loss: 0.9944 - accuracy: 0.4802 - val_loss: 1.0397 - val_accuracy: 0.4474
Epoch 5/1000
177/177 [==============================] - 0s 209us/step - loss: 0.9573 - accuracy: 0.5367 - val_loss: 1.0233 - val_accuracy: 0.4737
Epoch 6/1000
177/177 [==============================] - 0s 109us/step - loss: 0.9291 - accuracy: 0.5367 - val_loss: 1.0140 - val_accuracy: 0.4605
Epoch 7/1000
177/177 [==============================] - 0s 111us/step - loss: 0.

Epoch 112/1000
177/177 [==============================] - 0s 104us/step - loss: 0.1841 - accuracy: 0.9548 - val_loss: 1.1639 - val_accuracy: 0.5263
Epoch 113/1000
177/177 [==============================] - 0s 131us/step - loss: 0.1794 - accuracy: 0.9548 - val_loss: 1.1698 - val_accuracy: 0.5000
Epoch 114/1000
177/177 [==============================] - 0s 104us/step - loss: 0.1768 - accuracy: 0.9548 - val_loss: 1.1800 - val_accuracy: 0.5000
Epoch 115/1000
177/177 [==============================] - 0s 97us/step - loss: 0.1768 - accuracy: 0.9718 - val_loss: 1.1870 - val_accuracy: 0.4868
Epoch 116/1000
177/177 [==============================] - 0s 100us/step - loss: 0.1749 - accuracy: 0.9605 - val_loss: 1.1879 - val_accuracy: 0.5000
Epoch 117/1000
177/177 [==============================] - 0s 142us/step - loss: 0.1733 - accuracy: 0.9548 - val_loss: 1.1868 - val_accuracy: 0.5132
Epoch 118/1000
177/177 [==============================] - 0s 91us/step - loss: 0.1703 - accuracy: 0.9548 - val_lo

177/177 [==============================] - 0s 85us/step - loss: 0.0678 - accuracy: 0.9887 - val_loss: 1.5652 - val_accuracy: 0.5132
Epoch 223/1000
177/177 [==============================] - 0s 88us/step - loss: 0.0706 - accuracy: 0.9887 - val_loss: 1.5621 - val_accuracy: 0.5263
Epoch 224/1000
177/177 [==============================] - 0s 83us/step - loss: 0.0666 - accuracy: 0.9887 - val_loss: 1.5701 - val_accuracy: 0.5132
Epoch 225/1000
177/177 [==============================] - 0s 107us/step - loss: 0.0692 - accuracy: 0.9887 - val_loss: 1.5676 - val_accuracy: 0.5263
Epoch 226/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0670 - accuracy: 0.9887 - val_loss: 1.5755 - val_accuracy: 0.5263
Epoch 227/1000
177/177 [==============================] - 0s 111us/step - loss: 0.0663 - accuracy: 0.9887 - val_loss: 1.5850 - val_accuracy: 0.5263
Epoch 228/1000
177/177 [==============================] - 0s 108us/step - loss: 0.0666 - accuracy: 0.9887 - val_loss: 1.5871 - val_

177/177 [==============================] - 0s 120us/step - loss: 0.0357 - accuracy: 0.9944 - val_loss: 1.9064 - val_accuracy: 0.5263
Epoch 333/1000
177/177 [==============================] - 0s 107us/step - loss: 0.0358 - accuracy: 0.9944 - val_loss: 1.9252 - val_accuracy: 0.5263
Epoch 334/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0355 - accuracy: 0.9944 - val_loss: 1.9206 - val_accuracy: 0.5263
Epoch 335/1000
177/177 [==============================] - 0s 104us/step - loss: 0.0350 - accuracy: 0.9944 - val_loss: 1.9187 - val_accuracy: 0.5263
Epoch 336/1000
177/177 [==============================] - 0s 124us/step - loss: 0.0348 - accuracy: 0.9944 - val_loss: 1.9198 - val_accuracy: 0.5263
Epoch 337/1000
177/177 [==============================] - 0s 100us/step - loss: 0.0342 - accuracy: 0.9944 - val_loss: 1.9216 - val_accuracy: 0.5263
Epoch 338/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0350 - accuracy: 0.9887 - val_loss: 1.9238 - va

177/177 [==============================] - 0s 91us/step - loss: 0.0230 - accuracy: 0.9944 - val_loss: 2.2325 - val_accuracy: 0.5395
Epoch 443/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0226 - accuracy: 0.9944 - val_loss: 2.2308 - val_accuracy: 0.5395
Epoch 444/1000
177/177 [==============================] - 0s 91us/step - loss: 0.0229 - accuracy: 0.9944 - val_loss: 2.2236 - val_accuracy: 0.5395
Epoch 445/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0226 - accuracy: 0.9944 - val_loss: 2.2306 - val_accuracy: 0.5395
Epoch 446/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0220 - accuracy: 0.9944 - val_loss: 2.2429 - val_accuracy: 0.5395
Epoch 447/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0230 - accuracy: 0.9944 - val_loss: 2.2396 - val_accuracy: 0.5395
Epoch 448/1000
177/177 [==============================] - 0s 109us/step - loss: 0.0225 - accuracy: 0.9944 - val_loss: 2.2424 - val_a

...

Epoch 609/1000
177/177 [==============================] - 0s 91us/step - loss: 0.0151 - accuracy: 0.9944 - val_loss: 2.6441 - val_accuracy: 0.5395
Epoch 610/1000
177/177 [==============================] - 0s 93us/step - loss: 0.0142 - accuracy: 0.9944 - val_loss: 2.6415 - val_accuracy: 0.5395
Epoch 611/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0142 - accuracy: 0.9944 - val_loss: 2.6424 - val_accuracy: 0.5395
Epoch 612/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0152 - accuracy: 0.9887 - val_loss: 2.6457 - val_accuracy: 0.5395
Epoch 613/1000
177/177 [==============================] - 0s 107us/step - loss: 0.0156 - accuracy: 0.9944 - val_loss: 2.6556 - val_accuracy: 0.5395
Epoch 614/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0147 - accuracy: 0.9944 - val_loss: 2.6515 - val_accuracy: 0.5395
Epoch 615/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0143 - accuracy: 0.9944 - val_loss: 

177/177 [==============================] - 0s 87us/step - loss: 0.0131 - accuracy: 0.9944 - val_loss: 2.8961 - val_accuracy: 0.5395
Epoch 720/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0134 - accuracy: 0.9944 - val_loss: 2.8862 - val_accuracy: 0.5395
Epoch 721/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0134 - accuracy: 0.9944 - val_loss: 2.8950 - val_accuracy: 0.5395
Epoch 722/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0122 - accuracy: 0.9944 - val_loss: 2.8691 - val_accuracy: 0.5395
Epoch 723/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0127 - accuracy: 0.9944 - val_loss: 2.8725 - val_accuracy: 0.5395
Epoch 724/1000
177/177 [==============================] - 0s 115us/step - loss: 0.0120 - accuracy: 0.9944 - val_loss: 2.8897 - val_accuracy: 0.5395
Epoch 725/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0120 - accuracy: 0.9944 - val_loss: 2.8949 - val_a

Epoch 775/1000
177/177 [==============================] - 0s 113us/step - loss: 0.0115 - accuracy: 0.9944 - val_loss: 2.9862 - val_accuracy: 0.5395
Epoch 776/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0132 - accuracy: 0.9944 - val_loss: 3.0078 - val_accuracy: 0.5395
Epoch 777/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0126 - accuracy: 0.9944 - val_loss: 2.9899 - val_accuracy: 0.5395
Epoch 778/1000
177/177 [==============================] - 0s 91us/step - loss: 0.0117 - accuracy: 0.9944 - val_loss: 2.9824 - val_accuracy: 0.5395
Epoch 779/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0173 - accuracy: 0.9887 - val_loss: 2.9544 - val_accuracy: 0.5395
Epoch 780/1000
177/177 [==============================] - 0s 126us/step - loss: 0.0142 - accuracy: 0.9944 - val_loss: 2.9894 - val_accuracy: 0.5395
Epoch 781/1000
177/177 [==============================] - 0s 92us/step - loss: 0.0122 - accuracy: 0.9944 - val_loss:

Epoch 886/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0099 - accuracy: 0.9944 - val_loss: 3.1424 - val_accuracy: 0.5395
Epoch 887/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0117 - accuracy: 0.9944 - val_loss: 3.1449 - val_accuracy: 0.5395
Epoch 888/1000
177/177 [==============================] - 0s 97us/step - loss: 0.0103 - accuracy: 0.9944 - val_loss: 3.1647 - val_accuracy: 0.5395
Epoch 889/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0136 - accuracy: 0.9887 - val_loss: 3.1967 - val_accuracy: 0.5395
Epoch 890/1000
177/177 [==============================] - 0s 88us/step - loss: 0.0096 - accuracy: 0.9944 - val_loss: 3.1711 - val_accuracy: 0.5395
Epoch 891/1000
177/177 [==============================] - 0s 95us/step - loss: 0.0103 - accuracy: 0.9944 - val_loss: 3.1658 - val_accuracy: 0.5395
Epoch 892/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0113 - accuracy: 0.9944 - val_loss: 

177/177 [==============================] - 0s 90us/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 3.3686 - val_accuracy: 0.5395
Epoch 997/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0124 - accuracy: 0.9944 - val_loss: 3.3896 - val_accuracy: 0.5395
Epoch 998/1000
177/177 [==============================] - 0s 83us/step - loss: 0.0112 - accuracy: 0.9944 - val_loss: 3.3692 - val_accuracy: 0.5395
Epoch 999/1000
177/177 [==============================] - ETA: 0s - loss: 5.6541e-04 - accuracy: 1.00 - 0s 92us/step - loss: 0.0117 - accuracy: 0.9887 - val_loss: 3.3437 - val_accuracy: 0.5395
Epoch 1000/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0106 - accuracy: 0.9944 - val_loss: 3.3414 - val_accuracy: 0.5395


In [196]:
acc_test_sel3 = model_sel3.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel3*100))

76/76 [==============================] - 0s 97us/step
test accuracy: 52.63%


In [110]:
pred7 = model_sel3.predict_classes(X_sel_test)
pred7

array([1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 1, 0, 0, 2, 0, 2, 1,
       1, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 2, 1, 2, 0, 2, 1, 2, 0, 0, 2, 1,
       0, 0, 1, 1, 1, 2, 2, 2, 1, 1, 0, 1, 0, 0, 2, 2, 2, 1, 2, 2, 2, 2,
       1, 1, 0, 1, 0, 2, 1, 0, 1, 1])

In [111]:
dat7['pred'] = pred7
dat7

,strain,test,pred
120,NRS063,1,1
187,NRS219,2,2
96,GA27,2,1
53,CFBREBSa119,1,2
197,NRS233,2,2
...,...,...,...
47,CFBREBSa110,2,2
68,CFBRSa05,0,1
56,CFBREBSa123,0,0
231,NY360,1,1


In [112]:
proba7 = model_sel3.predict_proba(X_sel_test)
dat_proba7 = pd.DataFrame(proba7)

In [113]:
dat_proba7

,0,1,2
0,6.264517e-07,9.999994e-01,3.025620e-08
1,2.892338e-11,5.078725e-09,1.000000e+00
2,9.716029e-04,9.990278e-01,5.976599e-07
3,1.854527e-08,7.854669e-06,9.999921e-01
4,5.090574e-05,6.075555e-03,9.938736e-01
...,...,...,...
71,5.411382e-03,2.433093e-03,9.921555e-01
72,1.233912e-02,9.874413e-01,2.196671e-04
73,9.999692e-01,1.842665e-05,1.222419e-05
74,3.303899e-06,9.736476e-01,2.634919e-02


In [114]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [115]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p40p.csv", index = False,
         header=None)

In [200]:
hist_sel3 = model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 121us/step - loss: 0.0096 - accuracy: 0.9944 - val_loss: 3.8291 - val_accuracy: 0.5263
Epoch 2/1000
177/177 [==============================] - 0s 122us/step - loss: 0.0127 - accuracy: 0.9944 - val_loss: 3.8395 - val_accuracy: 0.5263
Epoch 3/1000
177/177 [==============================] - 0s 153us/step - loss: 0.0105 - accuracy: 0.9944 - val_loss: 3.8248 - val_accuracy: 0.5263
Epoch 4/1000
177/177 [==============================] - 0s 165us/step - loss: 0.0169 - accuracy: 0.9887 - val_loss: 3.7977 - val_accuracy: 0.5263
Epoch 5/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0139 - accuracy: 0.9944 - val_loss: 3.8117 - val_accuracy: 0.5263
Epoch 6/1000
177/177 [==============================] - 0s 134us/step - loss: 0.0097 - accuracy: 0.9944 - val_loss: 3.8346 - val_accuracy: 0.5263
Epoch 7/1000
177/177 [==============================] - 0s 162us/step - loss: 0

177/177 [==============================] - 0s 145us/step - loss: 0.0094 - accuracy: 0.9944 - val_loss: 4.0204 - val_accuracy: 0.5263
Epoch 112/1000
177/177 [==============================] - 0s 126us/step - loss: 0.0096 - accuracy: 0.9944 - val_loss: 4.0299 - val_accuracy: 0.5263
Epoch 113/1000
177/177 [==============================] - 0s 116us/step - loss: 0.0107 - accuracy: 0.9944 - val_loss: 4.0252 - val_accuracy: 0.5263
Epoch 114/1000
177/177 [==============================] - 0s 266us/step - loss: 0.0095 - accuracy: 0.9944 - val_loss: 4.0215 - val_accuracy: 0.5263
Epoch 115/1000
177/177 [==============================] - 0s 128us/step - loss: 0.0096 - accuracy: 0.9944 - val_loss: 4.0148 - val_accuracy: 0.5263
Epoch 116/1000
177/177 [==============================] - 0s 191us/step - loss: 0.0094 - accuracy: 0.9944 - val_loss: 4.0019 - val_accuracy: 0.5395
Epoch 117/1000
177/177 [==============================] - 0s 146us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 3.9996 - 

177/177 [==============================] - 0s 664us/step - loss: 0.0104 - accuracy: 0.9944 - val_loss: 4.2185 - val_accuracy: 0.5395
Epoch 222/1000
177/177 [==============================] - 0s 202us/step - loss: 0.0100 - accuracy: 0.9887 - val_loss: 4.2174 - val_accuracy: 0.5395
Epoch 223/1000
177/177 [==============================] - 0s 145us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 4.2388 - val_accuracy: 0.5395
Epoch 224/1000
177/177 [==============================] - 0s 341us/step - loss: 0.0115 - accuracy: 0.9944 - val_loss: 4.2259 - val_accuracy: 0.5395
Epoch 225/1000
177/177 [==============================] - 0s 267us/step - loss: 0.0103 - accuracy: 0.9944 - val_loss: 4.2251 - val_accuracy: 0.5395
Epoch 226/1000
177/177 [==============================] - 0s 357us/step - loss: 0.0109 - accuracy: 0.9944 - val_loss: 4.2416 - val_accuracy: 0.5263
Epoch 227/1000
177/177 [==============================] - 0s 184us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 4.2321 - 

177/177 [==============================] - 0s 109us/step - loss: 0.0104 - accuracy: 0.9887 - val_loss: 4.4037 - val_accuracy: 0.5395
Epoch 332/1000
177/177 [==============================] - 0s 109us/step - loss: 0.0083 - accuracy: 0.9944 - val_loss: 4.4269 - val_accuracy: 0.5395
Epoch 333/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0120 - accuracy: 0.9944 - val_loss: 4.4363 - val_accuracy: 0.5263
Epoch 334/1000
177/177 [==============================] - 0s 122us/step - loss: 0.0069 - accuracy: 0.9944 - val_loss: 4.4091 - val_accuracy: 0.5263
Epoch 335/1000
177/177 [==============================] - 0s 569us/step - loss: 0.0135 - accuracy: 0.9944 - val_loss: 4.3739 - val_accuracy: 0.5395
Epoch 336/1000
177/177 [==============================] - 0s 252us/step - loss: 0.0135 - accuracy: 0.9944 - val_loss: 4.3662 - val_accuracy: 0.5395
Epoch 337/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0087 - accuracy: 0.9944 - val_loss: 4.3858 - 

177/177 [==============================] - 0s 102us/step - loss: 0.0097 - accuracy: 0.9944 - val_loss: 4.5562 - val_accuracy: 0.5395
Epoch 442/1000
177/177 [==============================] - 0s 97us/step - loss: 0.0103 - accuracy: 0.9944 - val_loss: 4.5592 - val_accuracy: 0.5395
Epoch 443/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0105 - accuracy: 0.9887 - val_loss: 4.5858 - val_accuracy: 0.5395
Epoch 444/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0095 - accuracy: 0.9944 - val_loss: 4.5820 - val_accuracy: 0.5395
Epoch 445/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0098 - accuracy: 0.9944 - val_loss: 4.5785 - val_accuracy: 0.5395
Epoch 446/1000
177/177 [==============================] - 0s 128us/step - loss: 0.0085 - accuracy: 0.9944 - val_loss: 4.5689 - val_accuracy: 0.5395
Epoch 447/1000
177/177 [==============================] - 0s 115us/step - loss: 0.0116 - accuracy: 0.9944 - val_loss: 4.5567 - va

Epoch 551/1000
177/177 [==============================] - 0s 248us/step - loss: 0.0118 - accuracy: 0.9944 - val_loss: 4.7049 - val_accuracy: 0.5263
Epoch 552/1000
177/177 [==============================] - 0s 322us/step - loss: 0.0129 - accuracy: 0.9944 - val_loss: 4.7191 - val_accuracy: 0.5395
Epoch 553/1000
177/177 [==============================] - 0s 268us/step - loss: 0.0095 - accuracy: 0.9944 - val_loss: 4.7411 - val_accuracy: 0.5263
Epoch 554/1000
177/177 [==============================] - 0s 147us/step - loss: 0.0106 - accuracy: 0.9944 - val_loss: 4.7457 - val_accuracy: 0.5263
Epoch 555/1000
177/177 [==============================] - 0s 147us/step - loss: 0.0091 - accuracy: 0.9944 - val_loss: 4.7307 - val_accuracy: 0.5395
Epoch 556/1000
177/177 [==============================] - 0s 157us/step - loss: 0.0085 - accuracy: 0.9887 - val_loss: 4.7253 - val_accuracy: 0.5263
Epoch 557/1000
177/177 [==============================] - 0s 536us/step - loss: 0.0093 - accuracy: 0.9944 - val_

177/177 [==============================] - 0s 134us/step - loss: 0.0094 - accuracy: 0.9944 - val_loss: 4.9062 - val_accuracy: 0.5132
Epoch 662/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0088 - accuracy: 0.9944 - val_loss: 4.9009 - val_accuracy: 0.5132
Epoch 663/1000
177/177 [==============================] - 0s 129us/step - loss: 0.0087 - accuracy: 0.9944 - val_loss: 4.9077 - val_accuracy: 0.5132
Epoch 664/1000
177/177 [==============================] - 0s 167us/step - loss: 0.0084 - accuracy: 0.9944 - val_loss: 4.9078 - val_accuracy: 0.5132
Epoch 665/1000
177/177 [==============================] - 0s 241us/step - loss: 0.0091 - accuracy: 0.9944 - val_loss: 4.9058 - val_accuracy: 0.5132
Epoch 666/1000
177/177 [==============================] - 0s 215us/step - loss: 0.0085 - accuracy: 0.9944 - val_loss: 4.9013 - val_accuracy: 0.5132
Epoch 667/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0095 - accuracy: 0.9887 - val_loss: 4.9057 - 

177/177 [==============================] - 0s 127us/step - loss: 0.0084 - accuracy: 0.9944 - val_loss: 5.1357 - val_accuracy: 0.5132
Epoch 772/1000
177/177 [==============================] - 0s 111us/step - loss: 0.0091 - accuracy: 0.9944 - val_loss: 5.1423 - val_accuracy: 0.5132
Epoch 773/1000
177/177 [==============================] - 0s 140us/step - loss: 0.0089 - accuracy: 0.9944 - val_loss: 5.1514 - val_accuracy: 0.5132
Epoch 774/1000
177/177 [==============================] - 0s 141us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 5.1552 - val_accuracy: 0.5132
Epoch 775/1000
177/177 [==============================] - 0s 344us/step - loss: 0.0103 - accuracy: 0.9944 - val_loss: 5.1267 - val_accuracy: 0.5000
Epoch 776/1000
177/177 [==============================] - 0s 450us/step - loss: 0.0115 - accuracy: 0.9944 - val_loss: 5.1160 - val_accuracy: 0.5000
Epoch 777/1000
177/177 [==============================] - 0s 90us/step - loss: 0.0097 - accuracy: 0.9944 - val_loss: 5.1588 - v

177/177 [==============================] - 0s 106us/step - loss: 0.0084 - accuracy: 0.9944 - val_loss: 5.2929 - val_accuracy: 0.5132
Epoch 882/1000
177/177 [==============================] - 0s 122us/step - loss: 0.0090 - accuracy: 0.9944 - val_loss: 5.3013 - val_accuracy: 0.5132
Epoch 883/1000
177/177 [==============================] - 0s 106us/step - loss: 0.0094 - accuracy: 0.9944 - val_loss: 5.2921 - val_accuracy: 0.5132
Epoch 884/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0082 - accuracy: 0.9944 - val_loss: 5.2808 - val_accuracy: 0.5132
Epoch 885/1000
177/177 [==============================] - 0s 127us/step - loss: 0.0088 - accuracy: 0.9944 - val_loss: 5.2749 - val_accuracy: 0.5000
Epoch 886/1000
177/177 [==============================] - 0s 209us/step - loss: 0.0092 - accuracy: 0.9944 - val_loss: 5.2820 - val_accuracy: 0.5132
Epoch 887/1000
177/177 [==============================] - 0s 178us/step - loss: 0.0082 - accuracy: 0.9944 - val_loss: 5.3071 - 

177/177 [==============================] - 0s 206us/step - loss: 0.0099 - accuracy: 0.9944 - val_loss: 5.4886 - val_accuracy: 0.5000
Epoch 992/1000
177/177 [==============================] - 0s 176us/step - loss: 0.0099 - accuracy: 0.9887 - val_loss: 5.4652 - val_accuracy: 0.5000
Epoch 993/1000
177/177 [==============================] - 0s 174us/step - loss: 0.0099 - accuracy: 0.9887 - val_loss: 5.4748 - val_accuracy: 0.5000
Epoch 994/1000
177/177 [==============================] - 0s 169us/step - loss: 0.0116 - accuracy: 0.9944 - val_loss: 5.4386 - val_accuracy: 0.5000
Epoch 995/1000
177/177 [==============================] - 0s 207us/step - loss: 0.0104 - accuracy: 0.9944 - val_loss: 5.4485 - val_accuracy: 0.5132
Epoch 996/1000
177/177 [==============================] - 0s 188us/step - loss: 0.0101 - accuracy: 0.9944 - val_loss: 5.5029 - val_accuracy: 0.5132
Epoch 997/1000
177/177 [==============================] - 0s 159us/step - loss: 0.0121 - accuracy: 0.9887 - val_loss: 5.4701 - 

In [201]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel3.history['accuracy'])*100))

train accuracy: 99.32%


In [98]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [99]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS271,1,0,0.854929,0.144940,0.000130
1,p0017kpresabs_qual,SR2852,2,2,0.000001,0.000075,0.999923
2,p0017kpresabs_qual,CA39,2,0,0.959992,0.013406,0.026602
3,p0017kpresabs_qual,NRS266,1,2,0.000018,0.000003,0.999980
4,p0017kpresabs_qual,NY356,2,2,0.009156,0.411991,0.578852
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa110,2,2,0.044135,0.010244,0.945621
604,p0040presabsSTCC_qual,CFBRSa05,0,2,0.145032,0.408530,0.446439
605,p0040presabsSTCC_qual,CFBREBSa123,0,0,0.994623,0.004152,0.001224
606,p0040presabsSTCC_qual,NY360,1,1,0.000795,0.948566,0.050639


In [100]:
y_prob7 = df_proba7[df_proba7['phage']=='p0040presabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[6.26451650e-07, 9.99999400e-01, 3.02561960e-08],
       [2.89233830e-11, 5.07872500e-09, 1.00000000e+00],
       [9.71602900e-04, 9.99027850e-01, 5.97659950e-07],
       [1.85452670e-08, 7.85466900e-06, 9.99992130e-01],
       [5.09057350e-05, 6.07555450e-03, 9.93873600e-01],
       [1.76100700e-02, 9.82363460e-01, 2.65074120e-05],
       [4.17064130e-02, 9.58200340e-01, 9.32146360e-05],
       [3.77792630e-01, 3.39194100e-03, 6.18815400e-01],
       [2.19081390e-01, 3.48196480e-02, 7.46099000e-01],
       [5.41138230e-03, 2.43309330e-03, 9.92155500e-01],
       [4.05266300e-03, 9.95947300e-01, 1.57739200e-10],
       [7.79511700e-04, 2.39005320e-06, 9.99218100e-01],
       [2.50673250e-14, 1.42828860e-09, 1.00000000e+00],
       [8.65873900e-08, 9.99958040e-01, 4.18719460e-05],
       [5.02680500e-08, 9.99975800e-01, 2.42077220e-05],
       [3.78547970e-08, 9.98661900e-01, 1.33808450e-03],
       [9.99988800e-01, 9.20111050e-06, 1.99948800e-06],
       [9.16032200e-01, 2.28910

In [101]:
ovo7 = rocauc_ovo(y_sel_test, y_prob7, average="macro", multi_class="ovo")
ovo7

0.7336291632439434

In [102]:
ovr7 = rocauc_ovr(y_sel_test, y_prob7, average="macro", multi_class="ovr")
ovr7

0.7336291632439434

In [103]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel)

In [104]:
dat8 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat8['test'] = y_sel_test

In [105]:
dat8

,strain,test
130,NRS103,2
92,EUH25,0
143,NRS148,2
107,NRS001,1
166,NRS192,0
...,...,...
139,NRS113,1
24,BCH-SA-09,2
133,NRS106,2
62,CFBREBSa131,2


In [106]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [107]:
model_sel4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [121]:
model_sel4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [122]:
model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 670us/step - loss: 1.1263 - accuracy: 0.3785 - val_loss: 1.0755 - val_accuracy: 0.4605
Epoch 2/1000
177/177 [==============================] - 0s 65us/step - loss: 1.0710 - accuracy: 0.4237 - val_loss: 1.0380 - val_accuracy: 0.5132
Epoch 3/1000
177/177 [==============================] - 0s 70us/step - loss: 1.0264 - accuracy: 0.4915 - val_loss: 1.0164 - val_accuracy: 0.4868
Epoch 4/1000
177/177 [==============================] - 0s 90us/step - loss: 0.9988 - accuracy: 0.5198 - val_loss: 1.0030 - val_accuracy: 0.4868
Epoch 5/1000
177/177 [==============================] - 0s 115us/step - loss: 0.9767 - accuracy: 0.5141 - val_loss: 0.9934 - val_accuracy: 0.5000
Epoch 6/1000
177/177 [==============================] - 0s 98us/step - loss: 0.9548 - accuracy: 0.5367 - val_loss: 0.9875 - val_accuracy: 0.5000
Epoch 7/1000
177/177 [==============================] - 0s 89us/step - loss: 0.9379

177/177 [==============================] - 0s 68us/step - loss: 0.3083 - accuracy: 0.9153 - val_loss: 0.9774 - val_accuracy: 0.5132
Epoch 113/1000
177/177 [==============================] - 0s 123us/step - loss: 0.3059 - accuracy: 0.9096 - val_loss: 0.9784 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 94us/step - loss: 0.3035 - accuracy: 0.9096 - val_loss: 0.9843 - val_accuracy: 0.5132
Epoch 115/1000
177/177 [==============================] - 0s 65us/step - loss: 0.3031 - accuracy: 0.9040 - val_loss: 0.9847 - val_accuracy: 0.5263
Epoch 116/1000
177/177 [==============================] - 0s 77us/step - loss: 0.2999 - accuracy: 0.9040 - val_loss: 0.9881 - val_accuracy: 0.5395
Epoch 117/1000
177/177 [==============================] - 0s 77us/step - loss: 0.2971 - accuracy: 0.9153 - val_loss: 0.9864 - val_accuracy: 0.5132
Epoch 118/1000
177/177 [==============================] - 0s 111us/step - loss: 0.2939 - accuracy: 0.9096 - val_loss: 0.9816 - val_a

Epoch 168/1000
177/177 [==============================] - 0s 83us/step - loss: 0.2077 - accuracy: 0.9435 - val_loss: 1.0814 - val_accuracy: 0.5395
Epoch 169/1000
177/177 [==============================] - 0s 73us/step - loss: 0.2071 - accuracy: 0.9435 - val_loss: 1.0820 - val_accuracy: 0.5395
Epoch 170/1000
177/177 [==============================] - 0s 67us/step - loss: 0.2045 - accuracy: 0.9435 - val_loss: 1.0825 - val_accuracy: 0.5395
Epoch 171/1000
177/177 [==============================] - 0s 112us/step - loss: 0.2042 - accuracy: 0.9435 - val_loss: 1.0877 - val_accuracy: 0.5395
Epoch 172/1000
177/177 [==============================] - 0s 87us/step - loss: 0.2023 - accuracy: 0.9435 - val_loss: 1.0906 - val_accuracy: 0.5263
Epoch 173/1000
177/177 [==============================] - 0s 88us/step - loss: 0.2012 - accuracy: 0.9435 - val_loss: 1.0941 - val_accuracy: 0.5263
Epoch 174/1000
177/177 [==============================] - 0s 70us/step - loss: 0.2002 - accuracy: 0.9379 - val_loss: 

Epoch 279/1000
177/177 [==============================] - 0s 86us/step - loss: 0.1190 - accuracy: 0.9774 - val_loss: 1.3598 - val_accuracy: 0.5658
Epoch 280/1000
177/177 [==============================] - 0s 63us/step - loss: 0.1190 - accuracy: 0.9774 - val_loss: 1.3583 - val_accuracy: 0.5658
Epoch 281/1000
177/177 [==============================] - 0s 73us/step - loss: 0.1188 - accuracy: 0.9774 - val_loss: 1.3657 - val_accuracy: 0.5658
Epoch 282/1000
177/177 [==============================] - 0s 73us/step - loss: 0.1181 - accuracy: 0.9774 - val_loss: 1.3653 - val_accuracy: 0.5658
Epoch 283/1000
177/177 [==============================] - 0s 73us/step - loss: 0.1174 - accuracy: 0.9774 - val_loss: 1.3720 - val_accuracy: 0.5658
Epoch 284/1000
177/177 [==============================] - 0s 126us/step - loss: 0.1171 - accuracy: 0.9774 - val_loss: 1.3734 - val_accuracy: 0.5658
Epoch 285/1000
177/177 [==============================] - 0s 81us/step - loss: 0.1169 - accuracy: 0.9774 - val_loss: 

Epoch 335/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0991 - accuracy: 0.9774 - val_loss: 1.4759 - val_accuracy: 0.5395
Epoch 336/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0987 - accuracy: 0.9774 - val_loss: 1.4852 - val_accuracy: 0.5395
Epoch 337/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0983 - accuracy: 0.9774 - val_loss: 1.4859 - val_accuracy: 0.5395
Epoch 338/1000
177/177 [==============================] - 0s 93us/step - loss: 0.0982 - accuracy: 0.9774 - val_loss: 1.4856 - val_accuracy: 0.5395
Epoch 339/1000
177/177 [==============================] - 0s 56us/step - loss: 0.0976 - accuracy: 0.9774 - val_loss: 1.4917 - val_accuracy: 0.5395
Epoch 340/1000
177/177 [==============================] - 0s 59us/step - loss: 0.0975 - accuracy: 0.9774 - val_loss: 1.4963 - val_accuracy: 0.5395
Epoch 341/1000
177/177 [==============================] - 0s 69us/step - loss: 0.0973 - accuracy: 0.9774 - val_loss: 1

...

Epoch 558/1000
177/177 [==============================] - 0s 67us/step - loss: 0.0602 - accuracy: 0.9831 - val_loss: 1.8612 - val_accuracy: 0.5263
Epoch 559/1000
177/177 [==============================] - 0s 55us/step - loss: 0.0607 - accuracy: 0.9831 - val_loss: 1.8628 - val_accuracy: 0.5395
Epoch 560/1000
177/177 [==============================] - 0s 134us/step - loss: 0.0598 - accuracy: 0.9831 - val_loss: 1.8674 - val_accuracy: 0.5263
Epoch 561/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0595 - accuracy: 0.9831 - val_loss: 1.8704 - val_accuracy: 0.5263
Epoch 562/1000
177/177 [==============================] - 0s 61us/step - loss: 0.0605 - accuracy: 0.9831 - val_loss: 1.8642 - val_accuracy: 0.5395
Epoch 563/1000
177/177 [==============================] - 0s 61us/step - loss: 0.0593 - accuracy: 0.9774 - val_loss: 1.8708 - val_accuracy: 0.5395
Epoch 564/1000
177/177 [==============================] - 0s 59us/step - loss: 0.0595 - accuracy: 0.9774 - val_loss: 

Epoch 669/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0507 - accuracy: 0.9831 - val_loss: 2.0304 - val_accuracy: 0.5395
Epoch 670/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0513 - accuracy: 0.9831 - val_loss: 2.0365 - val_accuracy: 0.5395
Epoch 671/1000
177/177 [==============================] - 0s 71us/step - loss: 0.0511 - accuracy: 0.9831 - val_loss: 2.0227 - val_accuracy: 0.5395
Epoch 672/1000
177/177 [==============================] - 0s 69us/step - loss: 0.0521 - accuracy: 0.9831 - val_loss: 2.0164 - val_accuracy: 0.5395
Epoch 673/1000
177/177 [==============================] - 0s 67us/step - loss: 0.0507 - accuracy: 0.9831 - val_loss: 2.0274 - val_accuracy: 0.5395
Epoch 674/1000
177/177 [==============================] - 0s 60us/step - loss: 0.0512 - accuracy: 0.9831 - val_loss: 2.0417 - val_accuracy: 0.5395
Epoch 675/1000
177/177 [==============================] - 0s 50us/step - loss: 0.0509 - accuracy: 0.9831 - val_loss: 2

Epoch 780/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0451 - accuracy: 0.9831 - val_loss: 2.1590 - val_accuracy: 0.5526
Epoch 781/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0451 - accuracy: 0.9831 - val_loss: 2.1595 - val_accuracy: 0.5395
Epoch 782/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0450 - accuracy: 0.9831 - val_loss: 2.1721 - val_accuracy: 0.5526
Epoch 783/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0447 - accuracy: 0.9831 - val_loss: 2.1706 - val_accuracy: 0.5526
Epoch 784/1000
177/177 [==============================] - 0s 82us/step - loss: 0.0445 - accuracy: 0.9831 - val_loss: 2.1724 - val_accuracy: 0.5526
Epoch 785/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0442 - accuracy: 0.9831 - val_loss: 2.1705 - val_accuracy: 0.5395
Epoch 786/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0445 - accuracy: 0.9831 - val_loss:

177/177 [==============================] - 0s 70us/step - loss: 0.0416 - accuracy: 0.9831 - val_loss: 2.2958 - val_accuracy: 0.5395
Epoch 891/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0414 - accuracy: 0.9774 - val_loss: 2.3024 - val_accuracy: 0.5395
Epoch 892/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0410 - accuracy: 0.9831 - val_loss: 2.3044 - val_accuracy: 0.5395
Epoch 893/1000
177/177 [==============================] - 0s 60us/step - loss: 0.0408 - accuracy: 0.9774 - val_loss: 2.3108 - val_accuracy: 0.5526
Epoch 894/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0403 - accuracy: 0.9831 - val_loss: 2.3082 - val_accuracy: 0.5395
Epoch 895/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0405 - accuracy: 0.9831 - val_loss: 2.3084 - val_accuracy: 0.5395
Epoch 896/1000
177/177 [==============================] - 0s 111us/step - loss: 0.0406 - accuracy: 0.9831 - val_loss: 2.3044 - val_a

Epoch 946/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0393 - accuracy: 0.9831 - val_loss: 2.3880 - val_accuracy: 0.5526
Epoch 947/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0388 - accuracy: 0.9831 - val_loss: 2.3757 - val_accuracy: 0.5395
Epoch 948/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0390 - accuracy: 0.9831 - val_loss: 2.3571 - val_accuracy: 0.5395
Epoch 949/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0393 - accuracy: 0.9831 - val_loss: 2.3557 - val_accuracy: 0.5395
Epoch 950/1000
177/177 [==============================] - 0s 117us/step - loss: 0.0393 - accuracy: 0.9831 - val_loss: 2.3569 - val_accuracy: 0.5395
Epoch 951/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0387 - accuracy: 0.9774 - val_loss: 2.3716 - val_accuracy: 0.5395
Epoch 952/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0386 - accuracy: 0.9831 - val_loss: 

In [217]:
acc_test_sel4 = model_sel4.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel4*100))

76/76 [==============================] - 0s 108us/step
over-sampling test accuracy: 53.95%


In [123]:
pred8 = model_sel4.predict_classes(X_sel_test)
pred8

array([2, 0, 2, 1, 0, 2, 1, 0, 2, 1, 2, 1, 1, 0, 2, 1, 2, 1, 2, 0, 2, 0,
       2, 2, 1, 1, 1, 1, 2, 1, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1,
       1, 0, 2, 2, 0, 2, 2, 1, 1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 2, 2, 1, 1,
       1, 1, 2, 1, 0, 2, 1, 1, 2, 0])

In [124]:
dat8['pred'] = pred8
dat8

,strain,test,pred
130,NRS103,2,2
92,EUH25,0,0
143,NRS148,2,2
107,NRS001,1,1
166,NRS192,0,0
...,...,...,...
139,NRS113,1,2
24,BCH-SA-09,2,1
133,NRS106,2,1
62,CFBREBSa131,2,2


In [125]:
proba8 = model_sel4.predict_proba(X_sel_test)
dat_proba8 = pd.DataFrame(proba8)

In [126]:
dat_proba8

,0,1,2
0,2.250468e-03,0.047139,9.506108e-01
1,9.997689e-01,0.000185,4.654982e-05
2,3.825874e-08,0.000048,9.999522e-01
3,3.019064e-03,0.995714,1.266727e-03
4,6.366351e-01,0.081548,2.818173e-01
...,...,...,...
71,4.269575e-09,0.001550,9.984504e-01
72,1.010665e-08,1.000000,2.646191e-08
73,2.405621e-02,0.975909,3.491244e-05
74,7.060066e-07,0.003368,9.966313e-01


In [127]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [128]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p40p.csv", index = False,
         header=None)

In [221]:
hist_sel4 = model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 86us/step - loss: 0.0351 - accuracy: 0.9831 - val_loss: 2.6088 - val_accuracy: 0.5395
Epoch 2/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0350 - accuracy: 0.9831 - val_loss: 2.6090 - val_accuracy: 0.5395
Epoch 3/1000
177/177 [==============================] - 0s 145us/step - loss: 0.0352 - accuracy: 0.9774 - val_loss: 2.6137 - val_accuracy: 0.5395
Epoch 4/1000
177/177 [==============================] - 0s 116us/step - loss: 0.0350 - accuracy: 0.9831 - val_loss: 2.6210 - val_accuracy: 0.5395
Epoch 5/1000
177/177 [==============================] - 0s 111us/step - loss: 0.0348 - accuracy: 0.9831 - val_loss: 2.6325 - val_accuracy: 0.5132
Epoch 6/1000
177/177 [==============================] - 0s 126us/step - loss: 0.0355 - accuracy: 0.9831 - val_loss: 2.6265 - val_accuracy: 0.5132
Epoch 7/1000
177/177 [==============================] - 0s 86us/step - loss: 0.03

177/177 [==============================] - 0s 80us/step - loss: 0.0338 - accuracy: 0.9831 - val_loss: 2.7000 - val_accuracy: 0.5395
Epoch 112/1000
177/177 [==============================] - 0s 89us/step - loss: 0.0354 - accuracy: 0.9774 - val_loss: 2.7281 - val_accuracy: 0.5132
Epoch 113/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0344 - accuracy: 0.9831 - val_loss: 2.7299 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 66us/step - loss: 0.0348 - accuracy: 0.9831 - val_loss: 2.7130 - val_accuracy: 0.5263
Epoch 115/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0336 - accuracy: 0.9831 - val_loss: 2.7017 - val_accuracy: 0.5263
Epoch 116/1000
177/177 [==============================] - 0s 60us/step - loss: 0.0332 - accuracy: 0.9831 - val_loss: 2.6971 - val_accuracy: 0.5395
Epoch 117/1000
177/177 [==============================] - 0s 69us/step - loss: 0.0336 - accuracy: 0.9831 - val_loss: 2.6971 - val_acc

Epoch 167/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0340 - accuracy: 0.9774 - val_loss: 2.7664 - val_accuracy: 0.5263
Epoch 168/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0338 - accuracy: 0.9831 - val_loss: 2.7726 - val_accuracy: 0.5263
Epoch 169/1000
177/177 [==============================] - 0s 57us/step - loss: 0.0339 - accuracy: 0.9831 - val_loss: 2.7537 - val_accuracy: 0.5395
Epoch 170/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0329 - accuracy: 0.9831 - val_loss: 2.7515 - val_accuracy: 0.5395
Epoch 171/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0344 - accuracy: 0.9774 - val_loss: 2.7592 - val_accuracy: 0.5263
Epoch 172/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0332 - accuracy: 0.9831 - val_loss: 2.7635 - val_accuracy: 0.5395
Epoch 173/1000
177/177 [==============================] - 0s 165us/step - loss: 0.0334 - accuracy: 0.9831 - val_loss: 

177/177 [==============================] - 0s 66us/step - loss: 0.0334 - accuracy: 0.9774 - val_loss: 2.8405 - val_accuracy: 0.5395
Epoch 278/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0339 - accuracy: 0.9774 - val_loss: 2.8406 - val_accuracy: 0.5395
Epoch 279/1000
177/177 [==============================] - 0s 61us/step - loss: 0.0334 - accuracy: 0.9831 - val_loss: 2.8510 - val_accuracy: 0.5395
Epoch 280/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0322 - accuracy: 0.9831 - val_loss: 2.8393 - val_accuracy: 0.5395
Epoch 281/1000
177/177 [==============================] - 0s 332us/step - loss: 0.0334 - accuracy: 0.9774 - val_loss: 2.8257 - val_accuracy: 0.5395
Epoch 282/1000
177/177 [==============================] - 0s 264us/step - loss: 0.0327 - accuracy: 0.9831 - val_loss: 2.8305 - val_accuracy: 0.5395
Epoch 283/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0327 - accuracy: 0.9831 - val_loss: 2.8326 - val_a

Epoch 333/1000
177/177 [==============================] - 0s 147us/step - loss: 0.0337 - accuracy: 0.9831 - val_loss: 2.8460 - val_accuracy: 0.5395
Epoch 334/1000
177/177 [==============================] - 0s 69us/step - loss: 0.0337 - accuracy: 0.9831 - val_loss: 2.8456 - val_accuracy: 0.5395
Epoch 335/1000
177/177 [==============================] - 0s 67us/step - loss: 0.0317 - accuracy: 0.9831 - val_loss: 2.8652 - val_accuracy: 0.5395
Epoch 336/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0312 - accuracy: 0.9887 - val_loss: 2.8915 - val_accuracy: 0.5263
Epoch 337/1000
177/177 [==============================] - 0s 83us/step - loss: 0.0343 - accuracy: 0.9831 - val_loss: 2.9296 - val_accuracy: 0.5000
Epoch 338/1000
177/177 [==============================] - 0s 83us/step - loss: 0.0350 - accuracy: 0.9831 - val_loss: 2.9327 - val_accuracy: 0.5000
Epoch 339/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0327 - accuracy: 0.9831 - val_loss: 

...

Epoch 555/1000
177/177 [==============================] - 0s 77us/step - loss: 0.0312 - accuracy: 0.9831 - val_loss: 3.0220 - val_accuracy: 0.5263
Epoch 556/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0312 - accuracy: 0.9831 - val_loss: 3.0269 - val_accuracy: 0.5263
Epoch 557/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0320 - accuracy: 0.9774 - val_loss: 3.0121 - val_accuracy: 0.5263
Epoch 558/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0320 - accuracy: 0.9831 - val_loss: 3.0072 - val_accuracy: 0.5263
Epoch 559/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0316 - accuracy: 0.9774 - val_loss: 3.0143 - val_accuracy: 0.5263
Epoch 560/1000
177/177 [==============================] - 0s 69us/step - loss: 0.0331 - accuracy: 0.9774 - val_loss: 3.0449 - val_accuracy: 0.5132
Epoch 561/1000
177/177 [==============================] - 0s 58us/step - loss: 0.0312 - accuracy: 0.9831 - val_loss: 3

Epoch 666/1000
177/177 [==============================] - 0s 115us/step - loss: 0.0313 - accuracy: 0.9831 - val_loss: 3.0853 - val_accuracy: 0.5395
Epoch 667/1000
177/177 [==============================] - 0s 190us/step - loss: 0.0322 - accuracy: 0.9831 - val_loss: 3.0744 - val_accuracy: 0.5395
Epoch 668/1000
177/177 [==============================] - 0s 159us/step - loss: 0.0317 - accuracy: 0.9831 - val_loss: 3.0874 - val_accuracy: 0.5395
Epoch 669/1000
177/177 [==============================] - 0s 125us/step - loss: 0.0313 - accuracy: 0.9774 - val_loss: 3.0883 - val_accuracy: 0.5263
Epoch 670/1000
177/177 [==============================] - 0s 142us/step - loss: 0.0313 - accuracy: 0.9774 - val_loss: 3.1014 - val_accuracy: 0.5263
Epoch 671/1000
177/177 [==============================] - 0s 165us/step - loss: 0.0309 - accuracy: 0.9774 - val_loss: 3.0961 - val_accuracy: 0.5395
Epoch 672/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0318 - accuracy: 0.9774 - val_

177/177 [==============================] - 0s 64us/step - loss: 0.0315 - accuracy: 0.9831 - val_loss: 3.1156 - val_accuracy: 0.5395
Epoch 777/1000
177/177 [==============================] - 0s 57us/step - loss: 0.0311 - accuracy: 0.9831 - val_loss: 3.1240 - val_accuracy: 0.5395
Epoch 778/1000
177/177 [==============================] - 0s 71us/step - loss: 0.0312 - accuracy: 0.9831 - val_loss: 3.1304 - val_accuracy: 0.5395
Epoch 779/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0309 - accuracy: 0.9831 - val_loss: 3.1409 - val_accuracy: 0.5395
Epoch 780/1000
177/177 [==============================] - 0s 141us/step - loss: 0.0303 - accuracy: 0.9831 - val_loss: 3.1538 - val_accuracy: 0.5395
Epoch 781/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0304 - accuracy: 0.9831 - val_loss: 3.1658 - val_accuracy: 0.5263
Epoch 782/1000
177/177 [==============================] - 0s 196us/step - loss: 0.0309 - accuracy: 0.9831 - val_loss: 3.1663 - val_a

Epoch 887/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0314 - accuracy: 0.9774 - val_loss: 3.2116 - val_accuracy: 0.5263
Epoch 888/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0309 - accuracy: 0.9774 - val_loss: 3.1954 - val_accuracy: 0.5395
Epoch 889/1000
177/177 [==============================] - 0s 65us/step - loss: 0.0309 - accuracy: 0.9774 - val_loss: 3.1839 - val_accuracy: 0.5263
Epoch 890/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0313 - accuracy: 0.9831 - val_loss: 3.1805 - val_accuracy: 0.5263
Epoch 891/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0310 - accuracy: 0.9774 - val_loss: 3.1817 - val_accuracy: 0.5395
Epoch 892/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0308 - accuracy: 0.9831 - val_loss: 3.1948 - val_accuracy: 0.5395
Epoch 893/1000
177/177 [==============================] - 0s 113us/step - loss: 0.0318 - accuracy: 0.9831 - val_loss: 

Epoch 998/1000
177/177 [==============================] - 0s 115us/step - loss: 0.0301 - accuracy: 0.9831 - val_loss: 3.2767 - val_accuracy: 0.5263
Epoch 999/1000
177/177 [==============================] - 0s 109us/step - loss: 0.0303 - accuracy: 0.9831 - val_loss: 3.2780 - val_accuracy: 0.5395
Epoch 1000/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0314 - accuracy: 0.9831 - val_loss: 3.2813 - val_accuracy: 0.5395


In [222]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel4.history['accuracy'])*100))

train accuracy: 98.15%


In [108]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [109]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS104,0,1,8.210638e-02,0.917809,8.477923e-05
1,p0017kpresabs_qual,NRS071,0,2,1.015229e-02,0.255327,7.345203e-01
2,p0017kpresabs_qual,NRS072,1,2,3.642946e-03,0.000002,9.963547e-01
3,p0017kpresabs_qual,BCH-SA-12,0,2,2.809318e-03,0.099188,8.980029e-01
4,p0017kpresabs_qual,CFBREBSa125,2,0,4.875667e-01,0.088657,4.237761e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS113,1,2,2.686909e-07,0.093121,9.068785e-01
604,p0040presabsSTCC_qual,BCH-SA-09,2,1,5.529492e-07,0.999990,9.559324e-06
605,p0040presabsSTCC_qual,NRS106,2,1,1.115902e-03,0.998816,6.852559e-05
606,p0040presabsSTCC_qual,CFBREBSa131,2,2,2.708040e-07,0.156768,8.432316e-01


In [110]:
y_prob8 = df_proba8[df_proba8['phage']=='p0040presabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[2.25046770e-03, 4.71387100e-02, 9.50610800e-01],
       [9.99768900e-01, 1.84529820e-04, 4.65498230e-05],
       [3.82587400e-08, 4.77981050e-05, 9.99952200e-01],
       [3.01906370e-03, 9.95714100e-01, 1.26672710e-03],
       [6.36635100e-01, 8.15476100e-02, 2.81817300e-01],
       [2.44267120e-03, 2.10779870e-02, 9.76479350e-01],
       [1.11265500e-07, 9.97339200e-01, 2.66063000e-03],
       [9.87931250e-01, 2.37006480e-03, 9.69873900e-03],
       [8.00082200e-08, 2.79098500e-01, 7.20901400e-01],
       [4.60323800e-02, 9.42734600e-01, 1.12330350e-02],
       [1.74381490e-06, 2.76652120e-03, 9.97231800e-01],
       [2.90968900e-07, 9.99999400e-01, 3.31779520e-07],
       [1.65304430e-05, 5.46076500e-01, 4.53907000e-01],
       [5.08789240e-01, 4.14343100e-01, 7.68677100e-02],
       [4.68910900e-10, 2.51337860e-05, 9.99974850e-01],
       [3.68158900e-04, 9.33981600e-01, 6.56502600e-02],
       [3.94966950e-04, 1.27771590e-01, 8.71833440e-01],
       [1.25011160e-01, 8.17092

In [111]:
ovo8 = rocauc_ovo(y_sel_test, y_prob8, average="macro", multi_class="ovo")
ovo8

0.7158469390813985

In [112]:
ovr8 = rocauc_ovr(y_sel_test, y_prob8, average="macro", multi_class="ovr")
ovr8

0.7158469390813985

In [113]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.6878091540038006

In [114]:
np.std(ovos2)

0.04023522036981266

In [115]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.6878091540038006

In [116]:
np.std(ovrs2)

0.04023522036981266

In [223]:
accs_l= [acc_test_sel, acc_test_sel2, acc_test_sel3, acc_test_sel4]

In [224]:
mean_l = np.mean(accs_l)
print('test accuracy mean after lasso: %.2f%%' % (mean_l*100))

test accuracy mean after lasso: 52.30%


In [225]:
std_l = np.std(accs_l)
print('test accuracy standard deviation after lasso:', std_l)

test accuracy standard deviation after lasso: 0.02996852595786627


In [226]:
accs_train_l = [np.mean(hist_sel.history['accuracy']), np.mean(hist_sel2.history['accuracy']), np.mean(hist_sel3.history['accuracy']),
             np.mean(hist_sel4.history['accuracy'])]

In [227]:
mean_train_l = np.mean(accs_train_l)
print('train accuracy mean after lasso: %.2f%%' % (mean_train_l*100))

train accuracy mean after lasso: 98.48%


In [228]:
std_train_l = np.std(accs_train_l)
print('train accuracy standard deviation after lasso:', std_train_l)

train accuracy standard deviation after lasso: 0.005120312
